In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Import Library

In [ ]:
import pandas as pd
import numpy as np
import csv
import string
import re #regex library

#import word tokenize & freqDist from NLTK
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords

#install dan import sastrawi package
!pip install Sastrawi
!pip install swifter
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter #mempercepat proses stemming

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 10.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 830.9/830.9 kB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.7 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.3.4-py3-none-any.whl size=16299 sha256=e6f377ed810b74d8ccc56100f4a48456809ff6043d39a0b77c8bd9d78bf74c83
  Stored in directory: /root/.cache/pip/wheels/6c/bd/3e/2d6afc9bc36c9975f8e4215a270bbac6580c4361ebd6bb2323
Successfully built swifter


# Load Data

In [ ]:
dataset= pd.read_csv('/content/Crawling_ulasan6.csv')
dataset

,content
0,Kenapa jaringan selalu tidak setabil padahal a...
1,Sudah login tp minta login lagi ketika klik li...
2,Saya tidak bisa melihat chat d hp saya.. Saya ...
3,Sangat bagus tingkat kan lagi fitur tampilannya.
4,Bagus
...,...
3582,Sangat bagus....
3583,Sudah dicoba berkali2. Kode tidak dirimkan ke ...
3584,Agak sebel jaringan sedikit jelek gak bisa dip...
3585,Mantap


In [ ]:
print(dataset.loc[[0]])

                                             content
0  Kenapa jaringan selalu tidak setabil padahal a...


#Text Preprocessing

In [ ]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
dataset['case_folding'] = dataset['content'].str.lower()


print(dataset.loc[[0]])

                                             content  \
0  Kenapa jaringan selalu tidak setabil padahal a...   

                                        case_folding  
0  kenapa jaringan selalu tidak setabil padahal a...  


In [ ]:
#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

dataset['number'] = dataset['case_folding'].apply(remove_number)

print(dataset.loc[[0]])

                                             content  \
0  Kenapa jaringan selalu tidak setabil padahal a...   

                                        case_folding  \
0  kenapa jaringan selalu tidak setabil padahal a...   

                                              number  
0  kenapa jaringan selalu tidak setabil padahal a...  


In [ ]:
#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

dataset['punctuation'] = dataset['number'].apply(remove_punctuation)

print(dataset.loc[[0]])

                                             content  \
0  Kenapa jaringan selalu tidak setabil padahal a...   

                                        case_folding  \
0  kenapa jaringan selalu tidak setabil padahal a...   

                                              number  \
0  kenapa jaringan selalu tidak setabil padahal a...   

                                         punctuation  
0  kenapa jaringan selalu tidak setabil padahal a...  


In [ ]:
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

dataset['no_emoticons'] = dataset['punctuation'].apply(remove_emojis)

In [ ]:
#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

dataset['whitespace_multiple'] = dataset['no_emoticons'].apply(remove_whitespace_multiple)

print(dataset.loc[[0]])

                                             content  \
0  Kenapa jaringan selalu tidak setabil padahal a...   

                                        case_folding  \
0  kenapa jaringan selalu tidak setabil padahal a...   

                                              number  \
0  kenapa jaringan selalu tidak setabil padahal a...   

                                         punctuation  \
0  kenapa jaringan selalu tidak setabil padahal a...   

                                        no_emoticons  \
0  kenapa jaringan selalu tidak setabil padahal a...   

                                 whitespace_multiple  
0  kenapa jaringan selalu tidak setabil padahal a...  


In [ ]:
#Tokenize
# NLTK word tokenize
def word_tokenize_wrapper(text):
    return word_tokenize(text)

dataset['review_tokens'] = dataset['whitespace_multiple'].apply(word_tokenize_wrapper)

print(dataset.loc[[0]])

                                             content  \
0  Kenapa jaringan selalu tidak setabil padahal a...   

                                        case_folding  \
0  kenapa jaringan selalu tidak setabil padahal a...   

                                              number  \
0  kenapa jaringan selalu tidak setabil padahal a...   

                                         punctuation  \
0  kenapa jaringan selalu tidak setabil padahal a...   

                                        no_emoticons  \
0  kenapa jaringan selalu tidak setabil padahal a...   

                                 whitespace_multiple  \
0  kenapa jaringan selalu tidak setabil padahal a...   

                                       review_tokens  
0  [kenapa, jaringan, selalu, tidak, setabil, pad...  


In [ ]:
#--------Normalizing-------
normalizad_word = pd.read_excel("/content/colloquial-id.xlsx")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1]

def normalized_term(document):
    result = []
    for term in document:
        if term in normalizad_word_dict:
            for word in normalizad_word_dict[term].split(' '):
                result.append(word)
        else:
            result.append(term)
    return result

dataset['review_clean'] = dataset['review_tokens'].apply(normalized_term)
print(dataset)

                                                content  \
0     Kenapa jaringan selalu tidak setabil padahal a...   
1     Sudah login tp minta login lagi ketika klik li...   
2     Saya tidak bisa melihat chat d hp saya.. Saya ...   
3     Sangat bagus  tingkat kan lagi fitur tampilannya.   
4                                                 Bagus   
...                                                 ...   
3582                                   Sangat bagus....   
3583  Sudah dicoba berkali2. Kode tidak dirimkan ke ...   
3584  Agak sebel jaringan sedikit jelek gak bisa dip...   
3585                                             Mantap   
3586  Apk ini sangat bagus  hanya saja terkadang sed...   

                                           case_folding  \
0     kenapa jaringan selalu tidak setabil padahal a...   
1     sudah login tp minta login lagi ketika klik li...   
2     saya tidak bisa melihat chat d hp saya.. saya ...   
3     sangat bagus  tingkat kan lagi fitur tampilannya.

In [ ]:
#Save hasil Text Preprocessing

dataset.to_csv("Text_Preprocessing6.csv")

In [ ]:
dataset

,content,case_folding,number,punctuation,no_emoticons,whitespace_multiple,review_tokens,review_clean
0,Kenapa jaringan selalu tidak setabil padahal a...,kenapa jaringan selalu tidak setabil padahal a...,kenapa jaringan selalu tidak setabil padahal a...,kenapa jaringan selalu tidak setabil padahal a...,kenapa jaringan selalu tidak setabil padahal a...,kenapa jaringan selalu tidak setabil padahal a...,"[kenapa, jaringan, selalu, tidak, setabil, pad...","[kenapa, jaringan, selalu, tidak, stabil, pada..."
1,Sudah login tp minta login lagi ketika klik li...,sudah login tp minta login lagi ketika klik li...,sudah login tp minta login lagi ketika klik li...,sudah login tp minta login lagi ketika klik li...,sudah login tp minta login lagi ketika klik li...,sudah login tp minta login lagi ketika klik li...,"[sudah, login, tp, minta, login, lagi, ketika,...","[sudah, masuk, tapi, meminta, masuk, lagi, ket..."
2,Saya tidak bisa melihat chat d hp saya.. Saya ...,saya tidak bisa melihat chat d hp saya.. saya ...,saya tidak bisa melihat chat d hp saya.. saya ...,saya tidak bisa melihat chat d hp saya saya su...,saya tidak bisa melihat chat d hp saya saya su...,saya tidak bisa melihat chat d hp saya saya su...,"[saya, tidak, bisa, melihat, chat, d, hp, saya...","[saya, tidak, bisa, melihat, chat, di, ponsel,..."
3,Sangat bagus tingkat kan lagi fitur tampilannya.,sangat bagus tingkat kan lagi fitur tampilannya.,sangat bagus tingkat kan lagi fitur tampilannya.,sangat bagus tingkat kan lagi fitur tampilannya,sangat bagus tingkat kan lagi fitur tampilannya,sangat bagus tingkat kan lagi fitur tampilannya,"[sangat, bagus, tingkat, kan, lagi, fitur, tam...","[sangat, bagus, tingkat, kan, lagi, fitur, tam..."
4,Bagus,bagus,bagus,bagus,bagus,bagus,[bagus],[bagus]
...,...,...,...,...,...,...,...,...
3582,Sangat bagus....,sangat bagus....,sangat bagus....,sangat bagus,sangat bagus,sangat bagus,"[sangat, bagus]","[sangat, bagus]"
3583,Sudah dicoba berkali2. Kode tidak dirimkan ke ...,sudah dicoba berkali2. kode tidak dirimkan ke ...,sudah dicoba berkali. kode tidak dirimkan ke n...,sudah dicoba berkali kode tidak dirimkan ke no...,sudah dicoba berkali kode tidak dirimkan ke no...,sudah dicoba berkali kode tidak dirimkan ke no...,"[sudah, dicoba, berkali, kode, tidak, dirimkan...","[sudah, dicoba, berkali, kode, tidak, kirimkan..."
3584,Agak sebel jaringan sedikit jelek gak bisa dip...,agak sebel jaringan sedikit jelek gak bisa dip...,agak sebel jaringan sedikit jelek gak bisa dip...,agak sebel jaringan sedikit jelek gak bisa dip...,agak sebel jaringan sedikit jelek gak bisa dip...,agak sebel jaringan sedikit jelek gak bisa dip...,"[agak, sebel, jaringan, sedikit, jelek, gak, b...","[agak, sebal, jaringan, sedikit, jelek, tidak,..."
3585,Mantap,mantap,mantap,mantap,mantap,mantap,[mantap],[mantap]


# Sentiment Scoring (Positive Negative)

Menggunakan kamus sentimen, kata negasi dan boosterwords

In [ ]:
import re
from collections import OrderedDict
import numpy as np

class sentistrength:
    def __init__(self, config=dict()):
        self.negasi = [line.replace('\n','') for line in open("/content/negatingword.txt").read().splitlines()]
        #create sentiment words dictionary
        self.sentiwords_txt = [line.replace('\n','').split(":") for line in open("/content/sentiwords_modif.txt").read().splitlines()]
        self.sentiwords_dict = OrderedDict()
        for term in self.sentiwords_txt:
            self.sentiwords_dict[term[0]] = int(term[1])
        #create boosterwords dictionary
        self.boosterwords_txt = [line.replace('\n','').split(":") for line in open("/content/boosterwords_id.txt").read().splitlines()]
        self.boosterwords_dict = OrderedDict()
        for term in self.boosterwords_txt:
            self.boosterwords_dict[term[0]] = int(term[1])
        self.negation_conf = config["negation"]
        self.booster_conf = config["booster"]
        self.mean_conf = False

    def senti(self,term):
        try:
            return self.sentiwords_dict[term]
        except:
            return 0

    def booster(self, term):
        try:
            return self.boosterwords_dict[term]
        except:
            return 0

    def cek_negationword(self, prev_term, prev_term2):
        #jika kata sebelumnya (index-1) adalah kata negasi, negasikan nilai -+nya
        if prev_term in self.negasi or prev_term2+" "+prev_term in self.negasi:
            # print prev_term
            self.score = -abs(self.score) if self.score>0 else abs(self.score)

    def cek_boosterword(self,term):
        booster_score = self.booster(term)
        if booster_score !=0 and self.score>0: self.score += booster_score
        if booster_score !=0 and self.score<0: self.score -= booster_score

    def cek_consecutive_term(self, prev_term):
        if self.prev_score>0 and self.score >=3: self.score+=1
        if self.prev_score<0 and self.score <=-3: self.score-=1

    def plural_to_singular(self, term):
        return re.sub(r'([A-Za-z]+)\-\1', r'\1',term)
    def classify(self):
        result = "positive"
        try:
            if self.mean_conf:
                mean_p = np.mean(self.mean_pos)
                mean_n = np.mean(self.mean_neg)
                print(mean_p, mean_n)
                if mean_p > mean_n:
                    result = "positive"
                elif mean_p < mean_n and not self.is_tanya:
                    result = "negative"
                elif mean_p < mean_n and self.is_tanya:
                    result = "positive"
            else:
                if abs(self.sentences_max_pos) > abs(self.sentences_max_neg):
                    result = "positive"
                elif abs(self.sentences_max_pos) < abs(self.sentences_max_neg):
                    result = "negative"
                elif abs(self.sentences_max_pos) == abs(self.sentences_max_neg):
                    result = "positive"
        except:
            print("error ",self.sentences_max_pos, self.sentences_max_neg)
        return result
    def cek_neutral_term(self,terms,i):
        if terms[i-1] in self.neutral_term or terms[i+1] in self.neutral_term: self.score=1

    def main(self,sentence):
        self.neutral_term = ['jika','kalau']
        sentences = sentence.split('.')
        self.sentences_max_neg = -1
        self.sentences_max_pos = 1
        self.sentences_score = []
        self.sentences_text = []
        for sentence in sentences:
            self.max_neg = -1
            self.max_pos = 1
            self.mean_neg = [1]
            self.mean_pos = [1]
            self.sentence_score=[]
            terms = sentence.split()
            # terms = re.split(r'[\s,.]',sentence)
            terms_length = len(terms)
            self.is_tanya = False
            self.sentence_text = ''
            # print self.max_pos, self.max_neg
            #SEMUA KALIMAT YANG MEMILIKI TANDA SERU MEMILIKI +ve minimal 2
            self.prev_score = 0
            self.pre_max_pos = []
            self.pre_max_neg = []
            for i,term in enumerate(terms):
                # repeated_term = ''
                is_extra_char = False
                plural = ''
                self.score = 0
                # if re.search(r'[A-Za-z\-.]+',term):
                # print term
                if re.search(r'([A-Za-z])\1{3,}',term):
                    is_extra_char = True
                    # repeated_term =term
                if re.search(r'([A-Za-z]+)\-\1',term):
                    plural = term
                    term = self.plural_to_singular(term)
                #GET SENTI SCORE#
                self.score = self.senti(term)
                # print "senti score",term, self.score

                #NEGATION HANDLER#
                if self.negation_conf and self.score !=0 and i>0:self.cek_negationword(terms[i-1],terms[i-2])
                # print  "negation score",term, self.score

                #BOOSTERWORD HANDLER#
                if self.booster_conf and self.score !=0 and i>0 and i<=(terms_length-1):self.cek_boosterword(terms[i-1])
                if self.booster_conf and self.score !=0 and i>=0 and i<(terms_length-1):self.cek_boosterword(terms[i+1])
                # print  "booster score",term, self.score

                # CEK neutral term
                if self.score!=0 and i>1 and i<(terms_length-2): self.cek_neutral_term(terms,i)
                # if self.score!=0 and i>0 and i<(terms_length-4): self.cek_neutral_term(terms,i)

                self.prev_score = self.score
                if self.mean_conf and self.score>0: self.mean_pos.append(self.score)
                if self.mean_conf and self.score<0: self.mean_neg.append(abs(self.score))
                #GET MAX SCORE +ve/-ve
                self.max_pos= self.score if self.score > self.max_pos else self.max_pos
                self.max_neg= self.score if self.score < self.max_neg else self.max_neg
                #insert score info current term
                self.pre_max_pos.append(self.max_pos)
                self.pre_max_neg.append(self.max_neg)
                # print self.pre_max_pos, self.pre_max_neg
                if plural !='': term = plural
                self.sentence_text += ' {}'.format(term)
                if self.score != 0:term = "{} [{}]".format(term, self.score)
                self.sentence_score.append(term)

            self.sentences_text.append(self.sentence_text)
            self.sentences_score.append(" ".join(self.sentence_score))
            if self.is_tanya:
                self.max_neg = -1
            self.sentences_max_pos = self.max_pos if self.max_pos > self.sentences_max_pos else self.sentences_max_pos
            self.sentences_max_neg = self.max_neg if self.max_neg < self.sentences_max_neg else self.sentences_max_neg
            # print self.sentences_max_pos, self.sentences_max_neg
        sentence_result = self.classify()
        # print self.sentences_text
        return {"classified_text":". ".join(self.sentences_score),"tweet_text":". ".join(self.sentences_text),"sentence_score":self.sentences_score,"max_positive":self.sentences_max_pos,"max_negative":self.sentences_max_neg,"kelas":sentence_result}

config = dict()
config["negation"] = True
config["booster"]  = True
senti = sentistrength(config)

In [ ]:
print(senti.main("kenapa jaringan selalu tidak stabil padahal aplikasi lain stabil terus"))
print(senti.main("sudah masuk tapi meminta masuk lagi ketika tekan tautan pertemuan"))
print(senti.main("saya tidak bisa melihat chat di ponsel saya saya sudah seminggu tidak bisa menggunakan di ponsel ribet kalau harus di laptop"))
print(senti.main("sangat bagus tingkat kan lagi fitur tampilannya"))
print(senti.main("bagus"))
print(senti.main("oke bagus"))
print(senti.main("ganggu banget kamu"))
print(senti.main("sangat membantu dalam pekerjaan"))
print(senti.main("bagus"))
print(senti.main("mantap"))
print(senti.main("bagus banget untuk chat kalau tidak punya whatsapp ini juga bagus untuk sekolah tau coba aplikasinya yuk"))
print(senti.main("hi teams kolega saya kirim chat ke teams saya tapi saya tidak menerima chat nya tolong dukungannya asap saya sudah cek berkali tapi tetap tidak ada chat dari kolega saya tolong di cek terima kasih"))
print(senti.main("bagus"))
print(senti.main("mantap"))
print(senti.main("iya aku suka sama teams mau tidak jadi pacar saya"))
print(senti.main("sangat membantu dan bermanfaat"))
print(senti.main("bagus"))
print(senti.main("sangat membantu pembelajaran"))
print(senti.main("sangat bagus karena ini untuk tugas sekolah dan banyak sekali tugas yang saya dapatkan pokoknya kalian itu untuk harus coba masih teams sekarang juga karena sangat bagus banget pokoknya tidak bisa berkata kata lagi pokoknya masih teams terbaik sejagat raya indonesia ibu pertiwi pokoknya salut deh sama masih teams semoga masih teams lebih baik kedepan nya untuk membuat karakter yang baik apalagi mengirim tugas stiker nya lebih banyak pokoknya terima kasih masih teams sudah menemani aku dari juli sampai sekarang terima kasih"))
print(senti.main("selalu membantu"))
print(senti.main("bagus"))
print(senti.main("unik dan bagus sekali"))
print(senti.main("lumayan oke"))
print(senti.main("tahi nya"))
print(senti.main("keren"))
print(senti.main("aplikasinya membantu tapi juga tidak jelas sering meminta masuk ulang terus"))
print(senti.main("aplikasinya kurang mendukung sering eror dan keluar sendiri juga sehingga dilepas dulu baru diunduh lagi tapi kadang juga tidak bisa diunduh mohon untuk bisa diperbarui lagi"))
print(senti.main("oke mantap"))
print(senti.main("aplikasi sangat buruk tiap saya mau pakai tidak bisa masuk ke ruangan sudah di ubah kata sandi juga tetap tidak bisa akhirnya selalu kalau mau pakai harus lepas lalu pasang lagi besar nomor terpaksa pakai ini karena harus ikut kelas"))
print(senti.main("bener² jelek sekali setiap keluar pasti tidak ada riwayat obrolan dan harus lepas pasang lagi begitu terus tolong di perbaiki lagi ya soalnya untuk kuliah aplikasinya"))
print(senti.main("membantu saya mengerjakan tugas dengan mudah"))
print(senti.main("alangkah baiknya aplikasi ini berikan fitur untuk digunakan di ponsel bukan hanya di komputer saja supaya dipermudah menggunakan semua media dibuat sama karena di ponsel berbeda dengan dikomputer mohon diperbarui"))
print(senti.main("bagus gambarnya jernih suaranya juga jernih"))
print(senti.main("bagus"))
print(senti.main("oke"))
print(senti.main("aplikasi bangkai tahi menyusahkan"))
print(senti.main("sangat membantu"))
print(senti.main("aplikasi ribet susah di gunakan"))
print(senti.main("susah sekali aplikasi ini"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("bagus untuk sekolah"))
print(senti.main("awal pakai teams saya suka aplikasinya tapi semakin lama semakin bingung kenapa saya setiap kali akan melakukan pertemuan rapat dengan teman teman pakai teams selalu saja saya harus pasang ulang aplikasi ini jujur saya kecewa"))
print(senti.main("oke"))
print(senti.main("bagus banget untuk anak sayang tugas sekolah"))
print(senti.main("kasih bintang sebagai parcobaan"))
print(senti.main("bagus kok tidak ribut seperti whatsapp"))
print(senti.main("oke"))
print(senti.main("aplikasi jelek susah gabung rapat"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("saya rasa sudah cukup baik akan tetapi lebih ditingkatkan lagi kualitasnya"))
print(senti.main("sangat membantu"))
print(senti.main("menyusahkan"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("aplikasinya seperti cuma sekali pakai selalu eror keluar sendiri kalau dipakai lagi jadi tiap mau pakai harus mengunduh ulang"))
print(senti.main("masih kurang nyaman kadang keluar sendiri saat lagi pelajaran"))
print(senti.main("susah banget harus selalu pasang ulang tolong perbaiki kembali"))
print(senti.main("untuk aplikasi yang benar lah untuk kompatibel sama semua perangkat lah woy pemilik nya salah satu orang terkaya di bumi tapi mana kualitas mu"))
print(senti.main("sangat bagus"))
print(senti.main("belum juga terpakai kirim verifikasi nya sudah terlampaui"))
print(senti.main("sangat bagus"))
print(senti.main("aplikasi berfaedah"))
print(senti.main("sering bermasalah"))
print(senti.main("sering eror dan keluar sendiri sehingga ketinggalan kuliah"))
print(senti.main("aplikasi susahnya sangat tidak jelas"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("aplikasi paling pusing sumpah"))
print(senti.main("bagus"))
print(senti.main("terima kasih"))
print(senti.main("mau mengubah foto profilnya susah sudah diganti tapi tidak berubah"))
print(senti.main("mantap"))
print(senti.main("bagus"))
print(senti.main("kalau buka karena sering rapat lantai di aplikasi ini tidak mau banget ku pasang aplikasi kok setiap mau di pakai pasti tidak bisa masuk hi sebal banget ku"))
print(senti.main("saya senang aplikasi ini"))
print(senti.main("aplikasi yang menyusahkan"))
print(senti.main("lepas"))
print(senti.main("oke"))
print(senti.main("tidak bisa terpasang walau penyimpanan masih banyak"))
print(senti.main("wow microsoft teams buatan pembayaran gerbang sangat hebat dan bisa kasih berkomunikasi walaupun jauh"))
print(senti.main("sudah gabung teams kok dibilang belum payah"))
print(senti.main("pengembang yang benar dong saya sudah memasukkan gmail sandi masa tidak bisa mau perang"))
print(senti.main("sangat buruk"))
print(senti.main("sangat membantu"))
print(senti.main("tidak bagus sering eror harus sering pasang ulang"))
print(senti.main("kurang bagus"))
print(senti.main("mantap"))
print(senti.main("cukup bagus untuk tambah pengalaman"))
print(senti.main("bagus"))
print(senti.main("sangat pintar dan bagus"))
print(senti.main("sudah bagus cuma biasanya eror eror begitu"))
print(senti.main("sangat bagus"))
print(senti.main("bagus"))
print(senti.main("mengapa aplikasi ini harus sering kali di perbarui membuat repot dan susah saja tolong di perhatikan atasan"))
print(senti.main("maaf yakni aplikasi aneh menurut saya masa saya harus menghapus dan mengunduh ulang aplikasi kali dalam seminggu"))
print(senti.main("memuaskan"))
print(senti.main("mencoba masuk saya malah tidak bisa hadeh susah saja"))
print(senti.main("aplikasinya semakin kesini semakin tidak nyaman padahal sudah masuk akunnya tapi mau bergabung ke tautan rapat malahan tidak bisa harus hapus terus pasang lagi mohon aplikasinya segera di perbaiki"))
print(senti.main("aplikasi ini paling sering eror atau keluar sendiri giliran masuk tidak bisa terpaksa harus dihapus lalu dipasang ulang"))
print(senti.main("untuk masuk ke teams dengan tautan sering susah dan harus di lepas dulu huft"))
print(senti.main("bagus"))
print(senti.main("tolong di perbaiki sering keluar sendiri"))
print(senti.main("bagus"))
print(senti.main("terlalu sering eror harus di lepas di mengunduh lagi waktu"))
print(senti.main("aplikasi ini sangat bagus banget cuma sayangnya ponsel saya tidak memadai ram kecil prosesor lelet dan memori deh penuh sepertinya harus ganti ponsel baru yang berkenan membelikan ponsel baru yang bagus untuk saya silahkan hubungi saya ya terima kasih sebelumnya"))
print(senti.main("bagus"))
print(senti.main("sangat berguna"))
print(senti.main("mantap"))
print(senti.main("bisa di tingkatkan lagi performa aplikasinya kadang kalau share hitam screen begitu terima kasih"))
print(senti.main("histori chat tidak muncul selalu memuat mutar lepas dan pasang lagi baru keluar histori nya tapi tetap sama ketika ditutup kemudian buka lagi"))
print(senti.main("aplikasi sampah"))
print(senti.main("belum ada speaker pemandangan"))
print(senti.main("keren"))
print(senti.main("bagus alat untuk mempunyai"))
print(senti.main("oke"))
print(senti.main("puas"))
print(senti.main("baik"))
print(senti.main("top"))
print(senti.main("bagus"))
print(senti.main("lumayan bagus dan akurat bagus keberuntungan"))
print(senti.main("bagus"))
print(senti.main("audio nya kurang keras"))
print(senti.main("bagus"))
print(senti.main("oke banget"))
print(senti.main("bagus"))
print(senti.main("aplikasi kontol tidak jelas tawaran tutup malu harus di hapus dulu baru di mengunduh lagi tidak jelas banget"))
print(senti.main("rumit"))
print(senti.main("gagal memasang aplikasi"))
print(senti.main("sangat bisa diandalkan"))
print(senti.main("berkali kali pasang microsoft teams tapi selalu gagal hanya ada tulisannya tidak dapat mengunduh aplikasi ini padahal ruang penyimpanan di ponsel saya itu tidak penuh dan internet saya pun stabil tapi tetap saja tidak bisa di mengunduh aplikasinya mohon diperbaiki aplikasi microsoft teams agar bisa digunakan semua orang dan tidak kesulitan untuk mengunduhnya terima kasih"))
print(senti.main("bagus bagus"))
print(senti.main("ui kurang ramah"))
print(senti.main("bagus sangat membantu"))
print(senti.main("tolong untuk jika perbarui beri tambahan untuk saat ditendang dari temu beritahu juga siapa yang meng tendang karena menjadi masalah jika fitur tendang dimainkan secara sengaja tanpa mengetahui siapa yang menendang"))
print(senti.main("cok"))
print(senti.main("lancar"))
print(senti.main("mantap"))
print(senti.main("oke"))
print(senti.main("tidak ada emosikon udang"))
print(senti.main("tambahkan fitur ganti background versi android dong admin"))
print(senti.main("di rekomendasikan untuk sekolah"))
print(senti.main("mantap"))
print(senti.main("wajib menggunakan teams untuk kuliah tapi selalu eror harus hapus pemasangan pasang ulang kesalahan dimana harus apa agar tidak seperti itu"))
print(senti.main("aplikasi nya jelek"))
print(senti.main("sangat memuaskan saat memakai aplikasi ini"))
print(senti.main("kok selalu membuka akun sih malas banget aplikasi lelet lagi"))
print(senti.main("gagal pertemuan terus"))
print(senti.main("bagus"))
print(senti.main("aplikasinya kurang mendukung sering eror atau keluar sendiri sehingga dilepas dulu baru diunduh lagi mohon untuk bisa diperbarui lebih baik lagi jika memang direkomendasikan untuk penggunaan kuliah"))
print(senti.main("baik"))
print(senti.main("sulit untuk masuk"))
print(senti.main("unduh aplikasi tidak bisa terus padahal memori masih banyak jaringan kuat karena pakai wifi bagaimana ini padahal butuh untuk zoom pertemuan"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("siap"))
print(senti.main("bagus dan baik"))
print(senti.main("sangat membantu dan sangat baik baik digunakan dalam pembelajaran"))
print(senti.main("aplikasi ini mohon jangan eror pada penggunaan"))
print(senti.main("sangat membantu"))
print(senti.main("mantap"))
print(senti.main("sangat cocok"))
print(senti.main("suara jelas mantap"))
print(senti.main("aplikasinya sangat membantu terutama dalam menunjang pekerjaan saya"))
print(senti.main("bagus"))
print(senti.main("teams adalah aplikasi yang bisa membuat kita belajar menjadi lebih baik terima kasih teams"))
print(senti.main("keren"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("bagus sekali teams nya saya bisa terbantu karena saya bisa mengirimkan semua tugas ke aplikasi ini terima kasih teams"))
print(senti.main("membantu pekerjaan"))
print(senti.main("kelas"))
print(senti.main("pakai teams"))
print(senti.main("masuk susah mau pertemuan malah beralih akun bagaimana aplikasi bukan mempermudah malah mempersulit"))
print(senti.main("tidak bisa masuk zoom"))
print(senti.main("mantap"))
print(senti.main("bagus"))
print(senti.main("ini bagus untuk pembelajaran jarak jauh akan tetapi aplikasi ini kadang suka tidak bisa di pakai entah kenapa otomatis harus di hapus dan di mengunduh ulang"))
print(senti.main("selalu suka keluar sendiri"))
print(senti.main("zoom yang nyaman"))
print(senti.main("menghambat pekerjaan keluar tiba tiba susah untuk bergabung kembali"))
print(senti.main("suka"))
print(senti.main("capek tahu hapus dan pasang terus"))
print(senti.main("oke"))
print(senti.main("bagus pekerjaan"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("mantap aplikasinya"))
print(senti.main("baik aplikasi ini"))
print(senti.main("bagus"))
print(senti.main("saya menggunakan aplikasi teams untuk belajar"))
print(senti.main("lumayan"))
print(senti.main("oke"))
print(senti.main("tidak bisa gabung pertemuan"))
print(senti.main("memberatkan tugas saya"))
print(senti.main("aplikasi busuk susah dibuka"))
print(senti.main("bagus"))
print(senti.main("aplikasi yang cocok untuk mahasiswi daring macam saya"))
print(senti.main("beritahu kami selengkapnya opsional"))
print(senti.main("bagus"))
print(senti.main("oke bagus"))
print(senti.main("bagus"))
print(senti.main("mbak terlalu besar kasihan yang ponsel nya tidak kuat"))
print(senti.main("oke"))
print(senti.main("selalu muncul beberapa tugas yang sebelumnya sudah saya tuntaskan di menu tugas hal itu sering kali membuat saya cemas dan kaget tolong tim pengembang perbaiki eror masalah ini terima kasih"))
print(senti.main("bagus"))
print(senti.main("puas"))
print(senti.main("bagus banget terutama kualitas video saat pertemuan daring memang sangat tepat untuk keperluan perusahaan"))
print(senti.main("ya allah sumpah menyesak banget gara gara aplikasi anda yang tidak guna ini saya gagal interview saat hari ini tepat tanggal aplikasi nya eror tidak bisa menampilkan video panggilan tidak bisa menyambung menyambung ya allah sakit hati banget"))
print(senti.main("sangat bermanfaat sebagai salah satu pertemuan media yang saya pakai untuk aktivitas kantor"))
print(senti.main("sangat bagus"))
print(senti.main("aku tidak bisa pakai kata sandinya salah terus"))
print(senti.main("aplikasi sampah pakai ponsel nge log keluar terus malah suruh memasukkan email manual padahal sudah tekan tautan dari kantor"))
print(senti.main("mantap sangat berguna untuk bantu pekerjaan saya"))
print(senti.main("tidak jelas kadang bisa kadang tidak"))
print(senti.main("aplikasi mematikan pada masanya"))
print(senti.main("terbaik"))
print(senti.main("bagus mantap"))
print(senti.main("alternatif terbaik"))
print(senti.main("aplikasi ini sangat bagus untuk belajar dan menyenangkan"))
print(senti.main("sering eror berkali kali dan harus lepas dan mengunduh lagi"))
print(senti.main("tersendat di tengah jalan tiba log keluar sendiri terus tidak bisa masuk lagi terus mau masuk lagi harus mengunduh lagi"))
print(senti.main("sangat membantu dalam mengerjakan tugas kuliah terima kasih microsoft"))
print(senti.main("bagus"))
print(senti.main("bagaimana sih suka eror terus"))
print(senti.main("aplikasi nya sering eror dan buruk mohon di evaluasi kembali kak"))
print(senti.main("sangat membantu untuk kegiatan tuweb"))
print(senti.main("sangat baik"))
print(senti.main("oke"))
print(senti.main("mantap"))
print(senti.main("teams aplikasi yang bagus untuk perkuliahan daring tapi kenapa ya saya susah hangat masuk ke teams melalui ponsel mohon pencerahannya terima kasih"))
print(senti.main("keluar terus lalu dibuat pakai email baru"))
print(senti.main("aplikasi nya membingungkan selalu ganti akun selalu ganti akun jadi harus mengunduh ulang kim malas"))
print(senti.main("bagus"))
print(senti.main("tidak bisa masuk tautan"))
print(senti.main("kenapa tidak bisa mengunduh aplikasi ini saya lagi perlu"))
print(senti.main("bagus aplikasi"))
print(senti.main("lima bintang"))
print(senti.main("tolong di cek untuk penggunaan teams di ponsel asus untuk penggunaan keyboard sering terjadi eror"))
print(senti.main("kenapa di ponsel note pro tak bisa mengunduh aplikasi ini"))
print(senti.main("tolong di bantu kerja samanya"))
print(senti.main("bagus"))
print(senti.main("tetap peningkatan"))
print(senti.main("kualitas suara kurang baik tolong ditingkatkan"))
print(senti.main("tombol share powerpoint tidak ada"))
print(senti.main("bagus"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("ini lumayan berguna untuk pembelajaran jarak jauh dan bisa di untuk menemukan teman di teams ini berguna dan aku memberi ranting bintang"))
print(senti.main("rumit"))
print(senti.main("tugasnya sejagat raya"))
print(senti.main("tidak jelas petunjuknya untuk masuk aplikasi susah benar"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("oke"))
print(senti.main("bagus banget aplikasi nya"))
print(senti.main("tidak tau"))
print(senti.main("sangat bagus untuk mengumpulkan tugas"))
print(senti.main("bagus"))
print(senti.main("terbaik"))
print(senti.main("bagus"))
print(senti.main("selamat bintang ini"))
print(senti.main("suara gambar dan teks jelas yang penting bagus deh"))
print(senti.main("sering gagal masuk kalau ponsel di mulai ulang"))
print(senti.main("belum sampai masuk log masuk saja sudah akun terkunci dilakukan berikutnya penggunaan terlampaui"))
print(senti.main("terima kasih microsoft teams"))
print(senti.main("keren"))
print(senti.main("oke"))
print(senti.main("terima kasih aplikasi teams membantu saya untuk kuliah daring"))
print(senti.main("sangat membantu untuk sistem belajar di ut"))
print(senti.main("sehari kali pasang beralih beralih masuk pakai nomor yang perbaiki malah nomor salah nomor salah"))
print(senti.main("bagus"))
print(senti.main("untuk biasanya dalam satu minggu hapus unduh ulang bisa ikut rapat dan langsung bisa masuk sekarang aku sudah hapus unduh hapus unduh kali untuk ikut acara rapat tidak bisa sampai sekarang sangat amat kecewa"))
print(senti.main("oke"))
print(senti.main("sangat bagus"))
print(senti.main("mohon di perbarui kembali aplikasinya"))
print(senti.main("sering eror layar mati"))
print(senti.main("mau masuk saja susah banget harus lepas dulu baru bisa masuk sangat tidak menghemat kuota ya"))
print(senti.main("tidak bisa pasang ulang jelek banget ini"))
print(senti.main("kendala teams di ponsel saya adalah tautan pertemuan yang saya berusaha gabungkan sebagai tamu tidak bisa di akses"))
print(senti.main("bagus lah"))
print(senti.main("kualitas biasa"))
print(senti.main("aplikasi meminta perbarui akun setiap masuk titip mau pertemuan keteter jadi nya mohon di perbaiki"))
print(senti.main("oke"))
print(senti.main("keren banget untuk mahasiswa"))
print(senti.main("bagus"))
print(senti.main("sangat membantu"))
print(senti.main("tidak bisa di untuk lihat soal kuis"))
print(senti.main("bagus"))
print(senti.main("aplikasi paling sering keluar sendiri giliran masuk tidak bisa terpaksa harus hapus lalu mengunduh kembali terus masuk pakai akun yang sama bisa merepotkan"))
print(senti.main("baru mengunduh aplikasinya semoga tidak lelet saat digunakan hari h nya wkwk oh ya kedepannya saya akan selalu pakai aplikasi ini karena anak ut membutuhkannya dan semoga aplikasi serta sinyal merestui adanya pembelajaran yang akan saya lakukan semoga tidak ada kendala amin"))
print(senti.main("susah masuk"))
print(senti.main("bagus"))
print(senti.main("membuat otak pusing"))
print(senti.main("sangat membantu"))
print(senti.main("mudah untuk gunakan bisa chat di ponsel dan chat ini tautan ke seluruh dunia terima kasih teams"))
print(senti.main("ribet banget ih harus pasang mengunduh setiap minggu saat ingin digunakan"))
print(senti.main("oke"))
print(senti.main("bagus aplikasi"))
print(senti.main("mantap"))
print(senti.main("oke lah untuk menemani aktivitas sehari hari"))
print(senti.main("the terbaik aplikasi"))
print(senti.main("bagus"))
print(senti.main("bagus banget aplikasinya untuk kuliah the terbaik lah pokoknya"))
print(senti.main("bagus"))
print(senti.main("sangat bagus untuk kantor tujuan"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("melalui aplikasi masih teams ini sangat membantu"))
print(senti.main("niat tidak membuat aplikasi"))
print(senti.main("pemasaran"))
print(senti.main("aplikasi bagus"))
print(senti.main("memudahkan"))
print(senti.main("setiap masuk teams belum sampai satu menit sudah keluar sendiri"))
print(senti.main("bagaimana sih mau ikut rapat sudah tekan tautan tapi tidak bisa masuk"))
print(senti.main("membantu pembelajaran"))
print(senti.main("tidak semua jenis android bisa memasang aplikasi ini"))
print(senti.main("sebagai sarana komunikasi yang dapat memudahkan dan lancar dalam berbisnis"))
print(senti.main("semakin kesini semakin mengaco saja masa tiap mau pakai harus diperbarui padahal jarang dipakai"))
print(senti.main("kenapa tidak bisa di untuk lihat kuis"))
print(senti.main("saya kasih bintang karena saya pakai teams untuk pertemuan sama guru saya"))
print(senti.main("bagus sangat membantu"))
print(senti.main("keren"))
print(senti.main("oke lah"))
print(senti.main("sangat membantu"))
print(senti.main("bagus"))
print(senti.main("aplikasi kurang bagus sering eror saat mengerjakan ujian padahal jaringan sudah bagus"))
print(senti.main("jelek kenapa saya tidak bisa mengunduh tolong dong ini penting"))
print(senti.main("bagus"))
print(senti.main("oke"))
print(senti.main("oke"))
print(senti.main("baik"))
print(senti.main("lelet suka keluar sendiri saat lagi rapat gentingnya"))
print(senti.main("tidak bisa di mengunduh"))
print(senti.main("tidak bisa di cari di google play dan tidak bisa pasang"))
print(senti.main("tolong dong jangan eror"))
print(senti.main("sangat membantu"))
print(senti.main("begitu dulu"))
print(senti.main("sering banget keluar tidak pakai wifi dan data sama saja"))
print(senti.main("bagus"))
print(senti.main("memuat lama sinyal buruk"))
print(senti.main("coba dulu"))
print(senti.main("bagus"))
print(senti.main("akunnya suka keluar dan harus ubah sandi terus"))
print(senti.main("sangat kecewa"))
print(senti.main("terima kasih masih teams mempermudahkan saya untuk belajar"))
print(senti.main("coba tombol kumpulkan ditaruh dibawah file saja anjing agar orang kagak lupa memencet"))
print(senti.main("mantap sarana komunikasi multi fungsional jarak dan sebagainya"))
print(senti.main("aplikasi microsoft teams ini sangat membantu"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("sangat memuaskan"))
print(senti.main("kok susah banget mengunduh mengulang mulu padahal sudah"))
print(senti.main("sangat bagus"))
print(senti.main("entah kenapa saya keluar keluar dari rapat disuruh menggunakan perangkat lain atau transfer perangkat dan entah kenapa kejadian ini terjadi cuma di hari rabu saja dan aneh banget"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("hebat jadi lebih mudah mengerjakan tugas dan menyelesaikan tugas"))
print(senti.main("saya mau keluar teams woy"))
print(senti.main("di coba"))
print(senti.main("oke banget"))
print(senti.main("oke"))
print(senti.main("aplikasi edit foto anda dengan ini saya akan memberikan informasi yang menarik untuk di konsumsi oleh para ahli sejarah chadwick boseman yang sudah memberikan tanggapan dari tadi davy alexander xi ips kelas walaupun tidak pernah balas menyebutkan the diam tersebut akan menjadi orang tua yang tidak ada lagi yang bisa ia dapatkan di sini apa itu dimana tidak ada salahnya jika kita belum tahu renyah nih atasan yang saya dapatkan dari tadi berangkat dari tadi berangkat ke sekolah yang sudah memberikan ta"))
print(senti.main("bagus"))
print(senti.main("bagus untuk pelajaran"))
print(senti.main("bagus tapi ada yang kurang kak coba tambahkan filter seperti di instagram agar kalau ada daring kelihatan seperti sudah mandi wkwk tambahkan filter ya kak filter seperti instagram"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("memudahkan dalam pembelajaran"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("keren dan kuat"))
print(senti.main("teams bermanfaat dan berguna"))
print(senti.main("teams itu mudah untuk mengerjakan tugas pertemuan itu membantu semoga teams lebih maju dan bagus"))
print(senti.main("top aplikasi"))
print(senti.main("bagus"))
print(senti.main("apakah kamu ingin berteman dengan nana"))
print(senti.main("walaupun aplikasi ini cukup berat tetapi bisa dimaklumi karena masih teams tidak hanya sekadar lms tetapi lebih dari itu"))
print(senti.main("bagus"))
print(senti.main("oke"))
print(senti.main("aplikasi untuk belajar daring yang luar biasa"))
print(senti.main("butut"))
print(senti.main("bagus"))
print(senti.main("oke"))
print(senti.main("keren"))
print(senti.main("luar biasa"))
print(senti.main("aplikasi tidak guna membuat ribet saja"))
print(senti.main("oke"))
print(senti.main("seperti"))
print(senti.main("aplikasi menyusahkan"))
print(senti.main("mantap betul"))
print(senti.main("sangat"))
print(senti.main("oke"))
print(senti.main("masyaallah"))
print(senti.main("sangat membantu"))
print(senti.main("ini aplikasi sampah"))
print(senti.main("mantap"))
print(senti.main("tidak bisa dibuka"))
print(senti.main("baik"))
print(senti.main("hoh benaran"))
print(senti.main("bagus pekerjaan bagus ide"))
print(senti.main("apa itu"))
print(senti.main("oke banget"))
print(senti.main("oke"))
print(senti.main("yang tidak tahu pasti anggapan buruk"))
print(senti.main("bermanfaat"))
print(senti.main("bagus"))
print(senti.main("oke"))
print(senti.main("sebaiknya tetap bisa digunakan di android juga admin"))
print(senti.main("lama banget nge mengunduh aplikasi nya"))
print(senti.main("keren pekerjaan jadi lebih optimal"))
print(senti.main("eror"))
print(senti.main("oke"))
print(senti.main("meresahkan pengganti gcr"))
print(senti.main("bagus"))
print(senti.main("kenapa susah di mengunduh"))
print(senti.main("saat temu tidak bisa kembali ke menu chat jika masuk temu melalui tautan"))
print(senti.main("semangat bagus"))
print(senti.main("yang jelas membuat aplikasi"))
print(senti.main("teams ini berguna untuk rapat daring dan belajar daring terima kasih teams"))
print(senti.main("bagus"))
print(senti.main("i seperti teams tapi a jaringan adalah sok lambat tolong perbaikan ini server"))
print(senti.main("bagus banget"))
print(senti.main("mantap"))
print(senti.main("eror terus"))
print(senti.main("aplikasinya bagus sangat membantu dan rekomendasi"))
print(senti.main("keren"))
print(senti.main("bismillah bisa"))
print(senti.main("parah ini aplikasi"))
print(senti.main("sangat bagus"))
print(senti.main("bagus"))
print(senti.main("bagus aplikasi"))
print(senti.main("membuat banyak tugas saja"))
print(senti.main("aplikasi nya bagus bisa lihat tugas dan mengerjakan nya"))
print(senti.main("mantap"))
print(senti.main("deh lah nomor komen"))
print(senti.main("bagus bagus"))
print(senti.main("mantap"))
print(senti.main("kenapa setiap masuk rapat keluar"))
print(senti.main("bagus sekali"))
print(senti.main("mantap"))
print(senti.main("mengunduh bolak balik tapi gagal terus sudah malah ulang lagi dari awal bagaimana ya solusinya"))
print(senti.main("mantap"))
print(senti.main("sangat baik"))
print(senti.main("sangat membantu"))
print(senti.main("bagus aplikasi"))
print(senti.main("gagal masuk mulu padahal sudah pakai email akun microsoft juga keterangannya selalu tidak menemukan akun dengan nama tersebut"))
print(senti.main("gara gara aplikasi ini tugas saya banyak terima kasih kami"))
print(senti.main("bagus"))
print(senti.main("terkeluar terus bangkai jadi susah tahu interview"))
print(senti.main("aplikasi yang bagus dengan kualitas video dan audio mantap"))
print(senti.main("keren"))
print(senti.main("bagus"))
print(senti.main("bagus aplikasi"))
print(senti.main("aplikasi yang bagus untuk pertemuan daring tapi masih ada tunda"))
print(senti.main("mudah gunakan kuat"))
print(senti.main("mantap"))
print(senti.main("sangat membantu"))
print(senti.main("menggunakan android sangat menyusahkan perlu aktivasi akun yang ribet tidak seperti zoom"))
print(senti.main("bagus aplikasinya"))
print(senti.main("tidak bisa antar makanan"))
print(senti.main("sangat bagus untuk mengerjakan tugas"))
print(senti.main("oke"))
print(senti.main("the aplikasi ukuran adalah sok besar tolong mengoptimalkan itu"))
print(senti.main("kalau lagi temu kadang suka eror tidak jelas contoh keluar tiba masuk belum sampai menit langsung keluar lagi dan sering terjadi berulang kali"))
print(senti.main("ini bagaimana kok tidak bisa masuk sih tidak seperti awal aku memasukkan email sudah selesai sekarang selalu kesalahan mulu"))
print(senti.main("di pasang di laptop eror saat masuk suara tidak terdengar saat pertemuan padahal wifi lancar"))
print(senti.main("lama di buka harus menunggu menit parah banget"))
print(senti.main("mantap"))
print(senti.main("jelek"))
print(senti.main("oke"))
print(senti.main("baik"))
print(senti.main("bagus"))
print(senti.main("terima kasih"))
print(senti.main("mantap"))
print(senti.main("salah satu fitur kelas maya yang cukup mudah diakses"))
print(senti.main("saya tidak tahu"))
print(senti.main("aplikasi nya bermanfaat banget tapi macet saat lagi belajar daring tapi tidak papa sih mungkin cuma eror saja aplikasinya"))
print(senti.main("sangat baik dan bagus"))
print(senti.main("bagus"))
print(senti.main("kenapa aplikasi ini tidak bisa di mengunduh sih sudah bisa di mengunduh eh kembali lagi ke pasang kalau aplikasi lain bisa bisa saja di mengunduh sudah tahu saya butuh banget untuk tugas ihk kesal saya jadinya dari kemarin deh di mengunduh tidak bisa bisa sudah pakai cara di youtube juga tidak bisa"))
print(senti.main("tidak bisa memasang"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("mantap setiap kali pertemuan pakai aplikasi ini"))
print(senti.main("mantap"))
print(senti.main("mantap"))
print(senti.main("oke"))
print(senti.main("mantap"))
print(senti.main("terima kasih teams"))
print(senti.main("susah masuk"))
print(senti.main("tidak bisa di unduh anjay susah dari tadi mulu saya mengunduh"))
print(senti.main("mau daftar akun saja susah"))
print(senti.main("tidak dapat di mengunduh"))
print(senti.main("mantap"))
print(senti.main("sangat nyata dan orisinil"))
print(senti.main("mantap"))
print(senti.main("bagus"))
print(senti.main("woy lah tolong dong bagaimana sih ini saya sudah log masuk pakai email dan sandi yang sudah di kasih kesekolah untuk saya tapi kok malah tidak ada tim nya di saya sementara teman saya kok bisa sih kesal saya lama padahal ada tugas dari sekolah dan saya tidak bisa mengerjakan tolong dong di perbaiki"))
print(senti.main("sangat cocok bagi pelajar"))
print(senti.main("aplikasi microsoft mengunduh lewat ponsel bermasalah semua sudah keluar terus masuk nomor lama padahal untuk tugas lah kalau membuat email juga percuma kalau nomor sama ya kali beli nomor terus"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("setiap habis dipakai mau masuk rapat lagi pasang ulang lagi"))
print(senti.main("aplikasinya agak hebat tapi bila bila"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("mohon informasi kenapa saya log masuk tidak bisa ya"))
print(senti.main("terjadi kesalahan terus saat baru masuk bagaimana sih ini tidak benar banget"))
print(senti.main("mantap"))
print(senti.main("sangat memuaskan"))
print(senti.main("bagus"))
print(senti.main("oke terima kasih"))
print(senti.main("susah mengunduh"))
print(senti.main("bagus"))
print(senti.main("bismillah belum tahu dalam nya sih"))
print(senti.main("bagus"))
print(senti.main("baik"))
print(senti.main("sangat membantu dalam pekerjaan"))
print(senti.main("oke"))
print(senti.main("bagus dan aman"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("baik"))
print(senti.main("buatlah aplikasi yang membuat kita hemat waktu ini sih banyak mulutnya mau daftar saja repot"))
print(senti.main("aplikasi ini sangat membantu saya dalam mengerjakan tugas dari sekolah"))
print(senti.main("sangat membantu rapat"))
print(senti.main("suka"))
print(senti.main("aplikasi menyusahkan kenapa sih pertemuannya harus pakai aplikasi ini"))
print(senti.main("sinyal bagus"))
print(senti.main("saya sarankan dengan sangat untuk kedepannya microsoft teams ini lebih baik dari pada sebelumnya bagus pekerjaan"))
print(senti.main("hebat"))
print(senti.main("sering eror dan banyak keluar aplikasi"))
print(senti.main("jelek"))
print(senti.main("sangat membantu saya dan memudahkan"))
print(senti.main("berat"))
print(senti.main("bagus"))
print(senti.main("kok tidak bisa di mengunduh ya"))
print(senti.main("jelek banget deh"))
print(senti.main("terus berkembang dengan baik aplikasi nya"))
print(senti.main("bagus dan membantu"))
print(senti.main("susah kali ya masuk ke tautan pertemuan sampai ketinggalan pertemuan daring dan harus copot pasang aplikasi dulu baru bisa bergabung tautan nya kalau bukan karena tuntutan atasan pakai ini aplikasi tidak mau saya memakai nya tolong diperbaiki masalah bergabung oleh tautan nya keseluruhan saya kecewa sama ini aplikasi"))
print(senti.main("masuk melalui web bisa tapi melalui ponsel aplikasi ini kok tidak bisa ya mohon solusinya untuk pengembang"))
print(senti.main("sampah"))
print(senti.main("bagus"))
print(senti.main("susah masuk ke rapat sudah masuk malah keluar mulu"))
print(senti.main("jelek banget"))
print(senti.main("sangat bermanfaat"))
print(senti.main("bagus"))
print(senti.main("bagus lah"))
print(senti.main("sangat membantu pekerjaan kita secara profesional"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("sangat membantu"))
print(senti.main("banyak amat tugasnya"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("sudah pasang tapi susah masuk dari tautan whatsapp malah aplikasi rusak gagal rencana"))
print(senti.main("sangat bagus"))
print(senti.main("belum bisa mengunduh"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("hanya bisa senyum pasrah dengan tugas sebanyak harapan orang tua"))
print(senti.main("bagus"))
print(senti.main("susah masuk ih"))
print(senti.main("sampah banget asli yang versi windows susah banget masuk e"))
print(senti.main("aplikasi keren"))
print(senti.main("bagus"))
print(senti.main("sangat baik"))
print(senti.main("oke"))
print(senti.main("oke"))
print(senti.main("setiap mau pertemuan selalu tidak bisa terus awal doang bisa sudah hapus mengunduh ulang masih saja sama tidak bisa"))
print(senti.main("ini apa lah kok tidak bisa di mengunduh parah sumpah"))
print(senti.main("aplikasi ini sangat bagus saya sangat menyukainya"))
print(senti.main("mantap lah"))
print(senti.main("bagus"))
print(senti.main("aplikasi sampah aplikasinya tiba keluar sendiri"))
print(senti.main("bagus"))
print(senti.main("kami adalah the terbaik teams pernah insya allah"))
print(senti.main("ya bagus agak"))
print(senti.main("mantap"))
print(senti.main("lumayan lah"))
print(senti.main("aplikasi bagus ketika anda harus belajar daring"))
print(senti.main("bagus"))
print(senti.main("aplikasi sampah tidak berguna sama sekali"))
print(senti.main("oke"))
print(senti.main("jadi banyak tugas"))
print(senti.main("mantap"))
print(senti.main("bagus aplikasi membantu"))
print(senti.main("mantap"))
print(senti.main("bagus"))
print(senti.main("aplikasi sampah sialan meminta diperbarui terus"))
print(senti.main("masuk susah banget sandi sudah benar malah salah apa apaan ini padahal tidak ganti sandi ketika masuk saya ada tugas sekolah melalui aplikasi teams ini pada hari ini hingga beberapa hari kedepan tolong beri saya konfirmasi"))
print(senti.main("ribet tidak bisa untuk linķ sendiri dari ponsel mesti pakai laptop"))
print(senti.main("ini bagus"))
print(senti.main("terbaik"))
print(senti.main("buruk"))
print(senti.main("kenapa tidak bisa mengunggah file pdf dan excel ya"))
print(senti.main("kalau ada pembaruan mengomong dong"))
print(senti.main("bagus"))
print(senti.main("aplikasi yang sangat berguna sekali"))
print(senti.main("ampas deh aplikasi nih"))
print(senti.main("bagus"))
print(senti.main("aplikasi keren"))
print(senti.main("microsoft teams sangat bermanfaat untuk pertemuan zoom para maha siswa dan peserta didik"))
print(senti.main("mantap banget nih"))
print(senti.main("mantap"))
print(senti.main("mantap pak muhajir"))
print(senti.main("aplikasinya bagus sehingga saya bisa belajar lewat aplikasi ini"))
print(senti.main("bagus"))
print(senti.main("mantap"))
print(senti.main("bagus"))
print(senti.main("oke"))
print(senti.main("mantap"))
print(senti.main("aplikasi aneh mau ganti kata sandi saja sampai detail tulisannya kami terlalu sering melihat kata sandi anda kan mau di ganti kenapa di permasalahan kenapa susah sih tolong lah pengembang jangan membuat orang orang pada bingung"))
print(senti.main("bagus"))
print(senti.main("aplikasi yang lengkap bisa temu membuat channel khusus berdasarkan mata pelajaran dan pemberian tugas"))
print(senti.main("mantap tepat waktu"))
print(senti.main("di web bisa masuk tapi di aplikasi tidak bisa"))
print(senti.main("pengetahuan untuk the terbaik"))
print(senti.main("tidak bisa di pasang padahal harus pakai teams kalau ada tugas"))
print(senti.main("mikrofon kamera kadang tidak bisa di tekan saat langsung"))
print(senti.main("baik"))
print(senti.main("mudah digunakan"))
print(senti.main("kenapa tidak bisa di mengunduh"))
print(senti.main("bagus"))
print(senti.main("membantu"))
print(senti.main("mantap"))
print(senti.main("sangat bermanfaat untuk komunikasi"))
print(senti.main("tidak tahu sih sama aplikasinya saya cuma mau mempermasalahkan pada masalah foto profilnya"))
print(senti.main("oke"))
print(senti.main("microsoft teams selalu terbaik untuk video pertemuan tetap berlanjut microsoft berharap mereka akan menjadi lite versi untuk android seperti teams dan semua kantor untuk lebih rendah android"))
print(senti.main("bagus"))
print(senti.main("bagus untuk komunikasi"))
print(senti.main("tidak berguna"))
print(senti.main("ribet woy"))
print(senti.main("oke"))
print(senti.main("tidak boleh di perbarui"))
print(senti.main("sangat membantu saat proses belajar mengajar secara daring"))
print(senti.main("oke"))
print(senti.main("lambat bergabung tidak pertemuan ya"))
print(senti.main("saat verifikasi nomor telepon kode nya tidak terkirim"))
print(senti.main("oke i tarik atas"))
print(senti.main("tidak permainan aku bisa sekolah dengan lancar"))
print(senti.main("bagus"))
print(senti.main("setelah saya meng perbarui aplikasi ini ternyata sama saja tugas masih banyak di berikan haha malah tambah banyak ya ges ya saya senang sekali karena membuat saya stres terus pusing dan akhirnya sakit ya ges terus tidak berangkat sekolah deh hore a"))
print(senti.main("mantap"))
print(senti.main("luar  biasa ini aplikasi yang aku suka"))
print(senti.main("bagus"))
print(senti.main("sangat membantu dalam menjalankan kuliah"))
print(senti.main("mantap"))
print(senti.main("bagus"))
print(senti.main("oke"))
print(senti.main("mantap"))
print(senti.main("oke"))
print(senti.main("alhamdulillah aplikasi yang sangat bagus dan dipakai juga di kantor saya sangat mudah di gunakan dan tidak ada batasan waktu saat pertemuan terima kasih"))
print(senti.main("mantap"))
print(senti.main("ini saya sudah log masuk tapi malah meminta kata sandi lagi ya maklum saya nya lupa"))
print(senti.main("mutar tidak jelas"))
print(senti.main("lebih praktis dan mudah komunikasi"))
print(senti.main("bagus sekali"))
print(senti.main("keren"))
print(senti.main("sangat membantu"))
print(senti.main("mantap arek"))
print(senti.main("jadi ribet begini sekarang banyak mintanya mau masuk keluar mulu"))
print(senti.main("aplikasi tidak jelas pengembang tolol anjing"))
print(senti.main("tidak bisa masuk"))
print(senti.main("membantu pekerjaan sehari hari"))
print(senti.main("ribet"))
print(senti.main("sangat bagus dan keren"))
print(senti.main("pada saat masuk keluar terus"))
print(senti.main("tidak bisa bergabung pakai kode"))
print(senti.main("wow bagus sekali"))
print(senti.main("teams kok sering perbarui sih memori saya habis nih"))
print(senti.main("aplikasi nya suka eror"))
print(senti.main("bagus banget untuk belajar sukses terus"))
print(senti.main("oke mudah di gunakan"))
print(senti.main("gara gara aplikasi ini aku tidak jadi dapat kerja jaringannya eror mulu padahal pakai discord zoom googlemeet saja normal"))
print(senti.main("lucu banget"))
print(senti.main("bagaimana kalau pasang di notebook"))
print(senti.main("masih teams kadang susah untuk masuk"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("lumayan"))
print(senti.main("baik"))
print(senti.main("mohon untuk pihak microsoft untuk pengguna android ios untuk bisa menghapus file yang telah terkirim itu sangat mengganggu"))
print(senti.main("saya sangat ingin maju bersama namun keterbatasan kemampuan digital"))
print(senti.main("aplikasi mantap"))
print(senti.main("keren"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("lancar deh pokoknya"))
print(senti.main("mantap"))
print(senti.main("tidak bisa di mengunduh"))
print(senti.main("bodoh"))
print(senti.main("kapan i mengunggah ku file kenapa itu selalu eror"))
print(senti.main("lelet"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("mantap"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("bagus aplikasi untuk daring pertemuan"))
print(senti.main("cepat dan jelas"))
print(senti.main("bagus banget wkwk"))
print(senti.main("sangat membantu saya saat belajar atau mengumpul tugas"))
print(senti.main("mantap"))
print(senti.main("nyaman sudah itu saja"))
print(senti.main("bagus"))
print(senti.main("kenapa tidak semua perangkat bisa ganti background saat nyala kamera"))
print(senti.main("deh lah"))
print(senti.main("alhamdulillah sejauh ini bisa mendukung pekerjaan agar berjalan baik terima kasih"))
print(senti.main("lebih ditinggalkan lagi"))
print(senti.main("bagus"))
print(senti.main("aplikasi ribet mengirimkan tugas saja susah banget saja masih bagusan aplikasi grup chat"))
print(senti.main("mantap"))
print(senti.main("bagus"))
print(senti.main("luar  biasa"))
print(senti.main("kok tambah aneh laporan terbaru tidak bisa langsung malah laporan lama yang terbaca ini kenapa ya"))
print(senti.main("oke"))
print(senti.main("daring dimana pun tanpa harus menggunakan laptop"))
print(senti.main("terima kasih untuk microsoft teams berkat aplikasi ini saya bisa berkenalan dengan cewek jurusan lain hingga pacaran sampai hampir setahun walaupun pada akhirnya kisah kami harus berakhir sampai disini pada tanggal juli tapi saya beruntung sekali bisa kenal dia berkat aplikasi ini saya harap dia bahagia suatu saat nanti dengan orang baru yang telah dia percayai dan dia pilih jujur saya tidak mau kehilangan dia tapi mungkin ini sudah takdirnya sekian sidoarjo minggu agustus"))
print(senti.main("aplikasi sampah"))
print(senti.main("aplikasi ini sangat bagus untuk sekolah"))
print(senti.main("terima kasih"))
print(senti.main("keren habis"))
print(senti.main("lumayan"))
print(senti.main("oke"))
print(senti.main("gaskan tong keluar saja email salah nya"))
print(senti.main("saya susah masuk nya"))
print(senti.main("moga moga han bisa di mengunduh"))
print(senti.main("sangat membantu"))
print(senti.main("sangat membantu dalam pertemuan saya"))
print(senti.main("kok tidak bisa diunduh ya sudah diulang beberapa kali tapi tetap tidak bisa"))
print(senti.main("sangat bermanfaat banyak pilihannya"))
print(senti.main("sangat bermanfaat"))
print(senti.main("mantap"))
print(senti.main("mantap"))
print(senti.main("bisa memudahkan belajar daring"))
print(senti.main("sebenarnya saya suka aplikasi ini tapi membutuhkan banyak kuota"))
print(senti.main("bagus"))
print(senti.main("sering gagal masuk rapat padahal sudah log masuk harus log masuk berkali kali baru bisa"))
print(senti.main("oke"))
print(senti.main("perbarui terus"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("saya sangat mencintai"))
print(senti.main("bagus aplikasi untuk pertemuan"))
print(senti.main("susah tersambung"))
print(senti.main("oke"))
print(senti.main("sangat bermanfaat"))
print(senti.main("oke"))
print(senti.main("keren"))
print(senti.main("setiap mau masuk pertemuan tidak bisa masuk"))
print(senti.main("mantap"))
print(senti.main("aneh kenapa tidak bisa gabung teams dosen sudah mengasih kode teams tapi kok tidak ada tempat untuk memasukkan kode teams"))
print(senti.main("jelas dan jelas"))
print(senti.main("bagus"))
print(senti.main("tidak bisa masuk"))
print(senti.main("mantap"))
print(senti.main("terima kasih aplikasinya sangat membantu pekerjaan saya"))
print(senti.main("bagus sekali"))
print(senti.main("tidak dapat digunakan teams microsoft untuk masuk saja tidak bisa"))
print(senti.main("keren"))
print(senti.main("tidak bisa di mengunduh emosi saya"))
print(senti.main("bintang"))
print(senti.main("bagus"))
print(senti.main("aplikasi untuk sekolahan"))
print(senti.main("diaplikasi saya tidak ada pilihan ubah background"))
print(senti.main("sering macet"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("suka eror tidak bisa masuk"))
print(senti.main("bagus tapi kenapa kok setiap saya mau bergabung harus masukkan nama pengguna terus kan sudah pernah daftar"))
print(senti.main("sangat bagus sekali aplikasi microsoft teams sangat oke lah"))
print(senti.main("baik"))
print(senti.main("sangat bermanfaat"))
print(senti.main("bagus untuk belajar dan daring"))
print(senti.main("terima kasih"))
print(senti.main("sangat bermanfaat"))
print(senti.main("siap"))
print(senti.main("lumayan bisa dibuat belajar"))
print(senti.main("sangat membantu"))
print(senti.main("aplikasi sudah kuat hanya kurang promosi saja"))
print(senti.main("sangat enak"))
print(senti.main("oke lah"))
print(senti.main("aplikasi yang bagus sekali"))
print(senti.main("bagus lah"))
print(senti.main("aplikasi sampah susah pakai nya pakai zoom deh paling benar"))
print(senti.main("cocok digunakan dalam organisasi"))
print(senti.main("aplikasi selalu eror disuruh masuk berulang"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("oke banget deh"))
print(senti.main("mantap"))
print(senti.main("semakin nge tersendat terus susah bergabung ke pertemuan suka tidak bisa kepencet mikrofon nya mohonlah kepada pihak pengembang untuk memperbaiki"))
print(senti.main("suka eror"))
print(senti.main("mudah digunakan dan banyak fitur berguna untuk koordinasi tim kantor"))
print(senti.main("sampah banget seperti kamu memek"))
print(senti.main("aplikasi bangkai disaat sudah masuk rapat malah keluar sendiri masuk lagi keluar lagi padahal lagi interview jadi gagal interview nya gara gara aplikasi bangkai ini"))
print(senti.main("siap"))
print(senti.main("wow"))
print(senti.main("aplikasi sampah anjing"))
print(senti.main("bagus"))
print(senti.main("mantap"))
print(senti.main("mantap"))
print(senti.main("bagus"))
print(senti.main("sangat membantu"))
print(senti.main("kok tidak bisa di mengunduhnya"))
print(senti.main("sangat membantu walaupun masih bingung"))
print(senti.main("mantap canggih"))
print(senti.main("aplikasi microsoft teams merupakan aplikasi terjelek di google dan play store karena aplikasi microsoft teams tidak dapat diunduh tidak dapat dibuka aplikasi microsoft nya sungguh menjijikkan dan memalukan sekali aplikasi karya bangsa milik siapa ini"))
print(senti.main("tuhan"))
print(senti.main("terlalu berat sering membebani performa ponsel saya"))
print(senti.main("oke"))
print(senti.main("buruk ini aplikasi kalau tidak kerja tidak mau pakai ini aplikasi sedikit perbarui terus perbarui pun tidak bisa dipakai lepas ulang begitu saja kerja"))
print(senti.main("mantap"))
print(senti.main("kurang bagus"))
print(senti.main("oke"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("bagus aplikasi terima kasih"))
print(senti.main("bagus aplikasinya"))
print(senti.main("kenapa tidak bisa digunakan sudah beberapa hari ini baik yang di laptop ponsel pertemuan saya berantakkan semua"))
print(senti.main("maaf kenapa di samsung galaxy saja tidak bisa di mengunduh kenapa ya harusnya di semua ponsel android bisa tolong perbaiki lagi ya"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("oke mantap"))
print(senti.main("bagaimana cara masuk ke tim nya bambang"))
print(senti.main("aplikasi sampah ini setiap mau bergabung pertemuan selalu tidak bisa masuk sudah berulang kali dihapus dan di mengunduh masih sama saja tetap sampah juga tolong dong diperbaiki kalau tidak mending tidak usah ada saja jadi menyusahkan bukan membantu bintang juga sepertinya tidak pantas juga dikasih"))
print(senti.main("hilangkan fitur tendang peserta"))
print(senti.main("kenapa setelah mengunduh tidak bisa terpasang"))
print(senti.main("tidak jelas"))
print(senti.main("susah dibuka"))
print(senti.main("oke"))
print(senti.main("mantap"))
print(senti.main("baik"))
print(senti.main("untuk belajar aku"))
print(senti.main("mantap"))
print(senti.main("mantap"))
print(senti.main("oke banget"))
print(senti.main("hapus saja aplikasi nya ini permainan sangat sulit ditamatkan tolong dipermudah jika perlu maksimal tugas saja"))
print(senti.main("aplikasinya sangat mendukung pengiriman tugas"))
print(senti.main("mantap"))
print(senti.main("masih teams lagi eror hari ini tidak bisa masuk sama sekali mohon diperbaiki segera"))
print(senti.main("sangat bagus"))
print(senti.main("lumayan berat di ponsel"))
print(senti.main("mantap"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("dipakai untuk ut"))
print(senti.main("bisa komunikasi terus dengan rekan kerja walau dimanapun tempatnya"))
print(senti.main("bagus"))
print(senti.main("tugas kebanyakan kepala pusing paham tugas tidak"))
print(senti.main("kebanyakan tugas"))
print(senti.main("saya sudah mengunduh kali padahal tidak bisa memasang tetap masalah padahal aplikasi ini digunakan oleh kantor saya"))
print(senti.main("kurang bagus dan terlalu besar"))
print(senti.main("mantap"))
print(senti.main("bagus aplikasi"))
print(senti.main("bagus"))
print(senti.main("oke"))
print(senti.main("sering kali tidak bisa terhubung saat pertemuan"))
print(senti.main("sejauh ini aman aman saja"))
print(senti.main("dahsyat"))
print(senti.main("membantu sekali dalam mengirim² dokumen pekerjaan cuma akan baik lebih diperbaiki karena susah mendaftar menggunakan email pribadi karena saya pakai akun kantor"))
print(senti.main("sangat membantu dalam kerja saya"))
print(senti.main("wah saya sangat suka sekali ini ibel"))
print(senti.main("oke"))
print(senti.main("gagal masuk"))
print(senti.main("baik"))
print(senti.main("bagus"))
print(senti.main("sangat membantu pekerjaan saya selama ini terima kasih untuk yang membuat aplikasi ini"))
print(senti.main("kenapa di mengunduh tidak bisa ya"))
print(senti.main("oke"))
print(senti.main("sangat bagus dan mudah dipahami sayangnya belum ketemu cara terhapus chat lawan bicara admin mohon informasi apakah ada cara langkah proses terhapus chat lawan bicara"))
print(senti.main("aplikasi yang sangat bermanfaat"))
print(senti.main("bagus banget"))
print(senti.main("luar biasa keren menambah wawasan dan ilmu semoga bermanfaat untuk kita semua"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("kenapa tidak bisa ke buka ya kenapa ya"))
print(senti.main("mantap"))
print(senti.main("aplikasi yang sangat cocok untuk mengadakan rapat terutama di saya di perbankan karena cocok pada saat dilakukan pandemi waktu itu dan sekarang bisa digunakan dalam keadaan darurat walaupun di luar kota sekalipun"))
print(senti.main("oke"))
print(senti.main("tidak bisa masuk"))
print(senti.main("tidak bisa log masuk sudah hapus mengunduh berkali kalau"))
print(senti.main("bagus"))
print(senti.main("mantap"))
print(senti.main("bagus"))
print(senti.main("hebat"))
print(senti.main("bagus"))
print(senti.main("kebanyakan perbarui"))
print(senti.main("bagus"))
print(senti.main("bagus bisa saling konfirmasi sesama insan sig"))
print(senti.main("bagus"))
print(senti.main("kenapa ya jika aplikasi belum di buka tidak dapat notifikasi padahal sudah ada akun dulu sering mendapat notifikasi tapi mulai tahun kemarin sudah tidak mendapatkan notifikasi padahal sudah di perbarui mohon untuk di perbaiki atau adakah solusi lainnya terima kasih"))
print(senti.main("bagus"))
print(senti.main("saya memakai ini si sangat baik"))
print(senti.main("sangat bermanfaat untuk pekerjaan bagus"))
print(senti.main("mantap"))
print(senti.main("suka"))
print(senti.main("bagus"))
print(senti.main("saya kasih bintang dulu karena sering log keluar sendiri"))
print(senti.main("bagus lah jadi mudah"))
print(senti.main("mantap"))
print(senti.main("kebanyakan tugas min"))
print(senti.main("oke"))
print(senti.main("masalah log masuk terus"))
print(senti.main("mantap"))
print(senti.main("bagus cuy seru tidak"))
print(senti.main("susah masuk"))
print(senti.main("ini sudah diperbarui layar hitam terus sebelum diperbarui bagus ² saja saat diperbarui langsung layar hitam"))
print(senti.main("oke"))
print(senti.main("masih begini dulu nanti lihat"))
print(senti.main("sudah membuat email sudah di mengunduh tetap kakak bisa masuk pertemuan"))
print(senti.main("bagus"))
print(senti.main("mantap"))
print(senti.main("susah masuknya"))
print(senti.main("bagus"))
print(senti.main("tetap atas the bagus kerja"))
print(senti.main("tidak berfungsi"))
print(senti.main("aplikasi yang sangat membantu"))
print(senti.main("menyenangkan"))
print(senti.main("dulu ya"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("mantap"))
print(senti.main("kesalahan aplikasi"))
print(senti.main("kok di samsung tablet si lite tidak bisa masuk ke aplikasinya iya pas tekan tautan yang dikasih"))
print(senti.main("sangat membantu"))
print(senti.main("bagus"))
print(senti.main("buruk"))
print(senti.main("buruk"))
print(senti.main("bagus"))
print(senti.main("sering keluar padahal deh punya akun tapi suruh daftar baru lagi sekelas microsoft tidak menyangka untuk aplikasi seburuk ini"))
print(senti.main("mantap"))
print(senti.main("mantap"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("bagus aplikasi"))
print(senti.main("mantap"))
print(senti.main("bagus"))
print(senti.main("tiap hari meminta perbarui membuat habis kuota saja"))
print(senti.main("malah tidak bisa pakai yang domain outlook membuat repot saja"))
print(senti.main("tiap masuk lewat tautan di mintakan email nomor ponsel saat di memasukkan nomor salah aplikasi tidak guna"))
print(senti.main("tidak bisa masuk padahal kuota penuh sinyal stabil kenapa ya"))
print(senti.main("oke"))
print(senti.main("oke"))
print(senti.main("oke"))
print(senti.main("oke"))
print(senti.main("sangat bermanfaat untuk pekerjaan teams lebih mudah dan aman"))
print(senti.main("mantap"))
print(senti.main("bagus"))
print(senti.main("aplikasi kuat"))
print(senti.main("bagus banget"))
print(senti.main("bagus"))
print(senti.main("aplikasi hebat"))
print(senti.main("sangat membantu dalam pekerjaan kantor"))
print(senti.main("selalu log keluar sendiri giliran masukkan email malah salah"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("sangat memuaskan"))
print(senti.main("lumayan"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("lelet"))
print(senti.main("sudah baik"))
print(senti.main("oke"))
print(senti.main("oke"))
print(senti.main("oke"))
print(senti.main("bagus banget bisa mengerjakan tugas daring secara merata ke seluruh siswa walaupun jauh"))
print(senti.main("aplikasi ini bagus sebagai ruang dalam kegiatan belajar mengajar"))
print(senti.main("tidak ada ulasan"))
print(senti.main("baik"))
print(senti.main("di ponsel saya tidak bisa masuk keterangannya pasang pada perangkat lain"))
print(senti.main("oke"))
print(senti.main("mantap"))
print(senti.main("mudah dan sederhana satu"))
print(senti.main("mau masuk saja ribet banget kalau bukan karena kuliah sudah saya halus nih aplikasi dajal"))
print(senti.main("aplikasi yang sangat bermanfaat untuk proses pembelajaran dan berbagi materi"))
print(senti.main("oke"))
print(senti.main("baru mencoba"))
print(senti.main("oke"))
print(senti.main("harus lepas dan pasang ulang untuk bisa masuk keluar sendiri terus"))
print(senti.main("cukup puas menggunakan aplikasi ini"))
print(senti.main("bagus"))
print(senti.main("semoga berguna"))
print(senti.main("nomor juga buruk tapi sering  tiba tiba keluar"))
print(senti.main("oke"))
print(senti.main("terbaik"))
print(senti.main("bagus"))
print(senti.main("baik"))
print(senti.main("bagus"))
print(senti.main("baik sangat membantu"))
print(senti.main("sederhana mudah untuk gunakan"))
print(senti.main("sok jauh sok bagus"))
print(senti.main("setiap saya mengunduh selalu gagal padahal sudah mau coba saya ulang lagi tetap saja begitu kembali ke awal lagi akhirnya gagal temu interview lamaran kerja kesal"))
print(senti.main("mantap"))
print(senti.main("aplikasi tidak jelas susah di pakai"))
print(senti.main("bagus pekerjaan"))
print(senti.main("mantap"))
print(senti.main("ini sangat bagus untuk belajar daring wajib di mengunduh nih haha"))
print(senti.main("sangat bagus cuma kadang tidak ada notifikasi ₍•᎔•₎ა jadi sering ketinggalan temu atau terlambat mengumpulkan tugas ₍ ˃̵͈᷄ ˂̵͈᷅ ₎ა"))
print(senti.main("membuat orang emosi posisi rapat tiba “ keluar sendiri"))
print(senti.main("oke"))
print(senti.main("cukup membantu dalam pekerjaan"))
print(senti.main("aplikasi keren"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("mantap"))
print(senti.main("mantap"))
print(senti.main("baru mau coba pasti hebat"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("mantap"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("mantap"))
print(senti.main("bagus bisa membantu pertemuan"))
print(senti.main("kasih satu dulu kalau bagus saya tambah bintang"))
print(senti.main("tetap tingkatkan"))
print(senti.main("oke"))
print(senti.main("bagus cepat jelas"))
print(senti.main("mantap"))
print(senti.main("sangat membantu dalam kegiatan diklat daring dan pembelajaran"))
print(senti.main("bagus untuk pertemuan dari jauh"))
print(senti.main("masuknya sulit"))
print(senti.main("susah di mengunduh"))
print(senti.main("oke bagus"))
print(senti.main("aplikasinya sangat lambat"))
print(senti.main("oke"))
print(senti.main("sangat membantu"))
print(senti.main("berguna"))
print(senti.main("bagus"))
print(senti.main("mudah untuk gunakan"))
print(senti.main("mau masuk saja susah gagal terus email tidak dikenal lah nomor ponsel salah lah padahal untuk akun pakai email dan nomor yang terdaftar"))
print(senti.main("sangat oke"))
print(senti.main("sangat susah masuk untuk rapat malah tidak masuk padahal sudah tekan bergabung"))
print(senti.main("bagus aplikasi"))
print(senti.main("membantu perkuliahan"))
print(senti.main("bagus"))
print(senti.main("aplikasi lelet"))
print(senti.main("bagus"))
print(senti.main("praktis dan sederhana"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("bagus cukup membantu"))
print(senti.main("halo pengembang masih teams apa kabar masih sehat kangaji bulanannya lancar tidak ada masalah dengan pacar mantan wah mantap kalau semua tidak ada masalah harusnya kerja semakin bagus dong bukan semakin burik di gaji tiap bulan cuma makan gaji buta perbarui masih teams saja bukannya semakin bagus ini semakin burik memuatnya lelet malahan dari siput mengesot suka keluar tidak jelas mau masuk lagi malah tidak bisa beda banget dari masih teams versi dulu lembut sekarang perbarui versi lawaknya seson nya membuat mengakak wkwk"))
print(senti.main("mau interview malah log keluar sendiri jadi gagal interview"))
print(senti.main("aplikasi sangat bagus"))
print(senti.main("aplikasi ini sangat membantu saya untuk belajar ataupun sekolah"))
print(senti.main("tidak bisa digunakan padahal sudah masuk dengan akun microsoft mohon di perbaiki"))
print(senti.main("bagus aplikasi"))
print(senti.main("cocok untuk anak yang mengerjakan tugas lewat daring"))
print(senti.main("aplikasi jelek"))
print(senti.main("bagus"))
print(senti.main("tidak bisa di mengunduh"))
print(senti.main("mantap"))
print(senti.main("mantap"))
print(senti.main("lumayan"))
print(senti.main("mantap"))
print(senti.main("oke"))
print(senti.main("kenapa tidak bisa dibuka biasanya kalau gabung rapat tinggal tekan tautan langsung mau masuk sekarang harus isi mengunduh tautan mohon solusi"))
print(senti.main("kenapa sih kok tidak bisa dibuka padahal saat dicek tidak ada meminta perbarui aneh terkadang juga rapatnya tidak muncul maksud aplikasi ini apa kalau tidak perlu malas banget mengunduh nya"))
print(senti.main("selamat pagi"))
print(senti.main("jelek banget"))
print(senti.main("bagus sekali"))
print(senti.main("bagus dan sangat bermanfaat"))
print(senti.main("susah banget masuk aneh banget ganti sandi terus"))
print(senti.main("oke"))
print(senti.main("kok tidak bisa gabung ke kelas ya malah keluar sendiri meminta kata sandi terus"))
print(senti.main("mantab"))
print(senti.main("kenapa pengguna vivo tidak bisa mengunduh"))
print(senti.main("mantap"))
print(senti.main("bagus"))
print(senti.main("bagus pekerjaan"))
print(senti.main("bermasalah terus"))
print(senti.main("aplikasi nya sangat bagus"))
print(senti.main("mantap deh untuk pertemuan"))
print(senti.main("bagus"))
print(senti.main("keren"))
print(senti.main("keren"))
print(senti.main("baru buka sudah langsung keluar sendiri"))
print(senti.main("sudah mantap hanya video nya agar blur semoga bisa ditingkatkan lagi mantap"))
print(senti.main("sering keluar sendiri tolong perbaiki"))
print(senti.main("bagus"))
print(senti.main("seru"))
print(senti.main("tidak jelas"))
print(senti.main("mohon lebih di perbaiki lagi cara menjalankan aplikasinya sangat membingungkan"))
print(senti.main("nomor respon"))
print(senti.main("eror"))
print(senti.main("sudah ku lepas kenapa keluar lagi memuat dalam proses di tekan batal malah ponsel macet terus keluar dari aplikasi sendiri"))
print(senti.main("setelah di perbarui jadi tidak bisa mengumpulkan tugas tugas padahal tenggat waktu sebentar lagi alhasil mengumpulkan manual dengan chat dosen malu banget"))
print(senti.main("membantu"))
print(senti.main("mantap"))
print(senti.main("anjing lelet"))
print(senti.main("anjing kebanyakan lelet memek"))
print(senti.main("mantap"))
print(senti.main("oke"))
print(senti.main("membuat stres kontol"))
print(senti.main("sangat membantu saya"))
print(senti.main("mantap"))
print(senti.main("bagus pekerjaan"))
print(senti.main("masih teams di saat saat sekarang memerlukan pembaruan jadi selama pertemuan berlangsung biasanya permasalahan utama ialah terhentinya koneksi mikrofon ataupun kamera yang tidak bisa dinyalakan tombol bergabung tidak terlihat kesulitan masuk dari beberapa kendala saat ini saya mengharapkan aplikasi masih teams berbenah lebih baik lagi dan mengembalikan kualitas awal saat pertama digunakan untuk fitur pada aplikasinya cukup lengkap terima kasih atas perhatiannya"))
print(senti.main("bagus cepat"))
print(senti.main("bagus"))
print(senti.main("kenapa tidak bisa menggunakan background"))
print(senti.main("oke"))
print(senti.main("mantap"))
print(senti.main("maaf cuma soalnya jadi pajangan saja kantor saya sok suruh pakai teams tapi tidak pernah ada pertemuan"))
print(senti.main("terima kasih membantu sekali"))
print(senti.main("jelek"))
print(senti.main("oke"))
print(senti.main("bintang soalnya susah di pasang perlu di tingkatkan lagi layanannya"))
print(senti.main("oke bagus terima kasih"))
print(senti.main("mantap"))
print(senti.main("tidak ada"))
print(senti.main("oke"))
print(senti.main("wow"))
print(senti.main("log masuk harus berkali"))
print(senti.main("bagus"))
print(senti.main("tidak bisa bergabung rapat payah ini aplikasi"))
print(senti.main("eror susah masuk ke teams gagal saya interview sama pengguna"))
print(senti.main("jelek banget ampun bolak balik mau tanda masuk dari tautan rapat ya masuk keluar aku tanda keluar mau masuk lagi malah tidak bisa"))
print(senti.main("lebih ditambahkan lagi fitur fitur nya"))
print(senti.main("i tidak bisa bergabung teams melalui undangan  tautan i punya masuk ku masuk informasi tapi itu tetap bertanya saya untuk masuk tolong perbaiki ini eror sok i bisa mendapatkan menuju teams terima kasih"))
print(senti.main("bagus"))
print(senti.main("sialan"))
print(senti.main("biasa saja v"))
print(senti.main("aplikasi ini sangat bagus tapi ada kekurangan transkrip karena belum ada tersedia bahasa indonesia saya adalah disabilitas rungu tuna rungu mohon tersedia transkripsi bahasa indonesia karena saya butuh akses transkrip untuk mempermudah komunikasi mohon tolong menambahkan transkrip akses untuk disabilitas rungu mohon pembaruannya untuk tersedia transkripsinya terima kasih ya"))
print(senti.main("ini adalah the satu sesuatu itu microsoft di benar lainnya tahan masih kantor"))
print(senti.main("sangat membantu saat sekolah daring"))
print(senti.main("bagus"))
print(senti.main("speaker audio tidak terdengar"))
print(senti.main("bagus untuk sekolah atau kerja kelompok tapi ada eror tugasnya banyak banget anjing"))
print(senti.main("aplikasi laknat anjing"))
print(senti.main("kebanyakan autentikasi kebelet laku apa bagaimana aplikasinya"))
print(senti.main("bagus"))
print(senti.main("sangat jelek dan kurang baik tolong diperbaiki"))
print(senti.main("bagus"))
print(senti.main("suka tiba² aplikasi eror tidak bisa masuk kakak pertemuan zoom"))
print(senti.main("hmm sudah aku mengunduh tapi tidak bisa di pakai"))
print(senti.main("jelek banget aplikasi nya tolong ditingkatkan lagi agar tidak selalu eror"))
print(senti.main("tiba² akun saya keluar meminta kata sandi"))
print(senti.main("susah banget ini indah teams kesal"))
print(senti.main("oke"))
print(senti.main("ku kira komitmen ternyata hanyalah momen"))
print(senti.main("eror nih aplikasi padahal sudah masuk memakai kata sandi yang benar tapi malah salah sandinya padahal sebelumnya masih bisa"))
print(senti.main("sering tidak bisa masuk"))
print(senti.main("baiklah"))
print(senti.main("tidak jelas nih aplikasi akun saya log keluar terus sudah mah susah untuk log masuk"))
print(senti.main("tidak bisa bergabung dengan rapat akun pribadi semakin ribet"))
print(senti.main("mantap"))
print(senti.main("oke"))
print(senti.main("baru bulan di mengunduh malah di suruh masuk di teams baru"))
print(senti.main("itu membantu saya belajar nyala sudah ku sekolah"))
print(senti.main("semoga harimu senin terus"))
print(senti.main("mantap"))
print(senti.main("tidak bisa masuk harus begini begitu ribet"))
print(senti.main("keren"))
print(senti.main("tingkatkan"))
print(senti.main("versi android tidak bisa hapus file yang di mengunggah"))
print(senti.main("bagus banget"))
print(senti.main("mengunduh nya susah"))
print(senti.main("pekerjaan menjadi mudah"))
print(senti.main("oke"))
print(senti.main("aplikasinya bagus"))
print(senti.main("untuk belajar"))
print(senti.main("mantap"))
print(senti.main("oke"))
print(senti.main("susah masuk"))
print(senti.main("bagus keberuntungan"))
print(senti.main("bagus"))
print(senti.main("oke"))
print(senti.main("bagus dan sangat memuaskan"))
print(senti.main("jangan mendadak suka perbarui waktu mengerjakan tugas suka keluar sendiri"))
print(senti.main("aplikasi bagus tapi kenapa ada sering notifikasi tugas di aplikasi ini tolong di perbarui lagi"))
print(senti.main("bagus"))
print(senti.main("sangat lah bagus aplikasi nya"))
print(senti.main("bagi menurut ku sih aplikasi ini bagus banget mohon di jawab ya play store"))
print(senti.main("tampilan sering ganti ganti dan sekarang tempat tugas hilang dan saya bingung"))
print(senti.main("bagus"))
print(senti.main("bagus sih untuk belajar terus mengobrol sama murid yang tidak punya nomor whatsapp cuma masalahnya itu eror profilnya tiap mau ubah profilnya pasti selalu eror jadi misal foto profil lama tapi kalian berubah pikirkan mau ganti profil baru tapi saat kalian ganti dia malah berubah jadi profil lama jadi tolong pengembangnya perbaiki ya eror profilnya"))
print(senti.main("luar biasa"))
print(senti.main("perbarui yang benar lah anjing sebulan sekali atau bagaimana jangan tiap hari cok"))
print(senti.main("sering otomatis keluar reset sandi sudah bisa di reset masih saja kirim kode ke gmail untuk masuk tetap tidak bisa mengesalkan jadi ketinggalan telat untuk temu tolong diperbaiki microsoft microsoft"))
print(senti.main("oke"))
print(senti.main("oke"))
print(senti.main("oke"))
print(senti.main("butuh lebih peningkatan nyala the media pemain orang pada terakhir memakai itu mungkin untuk menjadi dimainkan nyala lainnya media pemain dan the animasi itu muncul kapanpun kamu giliran masuk tugas adalah selalu tersendat"))
print(senti.main("bermanfaat"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("aplikasi lelet"))
print(senti.main("dicoba"))
print(senti.main("jelek terlalu banyak tugas"))
print(senti.main("aplikasinya jelek sekali susah untuk bergabung rapat melalui tautan selalu minta masuk padahal sudah masuk"))
print(senti.main("mantap"))
print(senti.main("jelek banget tiba masak log keluar sendiri mana di suruh masukkan sandi katanya sandi salah terus jelek pokok nya jelek"))
print(senti.main("kenapa ya tiap mau pertemuan dan dibuka aplikasi ini langsung gagal keterangan salah email salah sandi padahal jelas itu sandinya saya mencatat sandi saya tapi akan pertemuan selalu log keluar sendiri dan meminta verifikasi sandi baru yang membuat saya ketinggalan pertemuan padahal sebelum jam pertemuan sayang sudah cek mau akun dan semuanya lancar sampai saat waktunya pertemuan sudah berulang kali begini bahkan kemarin saya ganti lagi sandi saya lalu coba masuk lagi dan benar saja teams langsung gagal lagi"))
print(senti.main("oke"))
print(senti.main("saya bisa daring dengan teman teman"))
print(senti.main("oke"))
print(senti.main("pakai oppo kok tidak bisa ganti latar"))
print(senti.main("oke"))
print(senti.main("sangat membantu dalam beraktivitas khususnya dengan pihak internal maupun eksternal"))
print(senti.main("sangat bagus aplikasi untuk pekerja dan profesional"))
print(senti.main("teams sangat membantu dalam bidang komunitas yang banyak"))
print(senti.main("baru ikut gabung"))
print(senti.main("sangat bagus untuk pertemuan orang mengajar"))
print(senti.main("sangat membantu terima kasih"))
print(senti.main("jelek"))
print(senti.main("bermanfaat banget untuk kerja"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("bagus sekali untuk belajar tapi kalau di laptop eror tolong di perbaiki"))
print(senti.main("oke"))
print(senti.main("sangat berguna"))
print(senti.main("aplikasi terbaik untuk belajar"))
print(senti.main("bagaimana sih kode verifikasi tidak mau muncul padahal darurat banget"))
print(senti.main("keren"))
print(senti.main("jelek kali masak iya mikrofon sudah nyala tapi suaranya tidak keluar tolong dong di perbarui lagi untuk mikrofonnya"))
print(senti.main("keren sih walau terkadang menyusahkan"))
print(senti.main("versi yang lama masih bagus tapi versi ini kok ribet amat suara tidak jelas video tidak jelas bahkan terputus sendiri kalau bisa untuk aplikasi yang baik sedikit ini di pakai untuk pertemuan tidak jadi terima kasih"))
print(senti.main("sangat puas"))
print(senti.main("sangat membantu terima kasih"))
print(senti.main("aplikasi ini sangat membantu saya saat kuliah"))
print(senti.main("suara nya kecil banget"))
print(senti.main("kok di pasang tidak bisa lagi ya"))
print(senti.main("oke"))
print(senti.main("kenapa teams selalu meminta di perbarui terus lagi pertemuan tidak bisa masuk karena harus perbarui mana lama pula perbarui nya"))
print(senti.main("mantap"))
print(senti.main("suka eror"))
print(senti.main("bagus"))
print(senti.main("top"))
print(senti.main("bagus"))
print(senti.main("tidak bisa bergabung selalu daftar lagi daftar lagi asem benaran"))
print(senti.main("bagus"))
print(senti.main("sangat membantu"))
print(senti.main("semoga kedepannya lebih baik lagi"))
print(senti.main("sangat bagus"))
print(senti.main("semoga di lancar kan rezeki"))
print(senti.main("mengunduh nya selalu tidak bisa cuma sekali sudah kali mengunduh waktu sudah balik lagi ke padahal penyimpanan juga kosong kuota habis aplikasi tidak ke mengunduh"))
print(senti.main("untuk akun susah amat ponsel nih ku coba"))
print(senti.main("susah masuk"))
print(senti.main("banget kerja"))
print(senti.main("mantap"))
print(senti.main("sangat membantu proses kuliah"))
print(senti.main("aplikasinya suka eror tidak bisa untuk masuk tolong di perbaiki"))
print(senti.main("oke"))
print(senti.main("kok tidak bisa masuk sih kenapa ya"))
print(senti.main("kecewa berat tiap mau ikut meminta verifikasi terus"))
print(senti.main("bagus"))
print(senti.main("share screen dengan audio adalah tidak bekerja"))
print(senti.main("bagus"))
print(senti.main("login nya super rumit kalau tidak karena pekerjaan tidak banget saya mengunduh aplikasi kaya begini sering eror pula harus di lepas lagi"))
print(senti.main("play store ku jadi tidak bisa"))
print(senti.main("ini aplikasi paling bagus karena semuanya itu lancar keren juga"))
print(senti.main("sangat membantu di saat pandemi"))
print(senti.main("kontol"))
print(senti.main("unggah video lama banget"))
print(senti.main("luar biasa"))
print(senti.main("sok banyak eror terkadang i berarti sering  i tidak bisa bergabung a temu masuk a grup karena disana adalah nomor bergabung tombol dan ku teman teman didalam the grup tidak bisa panggilan saya karena i dulu tidak masuk the grup the aplikasi berkata bahkan meskipun saya sudah bergabung the grup dan masih didalam itu itu mengecewakan"))
print(senti.main("bagus"))
print(senti.main("aplikasi yang bagus"))
print(senti.main("super buruk saat beberapa orang posting tidak ada notifikasi notifikasi akan muncul waktu kita balas postingan nya dan aplikasi sangat berat tidak cocok untuk rendah perangkat"))
print(senti.main("tidak bekerja masuk android marshmellow"))
print(senti.main("tolong dikasih fitur nama yang mengeluarkan kita dari temu"))
print(senti.main("bagus aplikasi"))
print(senti.main("kadang mikrofonnya tidak nyala walau sudah diatur"))
print(senti.main("sudah baik"))
print(senti.main("mantap"))
print(senti.main("aplikasi ini sangat membantu memudahkan dalam diskusi pembelajaran daring"))
print(senti.main("memasang nya lama banget aplikasi kontol"))
print(senti.main("bagus aplikasi untuk pertemuan"))
print(senti.main("aplikasi busuk tidak bisa di pakai kerja"))
print(senti.main("bagus"))
print(senti.main("bagus melanjutkan pengembangan"))
print(senti.main("oke"))
print(senti.main("aplikasi ini sangat bagus"))
print(senti.main("bagus"))
print(senti.main("teams sangat tidak penting"))
print(senti.main("bagus"))
print(senti.main("lelet jika ada pemberitahuan perbarui itu kita tidak bisa bergabung rapat selalu keluar dan tidak bisa masuk harus di perbarui dulu perbarui nya juga lama"))
print(senti.main("susahnya jaringan"))
print(senti.main("bagus untuk pertemuan bersama"))
print(senti.main("apak banget atasan"))
print(senti.main("terima kasih aplikasi teams berjalan dengan lancar"))
print(senti.main("baru mencoba semoga bagus ya"))
print(senti.main("sangat bermanfaat"))
print(senti.main("bagus"))
print(senti.main("tidak bisa perbarui"))
print(senti.main("bagus"))
print(senti.main("kalau yang presentasi share screen nya pakai ponsel entah kenapa di masih teams saya malah jadinya dobel perangkat meskipun sudah di transfer rapat nya tetap saja sama"))
print(senti.main("susah masuk ke pertemuan"))
print(senti.main("kalau dari ponsel pintar android belum ada fitur background efek yang lain sudah bisa yang belum sampai sekarang deh lama banget agak lambat penyesuaiannya bahkan lambat sekali"))
print(senti.main("oke"))
print(senti.main("aplikasi yang bagus"))
print(senti.main("jelek"))
print(senti.main("sangat membantu"))
print(senti.main("tidak aplikasi"))
print(senti.main("aplikasi ramah"))
print(senti.main("oke tapi kalau pakai ponsel selalu tidak bisa buka form tugas tolong di perbaiki"))
print(senti.main("boleh"))
print(senti.main("sering keluar tidak jelas terus masuk masih eh tidak bisa gabung"))
print(senti.main("saat ingin menampilkan ppt sangat ribet harus simpan drive sudah di simpan drive tidak terbaca sistem teams"))
print(senti.main("teams kenapa sih eror mulu untuk buka aplikasi lain seperti macet begitu tidak bisa dipencet"))
print(senti.main("sering tidak bisa masuk"))
print(senti.main("mantap kualitas audio dan video"))
print(senti.main("bagus"))
print(senti.main("jelek amat aplikasi teams kali titip minggu perbarui ulang terus"))
print(senti.main("agak kurang kalau lagi rapat suka keluar sendiri terus harus gonta ganti akun suka kesal kalau tiba eror"))
print(senti.main("mantap"))
print(senti.main("bagus"))
print(senti.main("aplikasi jelek banget padahal kemarin masih bisa di pakai kok ke log keluar sendiri masuk lagi tidak bisa"))
print(senti.main("penting untuk komunikasi kerja saya"))
print(senti.main("baik dan bagus"))
print(senti.main("mantap"))
print(senti.main("tolong admin meminta diperbaiki ketika presentasi mulai gambar ppt nya tidak bisa muncul"))
print(senti.main("bagus"))
print(senti.main("sangat baik"))
print(senti.main("suara jernih mudah menggunakan lebih ringan aplikasinya"))
print(senti.main("sangat membantu untuk rapat daring"))
print(senti.main("apaan tidak jelas buang buang kuota saja"))
print(senti.main("bagus"))
print(senti.main("mantap"))
print(senti.main("aplikasi bagus hebat"))
print(senti.main("aplikasi jelek jelek jelek"))
print(senti.main("bagus ide"))
print(senti.main("mantab"))
print(senti.main("bagus tapi suka tersendat"))
print(senti.main("mohon bantuannya akhir akhir ini saya sering mengalami eror dimana saya ingin masuk pertemuan ruangan namun saya tidak menemukan ruangan tersebut didalam grup padahal teman teman yang lain menemukannya"))
print(senti.main("mantap"))
print(senti.main("oke"))
print(senti.main("aplikasi apaan ini dapat tautan untuk rapat mau masuk tidak masuk"))
print(senti.main("sering eror temu rapatnya tidak muncul tombol bergabung"))
print(senti.main("bagus"))
print(senti.main("kenapa susah banget di mengunduh"))
print(senti.main("sangat bagus dan saya sangat menyukai aplikasi microsoft teams"))
print(senti.main("bagus aplikasi"))
print(senti.main("masuknya susah betul bolak balik sandi bolak balik sandi"))
print(senti.main("bagus"))
print(senti.main("membuat orang ganggu tidur saja"))
print(senti.main("masuk susah"))
print(senti.main("sangat membantu"))
print(senti.main("aplikasi yang payah"))
print(senti.main("cukup baik untuk pembelajaran"))
print(senti.main("oke"))
print(senti.main("aku suka teams karena aku bisa belajar dengan aman walaupun kadang tersendat aku bahagia terima kasih teams"))
print(senti.main("eror banget tiap suruh memasukkan nomor ponsel mau mengirim sandi tidak di kirim menunggu lama lama gagal"))
print(senti.main("parah setiap rapat mesti pemasangan baru"))
print(senti.main("sangat bermanfaat"))
print(senti.main("chat di ruangan temu masih belum terbaca di android"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("nomor komentar"))
print(senti.main("kadang susah bergabung sudah ada email disuruh untuk lagi"))
print(senti.main("lagi interview nya ini kamera tidak bisa nyala padahal sudah pencet kamera nyala speaker sudah hidup malah tidak kedengaran padahal tidak disunyikan"))
print(senti.main("bagus aplikasi"))
print(senti.main("bagus"))
print(senti.main("semakin tidak jelas aplikasi ini masa harus diperbarui terus tiap minggu"))
print(senti.main("oke"))
print(senti.main("bagus untuk pertemuan daring"))
print(senti.main("ya bagus"))
print(senti.main("jelek aplikasinya jelek amat bolak balik mau bergabung pertemuan saja tidak bisa mau masuk saja susah nya meminta ampun padahal tidak pernah ganti kata sandi atau email tapi tidak bisa masuk jelek banget nih aplikasi pengembang tolong perbaiki"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("bagus aplikasi"))
print(senti.main("sering ke keluar sendiri"))
print(senti.main("sangat membantu pembelajaran"))
print(senti.main("tidak bisa gabung padahal sudah diperbarui"))
print(senti.main("gagal masuk terus harus lepas baru pasang lagi masa harus begitu begitu terus"))
print(senti.main("suka"))
print(senti.main("microsoft teams suara jernih dan lebih stabil"))
print(senti.main("lelet"))
print(senti.main("mudah di gunakan"))
print(senti.main("tidak bisa masuk"))
print(senti.main("bagus dan berguna untuk daring dan mengerjakan tugas"))
print(senti.main("bagus sederhana"))
print(senti.main("bagus"))
print(senti.main("bagus untuk belajar daring"))
print(senti.main("mantap dan sangat bagus"))
print(senti.main("khusus android sudah tidak mendukung android"))
print(senti.main("bagus pekerjaan"))
print(senti.main("bagus banget aku jadi ingin menjadi penyatir"))
print(senti.main("anjing anjing kok tidak ada tombol teams nya padahal mau gabung kode kelas anjing anjing anjing anjing anjing membuat emosi blog cok"))
print(senti.main("sumpah menyusahkan banget setiap hari eror masak hari mengunduh tiap sebentar mohon perbaikan nya admin"))
print(senti.main("bagus"))
print(senti.main("sudah oke jika bisa ditambah lagi agar bisa di pasang di android tv"))
print(senti.main("kenapa ya kok susah gabung di rapat belakangan ini kadang susah bergabung di rapat atau keluar sendiri dari rapat dan kadang audio hilang²"))
print(senti.main("kok tidak bisa diunduh harus mengulang lagi"))
print(senti.main("foto profil saya kok jadi burik banget anjing"))
print(senti.main("kenapa tidak bisa bergabung digrup kelas ya mohon penjelasannya"))
print(senti.main("tolong dong di perbaiki lagi ini aplikasi nya kadang suka keluar sendiri"))
print(senti.main("bagus"))
print(senti.main("oke bagus"))
print(senti.main("lama banget masuk nya tiap minggu perbarui an mah tolong lah di perbaiki admin"))
print(senti.main("jelek banget aplikasi nya mudah eror mengunduh nya lama banget butuh waktu berjam jam unduh nya"))
print(senti.main("aplikasinya sering eror susah masuk ke aplikasi suara sering hilang ke batal sunyikan sendiri sering kali keluar kata memegang nyala"))
print(senti.main("eror terus"))
print(senti.main("bagus"))
print(senti.main("suka keluar sendiri aplikasi memek"))
print(senti.main("tiap dikasih tautan meminta masuk mulu tapi tidak bisa masuk"))
print(senti.main("oke"))
print(senti.main("aplikasinya jelek banget eror terus tidak bisa ikut rapat"))
print(senti.main("suka tidak bisa masuk aneh"))
print(senti.main("susah masuk masa setiap kali mau bergabung pertemuan harus masuk berkali² padahal sudah biasa pakai akun yang itu tapi harus masuk terus setiap mau bergabung dulu kalau sudah masuk langsung bergabung berkali² mah bebas bisa sekarang malah susah"))
print(senti.main("aplikasi yang sangat bermanfaat menambah pengetahuan dan informasi kegiatan tuweb diperkuliahan tapi butuh perjuangan dan kesabaran untuk pasang aplikasinya"))
print(senti.main("awalnya saya lepas terus mau mengunduh lagi kenapa tidak bisa"))
print(senti.main("aplikasi tidak jelas"))
print(senti.main("bagus"))
print(senti.main("bagus sih menurut ku microsoft teams ini mungkin ada beberapa hal yang susah karena jaringan sinyal karena kalau pakai teams ini harus memang benar benar stabil"))
print(senti.main("bagus"))
print(senti.main("makanya kuliah di universitas terbuka"))
print(senti.main("iya eror banget kesal deh saya"))
print(senti.main("aplikasi ini suka eror jelek"))
print(senti.main("the terbaik"))
print(senti.main("mantap"))
print(senti.main("bagus"))
print(senti.main("mantap"))
print(senti.main("kenapa tidak bisa bagikan layar presentasi"))
print(senti.main("akunya suka keluar sendiri"))
print(senti.main("kenapa di samsung a tidak bisa mengunggah file dari ponsel ya tidak ada tombol mengunggah mohon petunjuk nya"))
print(senti.main("cukup memuaskan tapi kenapa ya kadang tidak bisa dibuka"))
print(senti.main("tolong ram nya di kecil kan tolong"))
print(senti.main("kalau bukan karena kuliah harus pakai ini tidak mau jelek banget sering keluar dan gagal masuk terus"))
print(senti.main("cara login nya membingungkan ulang² diminta email nomor ponsel nama pengguna"))
print(senti.main("saya suka sekali sama aplikasi teams ini karena dengan aplikasi teams ini kita bisa mempelajari banyak hal bagaimana cara mengunggah tugas dan melihat tugas yang di berikan oleh guru"))
print(senti.main("kenapa ya saat lagi temu begitu suka keluar sendiri sudah berkali log keluar masuk sampai pasang lepas tapi tetap sama saja membuat bolos kuliah kali huwa menangis banget"))
print(senti.main("bagus"))
print(senti.main("sudah diperbarui tapi kenapa tidak bisa bergabung pertemuan ya tolong dong dibantu bagaimana caranya"))
print(senti.main("oke"))
print(senti.main("bisa belajar hehe"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("audio mati mulu membuat ketinggalan informasi kalau lagi pertemuan"))
print(senti.main("bodoh"))
print(senti.main("oke"))
print(senti.main("aplikasi tidak jelas masa mau bergabung pertemuan tidak ada tombolnya sudah di perbarui masih saja seperti begitu"))
print(senti.main("stres sih kadang enak kadang tersendat"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("kok bolak balik tulisannya maaf ada kesalahan ya padahal wifi lancar"))
print(senti.main("bagus"))
print(senti.main("ini untuk ku sekolah terima kasih teams"))
print(senti.main("sangat bagus untuk belajar"))
print(senti.main("direkomendasikan dan bagus aplikasi"))
print(senti.main("bagus"))
print(senti.main("bermasalah ketika membuka tugas dan sangat tidak stabil"))
print(senti.main("sering eror"))
print(senti.main("aplikasi eror mulu"))
print(senti.main("oke"))
print(senti.main("bagus sekali aplikasi"))
print(senti.main("anjing bodoh banget sesudah saja daftar sudah bisa kok tidak bisa masuk"))
print(senti.main("oke"))
print(senti.main("seperti anjay"))
print(senti.main("oke"))
print(senti.main("sering tersendat tb²"))
print(senti.main("nyaman untuk digunakan"))
print(senti.main("mantap"))
print(senti.main("oke"))
print(senti.main("alhamdulillah selama memakai teams selama bulan baik di ponsel pintar dan laptop aman jaya tidak ada kendala sering membuat rapat dan lain lain tidak terdapat kendala yang terlalu fatal pengalaman selama penggunaan sudah membuat rapat dan terdapat peserta rapat yang masuk ke ruangan yang dibuat dan terdapat terlempar ke ruangan lain yang tidak ada admin solusi admin membuat temu baru dan kendala tersebut terselesaikan"))
print(senti.main("bagus kok adik adik"))
print(senti.main("oke"))
print(senti.main("itu a aneh aplikasi kapan kamu registrasi untuk an akun kamu bahkan bertanya untuk registrasi berulang ulang the daftar adalah sungguh aneh untuk itu dari kamu siapa ingin untuk mengunduh itu identitas merekomendasi the zoom aplikasi"))
print(senti.main("pasang lagi tidak bekerja buruk aplikasi"))
print(senti.main("suara dan video sering  bocor padahal sudah dimatikan atau disunyikan mohon diperbaiki untuk keamanan privasi terima kasih"))
print(senti.main("tolong jangan diulangi datanya"))
print(senti.main("bagus"))
print(senti.main("saya mengasih bintang karena aplikasi ini sangat bagus dan saya bisa mengikuti pembelajaran dengan nyaman meskipun kadang suka tersendat atau eror tapi bagi saya itu tidak penting"))
print(senti.main("kenapa tidak bisa di mengunduh ya terus aplikasinya sering bermasalah"))
print(senti.main("mudah eror"))
print(senti.main("bagus"))
print(senti.main("cepat"))
print(senti.main("bagus"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("tidak nyaman karena biasanya tautan konferensi video ada tapi sekarang tidak ada tolong diperbaiki terima kasih"))
print(senti.main("dengan ada nya aplikasi ini saya bisa memudah kan untuk ada acara atau pertemuan"))
print(senti.main("terbaik memang"))
print(senti.main("tidak bisa di mengunduh"))
print(senti.main("keluh kesah setiap mau zoom pasti keluar akun harus mengunduh lagi"))
print(senti.main("tidak ada notifikasi jadi tidak tau perkuliahan rapat sudah di mulai atau belum dan harus membuka masih teams saat di buka harus di refresh terlebih dahulu"))
print(senti.main("oke"))
print(senti.main("mantap"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("wah bagus banget aplikasinya kalian wajib mengunduh deh keren"))
print(senti.main("beberapa minggu ini masih teams saya tidak bisa share screen dan menyalakan mikrofon serta mengalami kendala dalam membuka masih teams"))
print(senti.main("jangan di mengunduh tidak rekomendasi untuk pertemuan"))
print(senti.main("terima kasih bisa digunakan untuk bekerja"))
print(senti.main("sangat membantu"))
print(senti.main("eror mulu susah bergabung kalau deh bisa bergabung tiba tiba keluar sendiri terus mau bergabung lagi susahnya meminta ampun tolong dong diperbaiki"))
print(senti.main("mau masuk tidak bisa sepertinya perbarui terbaru malah tidak gagal"))
print(senti.main("mantap"))
print(senti.main("aplikasi jelek"))
print(senti.main("lumayan membantu"))
print(senti.main("terima kasih sudah memberikan kesempatan jadi mitra pt terbaik semangat baru untuk hidup yang lebih baik dimasa depan semangat pt terbaik"))
print(senti.main("kadang eror aplikasi ini payah"))
print(senti.main("lebih sering paksa tutup dari pada versi sebelumnya"))
print(senti.main("aplikasi yang bagus untuk melakukan kegiatan sekolah kantor saya hanya ingin memberi masukkan sedikit menurut saya lebih baik lagi jika masih teams yang sekarang ini diubah seperti dulu lagi setiap selesai merekam rapat hasil rekaman otomatis langsung tersimpan di file dibandingkan yang sekarang lebih baik seperti itu yang dulu karena jika seperti yang saat ini itu agak mempersulit untuk melihat rekamannya perlu dibuka dari laptop komputer atau melalui aplikasi lain sedangkan yang dulu bisa dibuka melalui akses apa saja terima kasih"))
print(senti.main("berubah terus"))
print(senti.main("sangat membantu saya dalam kegiatan pertemuan rutin mingguan bersama mahasiswa"))
print(senti.main("terkadang eror"))
print(senti.main("komunikasi yang baik"))
print(senti.main("kok tidak bisa ya admin ini murid murid jadi tidak bisa ujian"))
print(senti.main("malas virtual"))
print(senti.main("mengunduh nya pun lama kali"))
print(senti.main("tolong dong jangan sering keluar kalau lagi pertemuan ribet ini dan menghabiskan waktu"))
print(senti.main("bagus"))
print(senti.main("suka sekali microsoft teams"))
print(senti.main("susah masuk"))
print(senti.main("buruk sekali teams ini"))
print(senti.main("mudah untuk masuk ke layanan zoom"))
print(senti.main("sangat baik"))
print(senti.main("aplikasi ini sangat bagus"))
print(senti.main("cepat"))
print(senti.main("mantap"))
print(senti.main("baik sekali"))
print(senti.main("bagus"))
print(senti.main("sangat membantu"))
print(senti.main("tidak bisa di pasang"))
print(senti.main("mantap"))
print(senti.main("notifikasi aplikasi tidak muncul dan anehnya kok mode default nya mensunyikan ya untuk teams harusnya kan default nya hidup notifikasi aplikasi tidak memunculkan notifikasi saat aplikasi di tutup padahal sudah keadaan lonceng notifikasi hidup harusnya pengembang untuk aplikasi yang lebih ringan karena fitur di teams menurut saya banyak cuma tidak ke pakai tidak saya perlukan contohnya perpanjangan yang tidak penting"))
print(senti.main("bagus"))
print(senti.main("bagus sekali"))
print(senti.main("bagus"))
print(senti.main("keunggulan"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("eror"))
print(senti.main("tolong dong microsoft teams saya bermasalah setiap mau mengikuti rapat daring dari tautan undangan rapat tidak bisa malah tertera belum unduh aplikasi padahal sebelum sebelumnya baik saja"))
print(senti.main("tidak asyik"))
print(senti.main("bagus"))
print(senti.main("sangat membantu kinerja"))
print(senti.main("karena membuat saya banyak tugas"))
print(senti.main("oke"))
print(senti.main("aplikasi ini sangat berguna sekali untuk sekolah daring kegunaannya juga sangat bermacam macam terima kasih untuk pembuatnya karena sudah membuat aplikasi ini"))
print(senti.main("mantap"))
print(senti.main("eror mikrofon saya tidak berfungsi terus"))
print(senti.main("kok tidak bisa perbarui memuat saja terus menjadi menghambat pemasangan aplikasi yang lain hadeh"))
print(senti.main("oke"))
print(senti.main("ponsel poco kali kok tidak bisa masuk"))
print(senti.main("the aplikasi tidak stabil itu selalu paksa tutup setelah i mendapatkan menuju a pertemuan untuk a beberapa detik"))
print(senti.main("sering terkejut dari aplikasi"))
print(senti.main("coba dulu"))
print(senti.main("terima kasih"))
print(senti.main("sangat suka"))
print(senti.main("aplikasinya bagus tidak ada kendala yang membuat kendala di otak cuma tugas gurunya saja"))
print(senti.main("optimal anda"))
print(senti.main("oke"))
print(senti.main("terima kasih aplikasi teams sangat mudah digunakan untuk belajar"))
print(senti.main("suara sering hilang"))
print(senti.main("aplikasi berat memuat lama sampai sekarang keluar sendiri belum bisa masuk kembali"))
print(senti.main("bagus aplikasi"))
print(senti.main("ini dulu sok membantu"))
print(senti.main("bagus"))
print(senti.main("keren"))
print(senti.main("aplikasinya sangat baik dan sangat membantu"))
print(senti.main("oke"))
print(senti.main("sudah diregistrasi tapi buka tautan untuk rapat tidak bisa mohon diperbaiki lebih baik"))
print(senti.main("sangat bagus"))
print(senti.main("netral"))
print(senti.main("sekarang ya ampun tersendat nya bukan main notifikasi telepon tidak muncul buka tugas terjadi kesalahan terus chat tidak terkirim foto jadi emosikon kotak doang screen tidak kelihatan mikrofon mati sendiri tiba tiba pergi dari pertemuan tertunda chat lama tidak ada notifikasi padahal sudah dinyalakan semua tolong diperbaiki ya jangan sampai karena hal ini kita tidak bisa belajar dan penurunan nilai terus menerus hanya karena aplikasinya terima kasih jika ada kata kata yang salah mohon dimaafkan"))
print(senti.main("kamu satir"))
print(senti.main("mantap"))
print(senti.main("sok bagus ini mah"))
print(senti.main("bagus"))
print(senti.main("ini sang pada bagus sekali untuk sekolah dan sekolah minggu"))
print(senti.main("terima kasih kamu"))
print(senti.main("terima kasih"))
print(senti.main("bagus bisa sekolah daring"))
print(senti.main("sudah masuk masih disuruh masuk"))
print(senti.main("mempermudah metkor"))
print(senti.main("jelek"))
print(senti.main("bagus"))
print(senti.main("lebih detail dan cepat lagi ketika menginformasikan ada tugas yang telah dikirim maka akan saya beri bintang"))
print(senti.main("mudah di gunakan apalagi saya yang wfh sangat membantu"))
print(senti.main("beberapa bulan belakang teams suka eror tombol bergabung temu suka tidak muncul kalaupun muncul kadang bisa menimbulkan temu baru sehingga jadi banyak temu di dalam channel nama kadang tidak muncul di partisipan kadang suka tertendang sendiri dari temu"))
print(senti.main("baik sangat memuaskan"))
print(senti.main("tolong aplikasi nya jelek soalnya setiap mau masuk ikut rapat keluar tidak beralih ujungnya di mengunduh ulang"))
print(senti.main("bagus aplikasinya"))
print(senti.main("tolong ini nge perbarui tapi eror sudah di lepas malah tetap ada di daftar aplikasi perbarui"))
print(senti.main("play store eror gara gara aplikasi ini"))
print(senti.main("kenapa tidak bisa di pasang"))
print(senti.main("oke"))
print(senti.main("aplikasinya payah sekarang harus bolak balik pasang ulang sedang dipakai tiba keluar sendiri"))
print(senti.main("apa² sudah mengunduh malah tidak keluar di menu suruh mengunduh terus sampai kuota habis"))
print(senti.main("terima kasih teams aplikasi yang setiap hari minggu saya gunakan untuk mengajar"))
print(senti.main("sangat mudah di gunakan namun terkadang masih suka keluar sendiri"))
print(senti.main("awalan bagus di pakai kali sudah kelihatan buriknya maaf tolong di perbaikilah admin jangan mempersulit pengguna"))
print(senti.main("mohon di tingkatkan lagi ya soalnya kadang mau mati saat lagi rapat"))
print(senti.main("tolong aplikasi ini di perbarui lagi"))
print(senti.main("sangat bagus"))
print(senti.main("kurang efektif banget masa tiap minggu dan tiap jam mesti lepas dan pasang terus membuat kuliah jadi tidak efektif"))
print(senti.main("tidak dapat masuk rapat harus terus meng pasang aplikasi"))
print(senti.main("sering eror saat membuka aplikasi teams"))
print(senti.main("bagus"))
print(senti.main("mantap"))
print(senti.main("tidak bisa masuk rapat"))
print(senti.main("sangat membantu dalam kegiatan belajar mengajar semua fitur di aplikasi ini lebih lengkap dibanding beberapa aplikasi belajar daring lainnya"))
print(senti.main("bintang saja yang jawab"))
print(senti.main("bodoh akun hilang terus"))
print(senti.main("mantap"))
print(senti.main("tersendat mulu"))
print(senti.main("kendala tiap kali masuk dengan tautan ribet"))
print(senti.main("bagus"))
print(senti.main("mantap"))
print(senti.main("ini sangat rumit berkali kali unduh dan tidak bisa terhubung padahal selama ini tidak seperti itu selalu ketinggalan rapat karena susah sekali untuk diakses"))
print(senti.main("bismillah kalau bisa perbarui daripada aplikasi di perbaiki kembali di karena sebelumnya saya mengunduh aplikasi nya dan menggunakan aplikasi nya ppt ketika dosen jelaskan materi pembahasan pada perkuliahan sering kali slide gambar untuk ppt nya tidak muncul timbul dan hari ini saya saya hapus aplikasi nya serta mengunduh kembali masuk saya allah semoga besok kuliah akhir pekan nya tidak ada kendala di permudah serta di perlancar amin allahuma amin"))
print(senti.main("mantap juragan"))
print(senti.main("oke untuk rapat dimana pun berada mantap"))
print(senti.main("sering gagal masuk"))
print(senti.main("tiap mau masuk teams harus lepas dan mengunduh lagi buruk banget"))
print(senti.main("bagus sekali untuk pertemuan grup"))
print(senti.main("ribet memakai nya"))
print(senti.main("bagus"))
print(senti.main("eror"))
print(senti.main("bagus"))
print(senti.main("coba dulu kalau bagus aku kasih bintang"))
print(senti.main("fitur aplikasi lengkap sangat memuaskan"))
print(senti.main("aplikasi teams ini memudahkan saya dalam belajar daring"))
print(senti.main("bagus"))
print(senti.main("jelas dan terang serta tidak boros baterai"))
print(senti.main("oke"))
print(senti.main("kenapa tidak bisa diunduh lagi"))
print(senti.main("tidak bisa bergabung teams dan ketika masuk juga tidak bisa"))
print(senti.main("tidak bisa mengikuti rapat dan kalau sudah masuk bisa keluar sendiri"))
print(senti.main("jelek banget lagi ikut rapat audio mati terus padahal sudah pilih yang speaker cuma bertahan beberapa detik saja terus audio mati terus terus kalau saat mau gabung rapat malah ada pilihan tambahkan perangkat ini tapi tanpa audio atau transfer ke perangkat ini tapi mencoba satu tetap saja audio nya selalu hilang terus dalam beberapa detik menghambat banget jadi ketinggalan rapat tidak tahu orang mengomong apa"))
print(senti.main("sering eror"))
print(senti.main("dosen"))
print(senti.main("bingung"))
print(senti.main("aplikasi ini membantu saya dalam belajar daring"))
print(senti.main("mantap mantap betul"))
print(senti.main("memek tugas ku tidak ke kirim ke refresh terus masa setiap mau kumpul tugas harus lepas dulu baru masuk lagi"))
print(senti.main("keren"))
print(senti.main("kerja bagus"))
print(senti.main("sering eror"))
print(senti.main("sangat bermanfaat"))
print(senti.main("oke"))
print(senti.main("sangat memudahkan aktivitas keren"))
print(senti.main("baik untuk belajar anak"))
print(senti.main("saya malas"))
print(senti.main("tidak bisa log masuk suruh reset pas sudah reset pas ketika mau bergabung pertemuan tetap tidak bisa malah log keluar sendiri terus suruh reset pas lagi begitu saja terus sampai pertemuan kantor selesai dan saya tidak ikut pertemuan payah"))
print(senti.main("sering banget eror harus di lepas dulu terus mengunduh lagi tiap mau pertemuan bulanan padahal sudah di perbarui"))
print(senti.main("sampah"))
print(senti.main("lumayan bisa untuk mengerjakan tugas tapi sering eror kadang teams nya kenapa ya terus kalau ada tugas tidak ada notifikasi tidak ada pemberitahuan padahal suara notifikasi nya ya aktif lho dan ya teams saya sering ke log keluar sendiri tiba² tolong di perbaiki ini sering membuat saya terlambat mengerjakan kalau ada tugas dan sering tidak nyaman saat saya menggunakan aplikasi teams"))
print(senti.main("tolong diberi fitur hapus masa harus hapus lewat komputer sih"))
print(senti.main("sangat membantu dalam hal pekerjaan"))
print(senti.main("teams ini juga sangat berguna sekali untuk para pelajar ketika sedang daring dan mengumpulkan tugas tugas sekolahnya dan dari saya semoga teams kedepannya bisa lebih baik lagi dari yang sekarang ini"))
print(senti.main("aplikasi pengguna ramah"))
print(senti.main("bagus"))
print(senti.main("sangat baik mudah untuk membuat diskusi maupun pertemuan"))
print(senti.main("siap sekali"))
print(senti.main("bagus untuk sekolah kerja sangat bagus"))
print(senti.main("oke"))
print(senti.main("oke"))
print(senti.main("mantap"))
print(senti.main("perbarui terus sudah diperbarui tidak bisa di pakai eror terus ah tidak bisa di pakai"))
print(senti.main("oke"))
print(senti.main("setiap mau dipakai mendadak masuk mulu kontol"))
print(senti.main("jadi susah masuk ruangan"))
print(senti.main("luas aplikasi untuk kantor pertemuan diskusi"))
print(senti.main("kenapa tugas yang ada di teams tidak bisa dibuka malah mutar terus"))
print(senti.main("sangat pintar aplikasi ya"))
print(senti.main("sangat membantu di dalam proses pembelajaran daring terintegrasi dengan beberapa platform belajar yang mendukung"))
print(senti.main("bagus banget"))
print(senti.main("oke"))
print(senti.main("keren btw apakah ada fitur mencari histori chat"))
print(senti.main("suka log keluar sendiri aplikasi tidak jelas perbaiki dulu eror nya"))
print(senti.main("lumayan"))
print(senti.main("aplikasi rusak"))
print(senti.main("aplikasi yang sangat bagus"))
print(senti.main("sangat bagus"))
print(senti.main("bagus"))
print(senti.main("aplikasi ini sangat membantu"))
print(senti.main("biasa saja"))
print(senti.main("mantap tolong jangan eror lagi"))
print(senti.main("masih sering eror tingkatkan lagi ya terima kasih"))
print(senti.main("sangat menyusahkan sering eror berkali kali tidak bisa bergabung tiap kali masuk lewat tautan suruh masuk padahal sudah masuk coba masuk lagi email gagal"))
print(senti.main("setiap kali saya kau masuk mengikuti rapat selalu tidak bisa harus di lepas dulu terus di pasang kembali kurang puas dengan aplikasinya kalau begitu kan boros kuota mengunduh ulang terus terus an"))
print(senti.main("aplikasi tiba tiba eror setiap mau masuk rapat pasti beralih dan masuk lagi ribet"))
print(senti.main("menyebalkan kenapa eror terus"))
print(senti.main("kenapa selalu begini sih eror terus mau masuk tautan tidak bisa terus kuliah jadi telat terus"))
print(senti.main("sulit untuk masuk"))
print(senti.main("bagus"))
print(senti.main("oke"))
print(senti.main("aplikasi bagus tapi sering eror membuat pusing saat kuliah daring terus meminta masukkan sandi padahal pertama pakai aplikasinya bagus kenapa ke depannya eh malah sering eror"))
print(senti.main("oke"))
print(senti.main("bagaimana ini kenapa sering keluar sendiri daftar ulang akun lagi tidak bisa langsung tersambung ke tautan padahal sudah di daftarkan"))
print(senti.main("tidak bisa masuk padahal dulu tidak begini amat nih teams aplikasi terburuk"))
print(senti.main("oke"))
print(senti.main("suka eror jaringan bagus tapi saat masuk teams jadi hilang jaringan suka keluar sendiri mikrofon tidak bisa nyala kadang tombol mikrofon sama kamera nya tidak ada parah sih"))
print(senti.main("eror tidak bisa bergabung panggilan lagi tidak bisa ganti akun untuk akun baru tidak bisa masuk"))
print(senti.main("log masuk sulit"))
print(senti.main("semenjak perbarui sering keluar sendiri habis itu tidak bisa gabung lewat pengiriman tautan selalu di meminta email atau nomor ponsel saat dimasuki katanya salah tolong lah di perbaiki"))
print(senti.main("susah sekali mengunduh aplikasinya"))
print(senti.main("susah masuk"))
print(senti.main("kok tidak bisa di mengunduh sudah ke kalinya tidak bisa juga tolong dong di memperbaiki dulu aplikasinya"))
print(senti.main("tolong kak ini aplikasinya akhir akhir ini banyak sekali menggunakan kuota mohon di perbaiki seperti dulu"))
print(senti.main("setiap mau masuk tautan kuliah selalu di alihkan sudah memasukkan akun tapi tidak bisa katanya akun salah lah nomor salahlah aduh bikin pusing akhirnya tidak bisa kuliah deh"))
print(senti.main("menyenangkan"))
print(senti.main("sering eror tidak bisa bergabung disuruh daftar terus padahal sudah daftar"))
print(senti.main("aplikasi buat kuliah tapi kadang eror tidak bisa gabung ruangan kalau lewat tautan"))
print(senti.main("sangat tidak memuaskan terpaksa saya memakai ini kalau ada aplikasi selain ini tidak mau saya memakai saya benci aplikasi ini saya benci aplikasi ini karena sangat tidak membantu karena sangat tidak membantu pergi kau dari sini"))
print(senti.main("penggunaan sangat mudah diakses"))
print(senti.main("selalu bermasalah"))
print(senti.main("sulit digunakan untuk grup"))
print(senti.main("untuk di ponsel audio tidak bisa menyambung tolong diperbaiki jangan jadi aplikasi sampah orang murahan terima kasih"))
print(senti.main("aplikasi nya suka eror tidak mau masuk bahkan berjam jam tidak mau masuk padahal jaringan nya bagus"))
print(senti.main("oke"))
print(senti.main("oke"))
print(senti.main("pemerintah tolong lah perbarui penuh ini aplikasi"))
print(senti.main("bagus"))
print(senti.main("mantap banget"))
print(senti.main("tidak bisa dibuka lewat ponsel"))
print(senti.main("bagus"))
print(senti.main("kenapa tidak bisa buka punya aku"))
print(senti.main("kalau deh masuk mau dipakai lagi harus beralih ke akun lain jangka pendek banget kalau masuk tidak bisa masuk harus hapus dan mengunduh ulang pihak aplikasi boleh tolong diperbarui lagi"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("tidak bisa mengunduh"))
print(senti.main("sangat membantu untuk berbagi informasi dan pertemuan daring"))
print(senti.main("oke"))
print(senti.main("lumayan bagus"))
print(senti.main("bagus"))
print(senti.main("sering eror jelek beban pemula"))
print(senti.main("baik sekali"))
print(senti.main("oke mantap"))
print(senti.main("bagus"))
print(senti.main("aplikasi yang sangat buruk masa sudah di mengunduh kali tetap tidak bisa"))
print(senti.main("kalian bisa tidak sih membuat aplikasi yang bagus ini aplikasi jelek sekali banyak eror"))
print(senti.main("aplikasi payah tidak bermutu padahal suka di pakai tapi tiap ada pertemuan tidak bisa bergabung terus perbaiki dong"))
print(senti.main("sangat buruk"))
print(senti.main("memuat lama pkyuu"))
print(senti.main("karena saya lupa kata sandi ya bagaimana nih"))
print(senti.main("bagus"))
print(senti.main("mantap"))
print(senti.main("masuk terus tidak bisa masuk"))
print(senti.main("iya"))
print(senti.main("buruk sekali membuat pekerjaan saya terhambat tidak bisa masuk giliran sudah masuk tidak ada gambar nya parah tolong di perbaiki"))
print(senti.main("tidak bisa di perbarui"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("kok tidak bisa diunduh malah eror bagaimana ini"))
print(senti.main("sering eror tidak bisa bergabung harus di undang dulu baru bisa bergabung aduh parah"))
print(senti.main("eror terus admin tugasnya tidak mau berhenti"))
print(senti.main("fitur untuk membagikan kuat point tidak ada"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("kebiasaannya eror"))
print(senti.main("sederhana namun sangat membantu pekerjaan"))
print(senti.main("baik"))
print(senti.main("sangat mantap"))
print(senti.main("bermasalah mulu kesal deh"))
print(senti.main("kadang butuh sinyal yang kuat banget jika ditautkan dengan sama form"))
print(senti.main("tidak guna lelet banget"))
print(senti.main("aplikasi nya sangat bagus"))
print(senti.main("mantap dan keren"))
print(senti.main("sangat bagus"))
print(senti.main("mantap"))
print(senti.main("banyak eror lagi uas tidak bisa menyetorkan gagal ujian ini terjadi berulang kali kali gagal ujian gara tidak bisa menyetorkan hasil ujian di masih teams"))
print(senti.main("membantu untuk belajar"))
print(senti.main("bagus"))
print(senti.main("mantap"))
print(senti.main("mantap"))
print(senti.main("oke"))
print(senti.main("tidak jelas"))
print(senti.main("bagus"))
print(senti.main("kadang suka eror menyebalkan"))
print(senti.main("top"))
print(senti.main("buruk"))
print(senti.main("lebih aman dan mudah mengoperasikan tugas dan sebagainya"))
print(senti.main("tiap mau pertemuan harus perbarui terus"))
print(senti.main("sering eror kadang sampai mulai ulang komputer berkali kali"))
print(senti.main("mantap"))
print(senti.main("oke lah"))
print(senti.main("bisa chat sama teman teman tim"))
print(senti.main("bagus"))
print(senti.main("aplikasi suka eror tolong benarkan lagi sistem"))
print(senti.main("bagus lancar"))
print(senti.main("saat saya masuk ke pertemuan nya malah sendiri tidak ada orang sering begitu setiap hari saat saya mau pertemuan"))
print(senti.main("bagus untuk kuliah daring"))
print(senti.main("tidak ada"))
print(senti.main("aku duka"))
print(senti.main("bagus"))
print(senti.main("oke"))
print(senti.main("luar biasa dan pengalaman terus bertambah"))
print(senti.main("aplikasi ini bagus untuk pembelajaran jarak jauh"))
print(senti.main("bagus"))
print(senti.main("bagus sekali"))
print(senti.main("ribet tidak bisa masuk"))
print(senti.main("aplikasinya cocok untuk sekolah daring dan kualitas nya sangat bagus sekali"))
print(senti.main("bagus sekali aplikasi nya"))
print(senti.main("bagus banget cuma kalau kumpulkan tugas seperti video begitu kalau panjang tidak bisa coba di pertimbangkan lagi ya terima kasih"))
print(senti.main("oke"))
print(senti.main("sampah"))
print(senti.main("bagus"))
print(senti.main("terkadang sinyalnya yang kurang baik"))
print(senti.main("mendukung sekali"))
print(senti.main("sering eror"))
print(senti.main("bagus sekali"))
print(senti.main("kenapa tidak bisa mengunduh aplikasi teams"))
print(senti.main("harus bolak balik di pasang ulang kalau tidak nanti tidak bisa ikut rapat"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("sangat bermanfaat tapi jangan meminta perbarui terus dong"))
print(senti.main("kenapa mau bergabung kakak rapat mentok di memuat doang padahal sinyal bagus"))
print(senti.main("mantap"))
print(senti.main("sering eror"))
print(senti.main("gambar bagus"))
print(senti.main("susah"))
print(senti.main("bagus untuk teams"))
print(senti.main("aplikasi tidak bisa digunakan adakah versi terbaru nya terima kasih"))
print(senti.main("tolong saya mau mengerjakan ujian dengan tenang di kertas bukannya dari ini sudah puasa berpikir keras mudahkan lah ujian besok iya allah yang baca doakan juga ya bang kak terima kasih"))
print(senti.main("top banget"))
print(senti.main("aplikasi ini lelet selalu meminta kata sandi"))
print(senti.main("suara saya tidak bisa terdengar peserta lain"))
print(senti.main("baik"))
print(senti.main("sangat membantu"))
print(senti.main("oke"))
print(senti.main("suka keluar sendiri mau log masuk ke rapat harus pasang lagi"))
print(senti.main("tidak bisa bergabung pertemuan"))
print(senti.main("senang belajar di teams"))
print(senti.main("sangat mudah dan mengesankan"))
print(senti.main("aplikasi sampah"))
print(senti.main("kamu jahat"))
print(senti.main("baik"))
print(senti.main("foto profil tidak bisa dihapus"))
print(senti.main("boros tapi bagus untung oke wifi"))
print(senti.main("lelet banget ya allah"))
print(senti.main("aplikasi gagal"))
print(senti.main("tidak bisa masuk"))
print(senti.main("jadi bisa tatap muka dengan dosen lewat aplikasi ini bagus"))
print(senti.main("kanal data"))
print(senti.main("semoga semakin maju aplikasi nya membantu"))
print(senti.main("ini kenapa tidak bisa di mengunduh sih"))
print(senti.main("oke"))
print(senti.main("baik"))
print(senti.main("tidak mengerti pakai ini sumpah"))
print(senti.main("mantap"))
print(senti.main("aplikasi tidak jelas"))
print(senti.main("tidak bisa masuk saat mengikuti rapat"))
print(senti.main("aplikasi bagus digunakan untuk mengajar"))
print(senti.main("aplikasi ini bagus untuk kuliah daring saya suka"))
print(senti.main("alhamdulillah baik meski kadang keluar masuk pada saat pertemuan"))
print(senti.main("mau masuk saja ribet banget"))
print(senti.main("semoga lebih baik lagi"))
print(senti.main("bagus untuk kuliah"))
print(senti.main("baik"))
print(senti.main("tidak bisa di gunakan di ponsel"))
print(senti.main("membantu"))
print(senti.main("terbaik"))
print(senti.main("mantap"))
print(senti.main("oke"))
print(senti.main("cukup oke lumayan bagus nih aplikasinya"))
print(senti.main("semangat bekerja"))
print(senti.main("bagus"))
print(senti.main("oke"))
print(senti.main("rusak padahal mau zoom sekolah"))
print(senti.main("oke"))
print(senti.main("sering eror sendiri tidak bisa masuk"))
print(senti.main("bagus"))
print(senti.main("eror terus di perbarui malah banyak eror mana memuat nya lama tolong di perbaiki"))
print(senti.main("memuaskan dan bisa memudahkan diskusi"))
print(senti.main("suka heran sama gambar layarnya tiba tiba kecil misal dosen saya share konten tapi gambar di ponsel saya kecil banget terletak di pojok biasanya uluran dari ukuran seharusnya dan gambar berhenti lain lagi ketika tidak ada tombol bergabung padahal pertemuan sudah di buka dosen jadul banget aplikasi kamu padahal tidak seberat permainan permainan saja kalau ada eror langsung di perbaiki aplikasi kamu ada eror untuk emosi tidak karuan lamanya tidak ada selesai selesainya atau kamu tidak tahu apa yang dikerjakan bawahan kamu jadi tidak ada arahan"))
print(senti.main("oke banget"))
print(senti.main("memuat tugasnya lama padahal sinyal bagus"))
print(senti.main("kenapa aplikasinya tidak bisa diunduh sih"))
print(senti.main("mantap"))
print(senti.main("susah bergabungnya akh aplikasinya payah"))
print(senti.main("memudahkan dalam bekerja bisa di akses di ponsel"))
print(senti.main("mudah menggunakan membantu dalam pekerjaan saya"))
print(senti.main("membantu sekali"))
print(senti.main("memudahkan pekerjaan kantor bisa akses dimana saja"))
print(senti.main("erornya masyallah keluar sendiri terus"))
print(senti.main("lelet banget mengirim kode ke sms tidak mengirim"))
print(senti.main("microsoft ini aplikasi sudah bagus tapi kenapa tidak terkirim pesannya tolong ya microsoft agar pesannya terkirim cepat di perbaiki"))
print(senti.main("kenapa sih teams sini selalu meminta kata sandi tiap dibuka"))
print(senti.main("suka keluar masuk padahal jaringan bagus"))
print(senti.main("mengecewakan banget setiap zoom tidak bisa langsung masuk"))
print(senti.main("eror mulu bangsat jelek"))
print(senti.main("aplikasi tidak berguna membuat susah"))
print(senti.main("mantap sangat membantu kalau bisa dilengkapi fitur nya seperti masih teams untuk windows agak kesulitan untuk unggah file materi kelas"))
print(senti.main("baik sangat membantu untuk mahasiswa belajar tuweb"))
print(senti.main("oke"))
print(senti.main("unduksi"))
print(senti.main("bagus sekali"))
print(senti.main("sarana penunjang kegiatan belajar tapi sering eror untuk mengunggah file tidak bisa"))
print(senti.main("aplikasi rusak jelek banget baru mengunduh saja tidak bisa ke buka"))
print(senti.main("bagus aplikasinya"))
print(senti.main("keren dan"))
print(senti.main("sangat membantu bisnis"))
print(senti.main("aplikasinya bagus"))
print(senti.main("oke"))
print(senti.main("langsung kena lcd nya saat jatuh aplikasi ini kenapa iya"))
print(senti.main("baru saja pasang supaya tidak di masa seperti  ojek daring"))
print(senti.main("bingung"))
print(senti.main("dengan adanya masih teams belajar jarak jauh menjadi lebih produktif"))
print(senti.main("membantu sekali"))
print(senti.main("bagus tapi lagi di pertemuan malah mikrofon nya bermasalah terus membuat kesal pula lebih baik saya kasih bintang dulu"))
print(senti.main("sangat memudahkan proses belajar mengajar"))
print(senti.main("sangat bermanfaat bagi teknologi zaman sekarang"))
print(senti.main("bermanfaat dan mantap"))
print(senti.main("teams bagus untuk pertemuan jarak jauh"))
print(senti.main("kok tidak bisa masuk ya sandi nya salah terus padahal sudah sesuai bagaimana dong aplikasi nya padahal saya butuh banget untuk kuliah"))
print(senti.main("top"))
print(senti.main("oke"))
print(senti.main("aplikasinya eror masak tugasnya masuk terus"))
print(senti.main("oke untuk zoom"))
print(senti.main("saya tidak bisa pasang microsoft teams"))
print(senti.main("kenapa ya bermasalah terus ini kan untuk kuliah bagaimana mau kuliah nya fokus kalau mau masuk kelas saja susah banget"))
print(senti.main("aplikasi sering eror beberapa kali dibuka tetap tidak bisa ini aplikasi sebenarnya bisa dipergunakan tidak sih sementara mau di pakai untuk rapat harap diperbaiki kembali"))
print(senti.main("sangat membantu aplikasi nya"))
print(senti.main("jelek"))
print(senti.main("suka eror sendiri"))
print(senti.main("bagus"))
print(senti.main("saya kasih bintang saja karena setiap saya masuk ke microsoft teams selalu saja eror tolong lah di perbaiki lagi perangkat"))
print(senti.main("lambat masuk"))
print(senti.main("ketika akan bergabung rapat melalui tautan tautan selalu gagal masuk berkali² dan selalu mengulang² masuk akun microsoft nya padahal saya sudah memasukkan gmail dan verifikasi pun sudah benar tapi tetap saja tidak bisa masuk dan terus berulang disuruh masuk akun lagi dan gagal lagi"))
print(senti.main("tidak ada pokoknya aplikasi ini jelek banget"))
print(senti.main("teams sangat bagus"))
print(senti.main("sudah hampir dua tahun memakai microsoft teams alhamdulillah tidak ada kendala yang serius palingan cuma masalah eror sedikit tapi itu oke lah tidak apa apa"))
print(senti.main("bagus"))
print(senti.main("keren"))
print(senti.main("oke"))
print(senti.main("kalau di ponsel suruh bergabung manual terus lelah kalau ada notifikasi tidak tahu"))
print(senti.main("deh boros banyak masalah menyusahkan saja"))
print(senti.main("aplikasi ter ribet anjing"))
print(senti.main("oljgle ke oke uokiok oik oly oif jol looliooveloky ulama o ia lkom"))
print(senti.main("kenapa kode yang di nomor saya tidak terkirim terus ya"))
print(senti.main("sungguh menyenangkan"))
print(senti.main("oke"))
print(senti.main("mantap"))
print(senti.main("kenapa sekarang perangkat ini agak ribet"))
print(senti.main("bagus"))
print(senti.main("kontak"))
print(senti.main("oke"))
print(senti.main("padahal sudah disingkronkan akun sekolah sandinya disimpan di google tiap tahun aku memakai tidak ada kendala tiba² tidak bisa masuk katanya salah sandi kecewa banget"))
print(senti.main("aplikasi tidak bisa masuk lewat ponsel iya kali harus buka laptop mulu"))
print(senti.main("aplikasi ini sangat membantu namun untuk memasang ulang kok tidak bisa ya teman teman mohon bantuannya kuota masih banyak"))
print(senti.main("bagus"))
print(senti.main("aneh suka ke keluar sendiri tiap mau masuk kenapa keterangan sandi salah padahal tidak pernah ubah² sandi"))
print(senti.main("bagus"))
print(senti.main("mau masuk setelah lama tidak digunakan sulit harus perbarui dulu"))
print(senti.main("oke"))
print(senti.main("sering banget eror sampai untuk akun tetap saja tidak bisa masuk mana kuliah pakai ini lagi"))
print(senti.main("membantu"))
print(senti.main("sumpah jelek banget"))
print(senti.main("mikrofon tidak bisa sering keluar sendiri"))
print(senti.main("buruk sekali"))
print(senti.main("perdana saya gunakan untuk kuliah ut melalui tuweb malah eror sedih tidak bisa buka tautan dan aplikasinya kecewa malah cuma aplikasi ini yang sudah ditentukan"))
print(senti.main("biasa bagus banget"))
print(senti.main("sangat baik"))
print(senti.main("jelek eror mesti nge masukkan kata sandi padahal kata sandi e benar tetap salah"))
print(senti.main("keren tapi harus sinyal kuat"))
print(senti.main("aplikasi sering eror"))
print(senti.main("bagus"))
print(senti.main("tidak bisa masuk forum rapat terus"))
print(senti.main("sekarang buka kenapa harus perbaruan kata sandi tidak seperti dulu"))
print(senti.main("oke"))
print(senti.main("halo bu maaf"))
print(senti.main("kebanyakan perbarui bukannya bagus malah semakin tolol"))
print(senti.main("diklat microsoft teams sangat bermanfaat"))
print(senti.main("aplikasi bagus yang sangat berguna"))
print(senti.main("kenapa kalau di perbarui ulang fitur nya hilang semua untuk mengerjakan tugas tidak bisa"))
print(senti.main("aplikasi nya jelek banget ಠಗಠ"))
print(senti.main("banyak eror"))
print(senti.main("mantap"))
print(senti.main("bagus"))
print(senti.main("saya suka sekali dengan aplikasi teams ini"))
print(senti.main("aku sekolah pakai aplikasi ini"))
print(senti.main("bagus"))
print(senti.main("aplikasi jelek"))
print(senti.main("sangat membantu"))
print(senti.main("cukup baik"))
print(senti.main("tidak bisa mengunduh"))
print(senti.main("aplikasi jelek sudah untuk masuk"))
print(senti.main("kok setelah diperbarui mau buka tugas kok eror terus ya bagian tugasnya hilang woy tolonglah dibenarkan itu mau untuk tugas tidak bisa bisa sudah ditagih tagih ini sama guru tolong tolonglah cepat dibenarkan"))
print(senti.main("ini kenapa ya dari kemarin mau verifikasi nomor kok tidak dapat kodenya"))
print(senti.main("jelek menyusahkan"))
print(senti.main("memusingkan"))
print(senti.main("sangat bagus"))
print(senti.main("sok jauh sok bagus"))
print(senti.main("lengkap bisa untuk chat dan lain lain"))
print(senti.main("tahi lah masukkan kata sandi terus mutar terus verifikasi terus begitu terus"))
print(senti.main("dengan aplikasi ini memudahkan kita rapat koordinasi"))
print(senti.main("aplikasinya bagus sudah lengkap kelas temu dan pengumpulan tugas dalam aplikasi"))
print(senti.main("mantap"))
print(senti.main("oke siap"))
print(senti.main("bisa belajar"))
print(senti.main("banyak kendala nya masuk susah masuk"))
print(senti.main("apa terjadi kenapa adalah disana nomor background berubah fitur masuk android sekarang"))
print(senti.main("kenapa saya tidak bisa mengunduh microsoft teams ini"))
print(senti.main("bagus aplikasi"))
print(senti.main("kadang tiba tiba eror tidak ada temu"))
print(senti.main("kenapa saya tidak dapat kode lewat sms"))
print(senti.main("sangat membantu dalam pertemuan kerja"))
print(senti.main("baik"))
print(senti.main("sering gangguan sinyal"))
print(senti.main("aplikasi yang mumpuni untuk pekerjaan"))
print(senti.main("aneh memuat lama tidak jelas mulu padahal jaringan bagus"))
print(senti.main("tolong perbaiki background untuk rapat tidak bisa digunakan"))
print(senti.main("kebanyakan tugas anjing"))
print(senti.main("mantap"))
print(senti.main("tampilan cukup bagus"))
print(senti.main("bermanfaat"))
print(senti.main("makan banyak memori suara tidak jelas padahal sinyal tidak"))
print(senti.main("tidak buruk"))
print(senti.main("mantap"))
print(senti.main("tidak bisa masuk"))
print(senti.main("siap deh"))
print(senti.main("erornya tidak bisa bergabung teams ke log keluar sendiri tidak bisa menyelesaikan panggilan chat tidak bisa dikirim sering ke batal sunyikan sendiri akun sering eror mikrofon tidak bisa dibuka kamera hitam padahal sinyal bagus submit tugas kadang ada yang tidak ke kirim padahal sudah dikirim pencarian kontak kadang tidak menemukan padahal sudah lengkap namanya sering tersendat keluar sendiri padahal sudah sinyalnya bagus tautan kadang eror tidak bisa masuk tugas ada yang tidak kelihatan tolong diperbaiki kalau begitu microsoft"))
print(senti.main("mulai tidak jelas ini eror mulu"))
print(senti.main("tidak bisa bergabung video sekarang"))
print(senti.main("terlalu banyak tugas"))
print(senti.main("saya harap aplikasi ini tidak memakan banyak ruang penyimpanan pada ponsel pengguna dan juga saya sangat mengharapkan kedepannya aplikasi ini bisa untuk dipindahkan ke kartu sd dan tidak memakan banyak ruang penyimpanan internal pada ponsel pengguna dan satu lagi terus diberikan dukungan dan perbarui untuk versi android marshmellow"))
print(senti.main("tidak bisa dibuka saat di tugas padahal kalau dari galeri bisa"))
print(senti.main("oke"))
print(senti.main("kok saya tidak bisa mengunduh padahal penyimpanannya bisa cukup loh"))
print(senti.main("sangat membantu dalam rapat atau kelas karena semua media tulis dapat di akses serta dapat di rekam"))
print(senti.main("sangat bagus"))
print(senti.main("kurang menguasai"))
print(senti.main("bagus"))
print(senti.main("oke lancar"))
print(senti.main("pihak pengembang mohon kalau perbarui itu jangan ada tersendat atau bahkan eror di aplikasi ini ya soalnya saya sebagai pengguna merasa tidak nyaman untuk belajar bahkan sering kejadian notifikasi tidak masuk padahal notifikasi sudah dihidupkan semua kalau ada tugas mohon perhatiannya"))
print(senti.main("tolol"))
print(senti.main("oke"))
print(senti.main("jelek dek"))
print(senti.main("terima kasih"))
print(senti.main("bagus dan selalu membantu"))
print(senti.main("mantap"))
print(senti.main("bagus tapi kenapa saluran kadang tidak bisa dibuka malah mutar mutar terus saya tunggu lama tapi sama saja"))
print(senti.main("tidak bisa dibuka keluar sendiri"))
print(senti.main("aplikasinya sering eror memuatnya lama setiap masuk ke teams selalu suruh memasukkan kata sandi teams padahal dulu tidak seperti begini sudah begitu kalau mau masuk ruangan susah banget padahal internet sudah bagus tolong perbaiki"))
print(senti.main("bisa memudahkan di masa pandemi untuk rapat daring"))
print(senti.main("sangat membantu"))
print(senti.main("keren"))
print(senti.main("woy kok di hilangkan t tugas yang masuk"))
print(senti.main("ini kenapa sih tiap perbarui pasti ada eror atau apalah itu tidak bisa lihat tugasnya dan memuat selalu lama padahal jaringan bagus sudah pakai wifi atau biasa tetap saja begitu tolong benarkan"))
print(senti.main("susah bergabung"))
print(senti.main("eror"))
print(senti.main("bagus"))
print(senti.main("tidak bisa mengunduh"))
print(senti.main("oke"))
print(senti.main("suka eror"))
print(senti.main("bagus"))
print(senti.main("sangat membantu dalam pertemuan daring dan untuk koordinasi pekerjaan karena tidak sempat bertemu luring juga membantu karena rapat di teams dapat menggantikan tatap muka"))
print(senti.main("parah dan super eror nya"))
print(senti.main("masih sering terjadi eror"))
print(senti.main("mantap"))
print(senti.main("oke"))
print(senti.main("sangat bermanfaat"))
print(senti.main("bagus tapi terkadang suka eror"))
print(senti.main("aplikasi sangat membantu dalam proses pembelajaran daring tapi saya tidak suka karena ukuran aplikasi nya terlalu besar untuk ponsel saya yang cuma ram gb"))
print(senti.main("sangat baik"))
print(senti.main("jelek karena banyak tugas"))
print(senti.main("untuk pertemuan keluar terus dibuka lewat browser tidak mau suruh mengunduh aplikasinya saat di pakai sama mengampasnya"))
print(senti.main("bagus dan membantu"))
print(senti.main("oke"))
print(senti.main("aplikasi sangat membantu"))
print(senti.main("jelek"))
print(senti.main("keren"))
print(senti.main("aplikasi ini sangat bagus dan berfungsi dengan baik saya harap kedepannya aplikasi ini bisa menjadi lebih baik dari yang sekarang"))
print(senti.main("susah masuk bosan"))
print(senti.main("banyak terima kasih"))
print(senti.main("pemasaran"))
print(senti.main("tidak stabil"))
print(senti.main("aku itu mau kirim tugas di tugas tapi tulisannya terjadi kesalahan mulu malas banget"))
print(senti.main("masih suka tersendat"))
print(senti.main("super bagus"))
print(senti.main("saya sudah perbarui dan mencoba pasang kembali tetapi tetap tidak ada menu untuk ganti background"))
print(senti.main("bagus"))
print(senti.main("tidak cocok untuk pertemuan banyak kendala lebih sederhana zoom dan google temu"))
print(senti.main("sangat lelet"))
print(senti.main("perdana"))
print(senti.main("mantap dan cocok untuk belajar mandiri"))
print(senti.main("peserta karantina ihg dari spie minyak"))
print(senti.main("keren habis"))
print(senti.main("oke"))
print(senti.main("aplikasi ini tidak bagus malas banget sekolah benci"))
print(senti.main("visual baik"))
print(senti.main("membingungkan untuk masuk sekali itu ditampilkan berhasil tapi membawa saya kembali untuk masuk halaman"))
print(senti.main("saya sangat terganggu dengan ini aplikasi terkadang i tidak bisa mendapatkan menuju pertemuan dan notifikasi tidak muncul atas kemudian the kamera kualitas adalah tidak bagus dan berubah the profil foto mengambil a banyak dari waktu"))
print(senti.main("oke"))
print(senti.main("aplikasi ini bagus banget saya paling suka saat muncul stiker saat sudah mengumpulkan lucu untuk kalian yang bilang aplikasi ini jelek yakin kalau tidak ada aplikasi ini kalian tidak bisa belajar di whatsapp sih bisa tapi kan ya begitu"))
print(senti.main("sangat bagus aplikasi bisa mengumpulkan tugas masuk semua form mulai dari email dokumen dan yang lain i memberi bintang untuk ini aplikasi"))
print(senti.main("bagus"))
print(senti.main("teams sangat jelek kita akan masuk gagal saat kita lepas teams masuk lewat google chrome harus mengunduh kan aplikasi menyebalkan"))
print(senti.main("mantap"))
print(senti.main("bagus"))
print(senti.main("sempurna"))
print(senti.main("mantap"))
print(senti.main("tidak tahu"))
print(senti.main("mantap"))
print(senti.main("aplikasinya bagus dan tidak ada gangguan sama sekali"))
print(senti.main("sering tersendat kosong dan tidak bisa mengunggah file dan pesan padahal wifi sangat bagus sangat kecewa"))
print(senti.main("bagus"))
print(senti.main("oke bagus"))
print(senti.main("tidak bisa dipasang"))
print(senti.main("baru coba nanti kalau bagus bintangnya di tambah"))
print(senti.main("bagus"))
print(senti.main("karena guru saya sering memberi tugas saya kasih bintang saja"))
print(senti.main("bagus"))
print(senti.main("terima kasih sangat membantu bagus banget"))
print(senti.main("bagus aplikasi"))
print(senti.main("bagus bisa membuat grup seperti di whatsapp bisa chat pokok nya jelek banget"))
print(senti.main("bagus"))
print(senti.main("wow"))
print(senti.main("masuk eror terus meminta verifikasi sudah di verifikasi kembali ke awal lagi suruh masuk ulang begitu saja terus"))
print(senti.main("keren sih"))
print(senti.main("aplikasi jelek untuk ponsel tersendat tersendat jangan di mengunduh"))
print(senti.main("bagus"))
print(senti.main("sangat bagus dan mantap jiwa"))
print(senti.main("sering tidak bisa bergabung tautan pertemuan harus beralih akun"))
print(senti.main("oke"))
print(senti.main("harus memasukkan sandi terus"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("sangat bermanfaat"))
print(senti.main("sangat bermanfaat dimasa pandemi ini"))
print(senti.main("susah sekali mau masuk saja berbelit"))
print(senti.main("terima kasih teams sudah merespon cepat dm instagram saya"))
print(senti.main("aku suka banget sama teams"))
print(senti.main("tidak tahu ah gelap cuma untuk belajar doang juga"))
print(senti.main("sangat tidak ramah"))
print(senti.main("sering sekali jika ada pertemuan susah masuk ke tautan sampai berkali² pernah sampai minta baru bisa masuk mohon agar di permudah terima kasih"))
print(senti.main("wow ini bagus banget karena aku bisa daring untuk sekolah wel agak eror sih tapi main bisa yey"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("bodoh"))
print(senti.main("membosankan"))
print(senti.main("terima kasih teams"))
print(senti.main("kenapa harus pakai microsoft authenticator"))
print(senti.main("tidak bisa"))
print(senti.main("semakin oke"))
print(senti.main("bagus"))
print(senti.main("baik"))
print(senti.main("bagus aplikasi"))
print(senti.main("aplikasi pertemuan paling ribet yang pernah ada tersendat mulu nyala kamera buram berat ini aplikasi untuk orang tolol saya menyarankan kalau mau pertemuan daring pakai zoom kalau pakai ini ribet parah"))
print(senti.main("diperbarui malah tambah aneh tombol untuk gabung tidak ada padahal rapatnya sudah dimulai iya sudah deh saya tidak bisa ikut rapat"))
print(senti.main("mengerikan aplikasi itu suka untuk menjadi eror untuk nomor jelas alasan ku koneksi adalah stabil tapi i gagal untuk bergabung a pertemuan ruangan untuk beberapa kali ini pagi dan itu dipersingkat ku praktikum durasi waktu"))
print(senti.main("bagus pertemuan software"))
print(senti.main("jelek"))
print(senti.main("tentu i seperti ini aplikasi karena i belajar masuk disini"))
print(senti.main("baik saat digunakan untuk rapat"))
print(senti.main("tolong dibenarkan dong masa harus memasukkan kata sandi lagi"))
print(senti.main("bintang"))
print(senti.main("bagus banget untuk media belajar asalkan sinyal nya juga bagus tapi kalau sinyal nya jelek juga tidak memuaskan"))
print(senti.main("sangat cocok untuk belajar"))
print(senti.main("semakin kesini semakin kurang performanya"))
print(senti.main("aplikasi nya bagus banget soal nya saat bergabung bisa langsung bergabung ke kelas nya keren pokok nya"))
print(senti.main("eror"))
print(senti.main("bagus bisa jadi seperti web whatsapp"))
print(senti.main("padahal ada pertemuan di channel tapi di teams saya sendiri malah tidak ada padahal ada tulisan pertemuan dimulai tapi tidak bisa bergabung"))
print(senti.main("sangat membantu"))
print(senti.main("saat mau rapat eh aplikasinya eror terpaksa perbarui lagi"))
print(senti.main("mantap"))
print(senti.main("masuk saja susah banget dan suka harus masuk berkali kali"))
print(senti.main("penting"))
print(senti.main("membuat segala jadi mudah"))
print(senti.main("tolong tambahkan fitur background saat rapat untuk semua perangkat android karena selama ini hanya perangkat ios saja yang bisa mengatur merubah background"))
print(senti.main("saya suka aplikasi ini"))
print(senti.main("bermanfaat sekali"))
print(senti.main("susah banget login nya lah"))
print(senti.main("cukup baik"))
print(senti.main("saya tidak bisa log masuk harus memasukkan nomor ponsel tapi tidak dikasih juga kode verifikasi nya menghambat banget ya sudah email ke pihak masih teams tapi belum dibalas juga"))
print(senti.main("semakin perbarui aplikasinya bukan semakin bagus malah semakin jelek karena setiap masuk atau bergabung kelas nya malah menyasar ke tempat yang lain tidak seperti sebelum nya kalau bergabung langsung ke kelas nya langsung"))
print(senti.main("aplikasi ini bagus dan bisa di gunakan untuk segala hal"))
print(senti.main("buruk"))
print(senti.main("oke"))
print(senti.main("lelet banget harus butuh kuota banyak tolong diperbaiki lagi"))
print(senti.main("tidak bisa diandalkan"))
print(senti.main("oke"))
print(senti.main("sering keluar sendiri saat rapat dan terkadang tidak bisa bergabung temu padahal sudah ada temu"))
print(senti.main("sering terputus banyak kendala"))
print(senti.main("oke"))
print(senti.main("banyak tugas tidak jelas"))
print(senti.main("ramah pengguna"))
print(senti.main("tidak bisa ikut pertemuan padahal sudah ada tautan untuk masuk ke masih teams tapi ada bacaan beralih akun terus"))
print(senti.main("bagus"))
print(senti.main("bagus banget aplikasi yang menemani saya tiap hari sambil mengerjakan tugas yang banyak"))
print(senti.main("bagus aplikasi nya untuk pakai sekolah daring sangat membantu saya"))
print(senti.main("aplikasinya bagus banget jadi guru guru kami meminta kamu untuk belajar menggunakan aplikasi ini"))
print(senti.main("puas"))
print(senti.main("sebulan sekali eror tidak bisa digunakan dan harus lepas"))
print(senti.main("keluar terus"))
print(senti.main("notifikasi kadang tidak muncul"))
print(senti.main("aplikasi yang sangat jelek"))
print(senti.main("bagus tapi sering eror seperti tidak bisa mengirim pesan kok pembaruan nya jelek ya dulu sebelum perbarui bisa mengirim file setelah diperbarui kok malah berbagi tidak mendukung sia saya perbarui menghabiskan penyimpanan tapi jelek tidak ada yang berubah malah fitur nya berkurang ini bagaimana sih saya meminta fitur berbagi file dikembalikan atau bintang"))
print(senti.main("bagus"))
print(senti.main("mantap lah untuk pertemuan urusan kantor namun suka susah masuk sering berulang tidak jelas perbarui terbaru mantap stabil"))
print(senti.main("kadang aplikasi nya suka eror"))
print(senti.main("mantap pokoknya"))
print(senti.main("bagus"))
print(senti.main("saya sangat senang"))
print(senti.main("suka eror"))
print(senti.main("di ponsel saya tidak bisa terunduh kenapa"))
print(senti.main("teams bagus banget aku bisa belajar"))
print(senti.main("berguna untuk acara apapun di masa kini"))
print(senti.main("sangat bagus"))
print(senti.main("baik digunakan untuk kelas daring"))
print(senti.main("aplikasi tidak jelas soal nya cuma bisa sekali"))
print(senti.main("mikrofon teams bermasalah"))
print(senti.main("susah bergabung"))
print(senti.main("oke"))
print(senti.main("mantap"))
print(senti.main("bagus"))
print(senti.main("biasa saja"))
print(senti.main("tidak jelas"))
print(senti.main("keren"))
print(senti.main("wah murah banget"))
print(senti.main("pelajaran mudah di cerna cuma saya tidak bisa ketik jawaban klarifikasi"))
print(senti.main("sangat membantu"))
print(senti.main("bermanfaat untuk kerja"))
print(senti.main("selalu tidak suka perbarui terbarunya banyak nge eror memakai ini cuma terpaksa untuk sekolah"))
print(senti.main("sering eror"))
print(senti.main("kadang eror"))
print(senti.main("oke"))
print(senti.main("saya kasih kontol"))
print(senti.main("bagus"))
print(senti.main("kenapa ya tidak bisa memasang aplikasi ini dan kalau mau membatalkan juga tidak bisa yang ada play store nya eror"))
print(senti.main("oleh"))
print(senti.main("tidak ada notifikasi nya"))
print(senti.main("dari fitur nya sudah tidak tapi tolong nyalakan notifikasi secara langsung atau tidak usah mengatur² lagi bagi orang yang tidak tahu cara mengaturnya dia akan memberikan peringkat buruk kepada aplikasi ini"))
print(senti.main("belum tahu cara pakai nya"))
print(senti.main("lumayan"))
print(senti.main("bagus"))
print(senti.main("akhir akhir ini sering ada kendala yaitu rapat sudah mulai tapi simbol dan tombol masuknya tidak ada sudah saya refresh dan mulai ulang hasilnya tetap sama akhirnya saya tidak bisa rapat"))
print(senti.main("memakai mudah untuk komunikasi dengan teams"))
print(senti.main("mantap"))
print(senti.main("repot banget tiap buka aplikasi harus memasukkan email atau nomor ponsel sudah di memasukkan yang benar malah tidak bisa masuk jadi harus lepas lagi"))
print(senti.main("sangat baik dan bagus"))
print(senti.main("saya tidak bisa pasang aplikasi ini entah kenapa padahal di butuhkan ketika ada undangan pertemuan disini"))
print(senti.main("sangat bagus untuk saya"))
print(senti.main("terima kasih"))
print(senti.main("keluar sendiri terus anjing"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("oke"))
print(senti.main("aplikasi ini sangat baik bisa untuk belajar menjadi seru dan mengumpulkan tugas menjadi seru karena ada stiker muncul pada saat sudah mengumpulkan tugas itu lah kesenangan saya dengan aplikasi ini untuk sekolah saya tapi juga sering eror kadang saya harus buka dari play store karena tidak ada notifikasi"))
print(senti.main("susah di mengunduh dan memuatnya lama"))
print(senti.main("aplikasi aneh kenapa saya mengunduh berulang sampai kali aduh habis kuota saya"))
print(senti.main("bagus untuk bersekolah"))
print(senti.main("membantu"))
print(senti.main("bagus sekalian tapi saat zoom sebab eror atau eror"))
print(senti.main("mantap"))
print(senti.main("sangat bagus untuk untuk belajar"))
print(senti.main("sedekah bintang"))
print(senti.main("eror mulu dah tiba suruh masuk ulang tiba tidak bisa mengirim komentar di channel"))
print(senti.main("mantap"))
print(senti.main("oke"))
print(senti.main("susah banget log masuk ke masih teams masa salah terus padahal kata sandi sudah benar sudah reset ke email tetap saja tidak bisa malah sering nge tersendat sendiri susah kalau punya tugas terus rapatnya di masih teams"))
print(senti.main("memuat"))
print(senti.main("aplikasi ini benar berfungsi dan cepat saat di gunakan"))
print(senti.main("teams ini membuat saya jadi susah untuk tidur dengan tenang"))
print(senti.main("tidak bisa diandalkan sebelum konfirmasi untuk memenuhi undangan pertemuan tanyakan kepada pengundang apakah mungkin bisa menggunakan aplikasi lain misalnya zoom"))
print(senti.main("i belajar untuk kebaikan dengan semua pejabat microsoft teams"))
print(senti.main("aplikasi bodoh tidak bisa masuk pertemuan"))
print(senti.main("bagus"))
print(senti.main("kesalahan"))
print(senti.main("bagus"))
print(senti.main("semakin lama semakin tidak jelas masa setiap ada chat harus keluar masuk dulu gara gara ini jadi tidak dianggap hadir"))
print(senti.main("aplikasi sangat jelek"))
print(senti.main("kenapa saya mau mengunduh teams kok tidak bisa saya ingin mengunduh aplikasi teams ini tolong di benar kan ya"))
print(senti.main("kenapa kok tidak bisa unduh setelah dapat kiriman dokumen oleh pdf mohon dibantu"))
print(senti.main("kepala program permesinan"))
print(senti.main("kenapa mutar mutar terus"))
print(senti.main("aplikasi ini hebat"))
print(senti.main("sering tersendat"))
print(senti.main("woy masih teams tidak enak soalnya setiap ada tugas tidak ada notifikasi padahal deh aktif tolong pada saat mengirim foto tugas lama banget tolong perbaiki"))
print(senti.main("oke"))
print(senti.main("sangat bagus"))
print(senti.main("eror"))
print(senti.main("itu menyebalkan ini aplikasi sudah a banyak dari eror tolong perbaiki ini itu sungguh mengganggu kapan i mencoba untuk log masuk disana adalah a lama memuat screen kenapa di i punya meletakkan ku kata sandi setiap saat i membuka the aplikasi dan kapan i coba untuk log masuk untuk lain akun itu selalu terganggu saya salah satu the putih screen dan the memuat screen perbaiki ini"))
print(senti.main("log keluar sendiri"))
print(senti.main("suka eror"))
print(senti.main("mantap"))
print(senti.main("sangat membantu"))
print(senti.main("mantap"))
print(senti.main("aplikasi bagus sangat membantu dalam daring cuma kadang sering keluar sendiri semoga sistemnya lebih diperbaiki lagi"))
print(senti.main("sangat membantu untuk adik saya yang mengerjakan tugas daring"))
print(senti.main("mantap"))
print(senti.main("keluar sendiri saat akan presentasi"))
print(senti.main("mencoba"))
print(senti.main("bagus untuk kuliah cuma jaringan kurang bersahabat"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("lumayan lah abang"))
print(senti.main("sering meminta log masuk terus setelah di pakai rapat sudah coba lepas dan pasang lagi malah gagal tolong dong pengembangnya sistem nya di perbaiki lagi"))
print(senti.main("bagus"))
print(senti.main("sebagian rapat yang dimulai tidak terdeteksi"))
print(senti.main("bismillah kenapa saya tidak bisa masuk tolong di perbaiki"))
print(senti.main("ingin mengunduh aplikasi ini terima kasih"))
print(senti.main("terima kasih microsoft teams"))
print(senti.main("bagus dan bagus juga mudah untuk gunakan"))
print(senti.main("aplikasi sampah saya sudah daftar sampai mengasih kode dari nomor ponsel eh keluar ke halaman awal dan waktu mau masuk tidak bisa"))
print(senti.main("bagus"))
print(senti.main("mantap"))
print(senti.main("saat log masuk susah padahal nomornya sudah benar"))
print(senti.main("saya sebagai pengguna microsoft teams banyak mengalami kendala seperti tidak bisa kirim komentar saat rapat ponsel mulai ulang sendiri saat membuka aplikasi ini padahal kalau tidak buka normal saja yang terakhir terjadi eror aplikasi pindah halaman sendiri tolong pengembang segera diperbaiki dan juga setiap kali pakai harus perbarui hadeh"))
print(senti.main("istimewa"))
print(senti.main("bagus"))
print(senti.main("oke"))
print(senti.main("sangat membantu cukup puas"))
print(senti.main("bagus"))
print(senti.main("ini kenapa setiap mau bergabung zoom pertemuan selalu keluar terus setelah tekan tautan nya seperti itu terus sebelum nya lancar saja setelah di perbarui malah begini kecewa banget"))
print(senti.main("aplikasi nya bagus saya bisa belajar di teams yang bagus ini saya kasih peringkat pokok nya bagus deh"))
print(senti.main("ini kenapa ya sudah daftar akun saat masuk tidak bisa terus disuruh daftar akun email yang lain karena email sudah terdaftar bagaimana sih ini hadeh"))
print(senti.main("banyak eror"))
print(senti.main("lumayan"))
print(senti.main("kadang kadang berhenti di mencari jaringan dan lama untuk membuka foto padahal jaringan ku bagus tiba tiba disuruh masukkan kata sandi lagi dan kadang tugas tidak muncul di notifikasi aktivitas"))
print(senti.main("sering eror"))
print(senti.main("oke"))
print(senti.main("aplikasi dari the abad"))
print(senti.main("mantap"))
print(senti.main("bagus untuk sekolah kita"))
print(senti.main("mau rapat saja dipersulit"))
print(senti.main("aplikasi tidak bisa digunakan"))
print(senti.main("bagus bisa untuk pertemuan sekaligus mengumpulkan tugas ini mempermudah pembelajaran jarak jauh tolong tambahan fitur sematkan di teams dan diberi notifikasi jika ada rapat kadang suka telat masuk temu"))
print(senti.main("menyenangkan"))
print(senti.main("aplikasi ini sangat cocok untuk belajar walapun ada tersendat tersendat padahal jaringannya bagus semoga cepat di perbaiki masalahnya ya terima kasih"))
print(senti.main("perbaiki eror nyala ku buku chrome  tolong"))
print(senti.main("leletnya itu loh membuat kesal padahal sinyal lancar terus juga menghabiskan banyak daya"))
print(senti.main("aplikasi buruk terus perbarui"))
print(senti.main("tidak bisa bisa ke mengunduh"))
print(senti.main("ini kenapa saya setiap mau masuk saya di suruh tanda masuk terus dan saat saya sudah tanda masuk halaman nya mulai dari awal"))
print(senti.main("sangat membantu"))
print(senti.main("terima kasih"))
print(senti.main("setiap masuk ada eror berupa tugas tolong dihapus eror nya"))
print(senti.main("kenapa saat saya berhasil bergabung terus saat ingin melihat chat tidak bisa seperti terkunci di pertemuan"))
print(senti.main("bagus"))
print(senti.main("aplikasinya bagus banget"))
print(senti.main("mantap"))
print(senti.main("bagus"))
print(senti.main("sangat membantu sekali terima kasih microsoft teams semoga tidak ada kendala"))
print(senti.main("sangat direkomendasikan untuk aktivitas belajar mengajar"))
print(senti.main("bagus"))
print(senti.main("mantap betul"))
print(senti.main("mantap"))
print(senti.main("mantap"))
print(senti.main("sangat membantu"))
print(senti.main("bagus"))
print(senti.main("mantap"))
print(senti.main("sangat membantu"))
print(senti.main("oke"))
print(senti.main("bagus aplikasi"))
print(senti.main("membantu mahasiswa undip banget"))
print(senti.main("kok tidak bisa mengunduh ya"))
print(senti.main("setiap ganti profil kadang suka eror atau tidak terbaca padahal profil sudah ganti eh malah mengulang profil yang pertama"))
print(senti.main("saya"))
print(senti.main("habis di perbarui kok saluran tugas hilang jadi tidak bisa lihat tugas yang sudah jatuh tempo tapi belum di tutup harap perbaiki"))
print(senti.main("oke"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("bagus bagus"))
print(senti.main("saya tidak bisa menerima sms kode di ponsel saya"))
print(senti.main("aplikasi sangat bagus dan memudahkan pengguna"))
print(senti.main("saya tidak suka aplikasi ini ini membuat saya stres dan pusing"))
print(senti.main("sangat membantu untuk pembelajaran jarak jauh daring terima kasih"))
print(senti.main("selalu eror"))
print(senti.main("tidak bisa dibuka file nya saat mengunggah file"))
print(senti.main("aplikasi sering eror misal tugas sudah dikumpulkan tapi tidak sampai"))
print(senti.main("microsoft teams ini mengunduh nya lama banget sampai emosi pengunduhan selalu tertunda"))
print(senti.main("keren"))
print(senti.main("keren"))
print(senti.main("bagus"))
print(senti.main("eror aplikasinya saat saya memasang tidak bisa"))
print(senti.main("terima kasih sudah membantu"))
print(senti.main("bagus sekali"))
print(senti.main("sangat memuaskan"))
print(senti.main("ini dari aku kelas kalau pertemuan tuh pakai ini lebih sering dari pada zoom sih apa² pakai ini"))
print(senti.main("bagus bisa rapat di manapun ponsel"))
print(senti.main("sangat bagus"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("aplikasi ribet tidak jelas seperti tahi"))
print(senti.main("pada"))
print(senti.main("suka eror tidak bisa buka file tugas sama absen jadi harus lepas terus pasang ulang tiap eror huh"))
print(senti.main("bagus"))
print(senti.main("bagus dan muda untuk mengumpulkan tugas dari sekolah"))
print(senti.main("aplikasi nya memang berarti untuk belajar aplikasi sangat bagus"))
print(senti.main("ada eror yang dimana sudah bergabung grup suatu mata pelajaran tiba tiba grupnya hilang dan tidak bisa dicari cuma ada notifikasi berhasil bergabung nya padahal lagi ulangan grupnya untuk temu disitu untung pakai teams yang di web tidak nge eror tolong perbaiki ini"))
print(senti.main("mantap"))
print(senti.main("biasa saja"))
print(senti.main("oke"))
print(senti.main("cukup mengerti dan tidak mau tahu hehe apa kamu lihat lihat kebawah dasar anak kecil haram"))
print(senti.main("bagus"))
print(senti.main("mantap"))
print(senti.main("bagus"))
print(senti.main("aplikasi nya tidak bisa di mengunduh kenapa ya"))
print(senti.main("agak ribet kalau di ponsel"))
print(senti.main("ingin ke googlemeet saja deh ribet amat pakai aplikasi ini ganti profil saja harus menunggu sehari baru berubah"))
print(senti.main("mantap"))
print(senti.main("oke"))
print(senti.main("lelet amat mengunduh nya"))
print(senti.main("memuat terus padahal pakai youtube lancar"))
print(senti.main("jelek banget teams itu bagus lah teams itu sungguh bagus"))
print(senti.main("sok buruk"))
print(senti.main("aplikasi rusak"))
print(senti.main("bagus"))
print(senti.main("membuat guru saya mengasih tugas"))
print(senti.main("emosikon nya bisa goyang"))
print(senti.main("anjay"))
print(senti.main("tahi sering banget eror kontol"))
print(senti.main("baik"))
print(senti.main("yayaya"))
print(senti.main("berguna karena tugas sekolah wkwk"))
print(senti.main("bagus"))
print(senti.main("anak belajar dengan baik materi dan tugaspun mudah dipelajari dan dikirim oke banget deh"))
print(senti.main("bekerja normal"))
print(senti.main("bagus"))
print(senti.main("oke"))
print(senti.main("mantap sekali"))
print(senti.main("bagus"))
print(senti.main("di perangkat ponsel masih banyak eror"))
print(senti.main("bagus"))
print(senti.main("ditagih tugas mulu bangsat"))
print(senti.main("mantap"))
print(senti.main("oke"))
print(senti.main("makanan"))
print(senti.main("kok tidak bisa di mengunduh sih padahal sudah persen loh menghabiskan kuota"))
print(senti.main("bagus"))
print(senti.main("oke"))
print(senti.main("ini aplikasi the terbaik sih"))
print(senti.main("membuat frustasi"))
print(senti.main("oke"))
print(senti.main("aplikasinya semakin aneh setiap dibuka selalu memuat terus waktu dilepas juga mau mengunduh lagi tidak bisa terus"))
print(senti.main("kenapa kalau pakai kartu tri sering tidak bisa menggunakan teams untuk pertemuan padahal sinyal bagus untuk yang lain juga bisa"))
print(senti.main("bagus"))
print(senti.main("bahagia"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("oke"))
print(senti.main("bagus ini teams aplikasi yang sangat membantu sekolah di waktu pandemi"))
print(senti.main("membantu saya dalam sekolah"))
print(senti.main("aplikasi nya bagus cocok untuk daring"))
print(senti.main("banyak eror di bagian pilih alat menyalin pilih nya kadang muncul kadang tidak tolong di perbaiki"))
print(senti.main("tersendat banget aku pakai laptop sudah menghabiskan mbak tapi tidak bisa"))
print(senti.main("tautan yang bagus dan bermanfaat"))
print(senti.main("keren banget sangat membantu pekerjaan saya terutama untuk pertemuan daring"))
print(senti.main("bagus"))
print(senti.main("masuk ribet"))
print(senti.main("sangat membantu"))
print(senti.main("sangat baik"))
print(senti.main("dapat dibuat belajar dengan baik sangat berguna juga untuk sekolah"))
print(senti.main("lelet kok hidupmu"))
print(senti.main("oke"))
print(senti.main("karena bisa main sama teman kalau lagi covid aku tetap senang"))
print(senti.main("i punya untuk zoom karena ku guru sudah untuk zoom masuk microsoft teams"))
print(senti.main("keren"))
print(senti.main("jelek"))
print(senti.main("kalau mengirim file di tugas tolong file nya yang terlanjur terkirim bisa di hapus lagi saya salah mengirim file malah tidak bisa dihapus"))
print(senti.main("lelet kalau mengunduh tolong perbaiki nya"))
print(senti.main("bagus sekali"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("sangat membantu saya dalam aktivitas sehari hari"))
print(senti.main("saya sudah lagi nge mengunduh dan lagi di perjalanan selesai ke kok malah balik lagi ke tolong lah"))
print(senti.main("aplikasi ini bisa membantu saya belajar dan daring"))
print(senti.main("bagus"))
print(senti.main("mantap"))
print(senti.main("mantap"))
print(senti.main("memuat terus anjing"))
print(senti.main("oke"))
print(senti.main("bagus pekerjaan"))
print(senti.main("suka mudah pemakaiannya"))
print(senti.main("karena bisa membantu saya mengerjakan tugas saya"))
print(senti.main("membantu sekali"))
print(senti.main("bagus aplikasinya"))
print(senti.main("bagus"))
print(senti.main("mau masuk saja susah padahal sudah selesai arahan saya menyarankan jangan pakai aplikasi ini menyesal banget"))
print(senti.main("aplikasi ini membuat kita pintar"))
print(senti.main("bagus"))
print(senti.main("aplikasi teams bagus banget teman teman buruan mengunduh kalau tidak mengunduh nanti menyesal deh tidak menyesal"))
print(senti.main("gambar dan suara jelas"))
print(senti.main("kadang cepat merespon kadang lambat padahal aku butuh banget untuk menyelesaikan tugas yang diberikan oleh guru"))
print(senti.main("sangat bagus"))
print(senti.main("aplikasi tidak guna tidak bisa untuk kirim sama temu"))
print(senti.main("kesalahan"))
print(senti.main("sangat baik terima kasih"))
print(senti.main("kenapa saya setelah masuk untuk rapat selalu keluar sendiri tolong di perbaiki lah untuk aplikasi nya semakin kesini semakin banyak kendalanya"))
print(senti.main("banyak eror"))
print(senti.main("kenapa sih kok perbarui terus tiap mau mengerjakan tugas harus memperbarui terlebih dahulu tolong memperbaiki lagi teams nya ya kok setiap bulan perbarui terus"))
print(senti.main("sangat membantu kalau tidak mengunduh akan menyesal selama nya"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("oke"))
print(senti.main("mantap"))
print(senti.main("oke"))
print(senti.main("banyak tugas oyy"))
print(senti.main("tatap matamu bagai busur panah yang kau lepaskan ke jantung hatiku meski kau simpan cintamu masih"))
print(senti.main("teams saya eror kalau pertemuan ada kata kata sandi sudah benar sesuai tapi malah sistem salah ini beberapa kali"))
print(senti.main("mudah untuk gunakan fitur wah"))
print(senti.main("hebat aplikasinya bisa untuk belajar anak saya"))
print(senti.main("aplikasinya bagus banget"))
print(senti.main("bagus"))
print(senti.main("tidak bisa diunduh kadang keluar sendiri"))
print(senti.main("sangat bagus dan mudah untuk di gunakan"))
print(senti.main("kualitas suara buruk kualitas gambar buruk"))
print(senti.main("aplikasi ini sangat membantu untuk mengerjakan tugas"))
print(senti.main("burik"))
print(senti.main("sederhana dan cepat"))
print(senti.main("bagus"))
print(senti.main("tolong menjadi untuk the teams aplikasi the tersembunyi channel fitur akan menjadi dihapus masuk the lanjut perbarui karena the fitur sok menyebalkan"))
print(senti.main("bagus"))
print(senti.main("jelek"))
print(senti.main("aplikasi bagus"))
print(senti.main("berisik"))
print(senti.main("the terbaik"))
print(senti.main("selamat pagi saya memang suka sama teams dan saya bisa mengerjakan tugas dengan baik saya suka banget sama teams karena saya sekolah di tunas pakai daring pakai teams kalau anda mau gabung dengan tunas silahkan anda tanya dengan dinas dan segera membayar di bank langsung gabung karena teams ini bagus dan semua semua guru guru baik cuma kelas ambil nilai orang lain kalian harus tahu karena kelas ibu tita kamu tidak tahu ibu tita saya nilai dia ambil kasih orang lain jadi sampai sini teri"))
print(senti.main("karena aplikasi teams sangat baik jadi i seperti"))
print(senti.main("aplikasi nya jelek banget kalian jangan mengunduh ini deh"))
print(senti.main("oke"))
print(senti.main("mantap"))
print(senti.main("mantap membantu dalam pekerjaan"))
print(senti.main("kenapa tidak bisa buka melalui tautan"))
print(senti.main("aplikasi mempersulit orang"))
print(senti.main("jelek banget teams nya eh tapi bohong wkwk bagus kok teams nya saya kasih bintang deh belum masuk di aplikasi nya terima kasih ya teams"))
print(senti.main("kenapa tidak ada notifikasi terus kadang file yang di beri tidak bisa di buka"))
print(senti.main("aplikasi yang sangat membantu untuk sekolah daring"))
print(senti.main("masih teams tolong diperbaiki masa tidak bisa log masuk ke log keluar terus sudah masukkan kode yang benar tidak bisa masuk"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("mantap"))
print(senti.main("mantap sangat membantu"))
print(senti.main("oke"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("sangat membantu proses pembelajaran jarak jauh"))
print(senti.main("bagus"))
print(senti.main("mantap"))
print(senti.main("aplikasi ini sangat menguntungkan dan bagus bagi saya"))
print(senti.main("ini aku sangat senang dengan tugas saya"))
print(senti.main("teams sekarang tidak bisa dipakai di windows di oppo fs tolong pengembang teams diperbaiki ya"))
print(senti.main("aplikasinya bagus banyak fitur yang bisa dimanfaatkan apalagi di era pembelajaran daring"))
print(senti.main("aplikasi nya bagus tetapi agak kurang suka saja"))
print(senti.main("mudah dan praktis"))
print(senti.main("aplikasi nya sangat bagus"))
print(senti.main("karena membantu saya dalam bersekolah"))
print(senti.main("nih bagaimana disuruh memasukkan kode dari nomor ponsel tapi tidak ada kode yang masuk"))
print(senti.main("sangat top top mantap"))
print(senti.main("pp profil saya tidak bisa diganti sudah di coba tapi masih saja sama tolong diperbaiki"))
print(senti.main("oke"))
print(senti.main("ini teams bermasalah apa wifi saya ya padahal sinyal nya kuat loh untuk aplikasi lain tapi di teams tidak mau ke buka tolong di benarkan"))
print(senti.main("untuk pihak microsoft perusahaan saya meminta tolong di bagian fitur file bagi pengguna pelajar mohon sebesar besarnya supaya di bagian file tersebut ada layanan hapus hanya disitu keluhan saya terima kasih"))
print(senti.main("membuat pusing"))
print(senti.main("pertahankan dan terus kembangkan untuk yang terbaik"))
print(senti.main("mantap deh"))
print(senti.main("oke"))
print(senti.main("oke"))
print(senti.main("ini sangat bagus"))
print(senti.main("bagus"))
print(senti.main("aplikasinya bagus tapi notifikasi tugasnya merusak pemandangan"))
print(senti.main("aplikasi ini sangat bagus"))
print(senti.main("siap sangat membantu pembelajaran jarak jauh"))
print(senti.main("keren dengan macam fitur nya yang mendukung"))
print(senti.main("semakin tidak jelas nih aplikasi tiba tiba sering log keluar sendiri membuat ribet kadang tiba tiba kamera tidak fungsi parah"))
print(senti.main("oke terima kasih deh dengarkan keluhanku sebelumnya dan sudah diperbaiki terima kasih channel yang hilang deh ada lagi"))
print(senti.main("bagus sekali"))
print(senti.main("banyak banget wahyu tadi menghisap telok kok sudah e kok katek kok"))
print(senti.main("sangat bermanfaat"))
print(senti.main("tidak bisa di mengunduh"))
print(senti.main("mendingan zoom saya mah ini aplikasi tidak guna kalau mau bergabung pertemuan susah banget pakai ini"))
print(senti.main("saya sekarang presentasi kenapa saya tidak bisa masuk sudah lebih kali saya coba masuk tapi kenapa masih tidak bisa"))
print(senti.main("sangat membantu di masa pandemi ini untuk koordinasi dan sosialisasi"))
print(senti.main("untuk belajar alhamdulillah berjalan untuk kirim tugas sama pertemuan masya allah"))
print(senti.main("benarkan dulu notifikasinya masa ada pertemuan dari tadi tidak dikasih tahu ada notifikasi dari laptop maupun ponsel gunanya sembunyi channel juga untuk apa menyembunyikan temu hapus saja tidak guna"))
print(senti.main("aplikasi nya sangat bagus banget"))
print(senti.main("ini aplikasi sangat bagus"))
print(senti.main("kenapa habis di perbarui terus memasukkan sandi lagi tidak bisa bisa padahal sandi sudah betul"))
print(senti.main("hebat"))
print(senti.main("keren"))
print(senti.main("bagus banget bisa untuk sekolah sama kerja saya sekolah ayah ku kerja pakai aplikasi ini pelajar dari darul surga"))
print(senti.main("penghambat kuliah"))
print(senti.main("kenapa kalau bahasa melayu selalu saja keluar dari rapat"))
print(senti.main("sangat membantu kerja"))
print(senti.main("bagus banget"))
print(senti.main("mantap keren dan mudah"))
print(senti.main("tolong perbaiki aplikasi ini atau kalau perlu hapus dari play store gara aplikasi ini rapat saya batal"))
print(senti.main("respon nya lambat eror terus"))
print(senti.main("sangat membantu dalam hal pekerjaan"))
print(senti.main("kenapa semakin lama teams semakin banyak tugas padahal lagi tekan rangking"))
print(senti.main("tugasnya tidak bisa di buka buruan benarkan nanti nilai saya turun"))
print(senti.main("saya sering tidak bisa membuka tab tugas"))
print(senti.main("mantap"))
print(senti.main("aplikasi ini bagus"))
print(senti.main("bagus"))
print(senti.main("mantap"))
print(senti.main("aplikasi nya sangat bagus bisa pindah masuk akun teams dari laptop ke ponsel kalau masih pergi luar kota"))
print(senti.main("top"))
print(senti.main("bagus aplikasi"))
print(senti.main("itu bagus banget"))
print(senti.main("kalau mengirim tugas lama banget padahal wifi lancar jaya"))
print(senti.main("sering eror jaringan bagus masih saja tersendat"))
print(senti.main("kadang sandi nya di salahkan tapi oke lah tolong di perbaiki yang admin"))
print(senti.main("sangat membantu saya untuk belajar daring bersama teman"))
print(senti.main("keren sekali"))
print(senti.main("bagus"))
print(senti.main("ini jelek"))
print(senti.main("akun suka keluar dari aplikasi jadi harus di pasang ulang baru bisa"))
print(senti.main("buruk"))
print(senti.main("oke"))
print(senti.main("ya baik"))
print(senti.main("selalu disuruh masuk padahal saya sudah masuk"))
print(senti.main("terlalu banyak eror dan sering mengalami file tidak bisa dibuka"))
print(senti.main("kontol tersendat"))
print(senti.main("tidak ada background nya"))
print(senti.main("mantap"))
print(senti.main("ah yemete"))
print(senti.main("bagus aplikasi"))
print(senti.main("oke"))
print(senti.main("saya banyak terbantu dengan kemudahan aplikasi utamanya saat menghadapi jadwal konferensi video yang sangat ketat dan mendadak teams sangat membantu dan memudahkan saya untuk koordinasi dengan seluruh teams kapanpun"))
print(senti.main("keren"))
print(senti.main("teams jadi tidak terlalu bagus setelah perbarui chat di kolom tidak bisa di edit atau di hapus lagi ketika salah ketika lah kirim"))
print(senti.main("aplikasi ini sangat berguna"))
print(senti.main("untuk daring ku sangat bagus kok"))
print(senti.main("aplikasinya lelet lelet padahal jaringan bagus bagus saja"))
print(senti.main("suka keluar sendiri terus juga konten yang sedang di tunjukkan tidak bisa muncul aplikasi juga sering terhenti"))
print(senti.main("burik"))
print(senti.main("kadang tidak bisa deh aneh"))
print(senti.main("perbarui terus penuh ini ponsel anjing"))
print(senti.main("bagus banget"))
print(senti.main("oke"))
print(senti.main("tolong dong bagi pengembangnya di perbarui apa nih teams soalnya aku mau buka saluran pelajaran ku susah amat serius tidak bisa tolong ya bagi pengembang nya aku kasih bintang dulu kalau deh bagus baru aku naikkan bintang nya jadi itu saja ulasan dari aku"))
print(senti.main("oke cukup membantu"))
print(senti.main("bagus"))
print(senti.main("oke"))
print(senti.main("mantap"))
print(senti.main("bermanfaat"))
print(senti.main("bagus"))
print(senti.main("kenapa saya tidak bisa hapus akun lain saya dan kenapa tiba tiba keluar sendiri anjing"))
print(senti.main("teams adalah terbaik aplikasi"))
print(senti.main("baik"))
print(senti.main("sangat membantu saya untuk melaksanakan kuliah daring di rumah"))
print(senti.main("kenapa saya beri bintang lima sok al nya saya suka sama aplikasi ini"))
print(senti.main("aplikasi nya sangat membantu saya untuk mengikuti pembelajaran daring terima kasih"))
print(senti.main("bagus untuk belajar"))
print(senti.main("aplikasi teams ini sangat membantu saya untuk sekolah daring"))
print(senti.main("bagus kok aplikasi nya kalau aku pakai ipad itu pasti macet macet karena memori nya penuh terima kasih salam dari binjai binjai eh tapi bohong"))
print(senti.main("mantap"))
print(senti.main("oke"))
print(senti.main("terima kasih"))
print(senti.main("bagus"))
print(senti.main("permainan nya bagus"))
print(senti.main("kenapa jaringan saya bagus cuma mau masuk ke pelajaran nya selalu mutar mutar terus"))
print(senti.main("terlalu banyak notifikasi saya tidak suka"))
print(senti.main("kenapa tiba tiba eror ya setelah di perbarui saya jadi tidak bisa daring dan mengumpulkan tugas"))
print(senti.main("aplikasi ini bagus lah untuk pertemuan mengerjakan tugas"))
print(senti.main("tidak ada notifikasi e memuat tugas lama sekali bahkan tidak muncul kalau tidak gara gara tugas sekolah ini aplikasi sepi tidak ada yang mengunduh dan nilai saya terancam karena aplikasi"))
print(senti.main("bagus"))
print(senti.main("teams"))
print(senti.main("saya senang sekali karena aplikasi nya bagus"))
print(senti.main("tidak mau ter unduh"))
print(senti.main("apakah bisa ditambahkan notifikasi saat ada yang menendang maupun mematikan mikrofon saat berada dalam teams"))
print(senti.main("bagus"))
print(senti.main("mau masuk sulit banget mual seperti kentut"))
print(senti.main("bagus"))
print(senti.main("setiap perbarui harus masuk lagi kalau ada tugas memuat terus kadang konferensi video eror"))
print(senti.main("baru pengguna pemula"))
print(senti.main("sempurna"))
print(senti.main("tolong ada the opsi untuk kami untuk pergi the teams jika kami ingin untuk i bergabung a salah kelas teams dan i wanted untuk pergi tapi mereka tidak memberi saya the opsi untuk pergi masuk keduanya ku android dan komputer dan itu menyebalkan sok tolong ada itu"))
print(senti.main("aplikasi nya selalu perbarui"))
print(senti.main("miring berkualitas"))
print(senti.main("ribet"))
print(senti.main("padahal sinyal nya bagus tapi kenapa saat masuk temu tidak bisa melulu sistem nya jelek lagi"))
print(senti.main("terima kasih"))
print(senti.main("sumpah dah jelek banget aplikasinya kalau seperti panggilan video begitu suka tidak ada suaranya terus kalau kirim tugas tulisannya kosong kamu terkirim dan untuk saya sendiri saya tidak bisa bergabung ke tim kelas saya"))
print(senti.main("bagus"))
print(senti.main("kok tiba tiba aplikasi nya eror ya materi nya hilang semua tugas hilang tolong benarkan kalau sudah saya kasih bintang lima deh mau ujian ini"))
print(senti.main("aplikasi e jelek banget"))
print(senti.main("jelek banget jangan di mengunduh"))
print(senti.main("membuat banyak tugas ingin"))
print(senti.main("bagus aplikasi"))
print(senti.main("bagus"))
print(senti.main("tolong dong kok setelah di perbarui kolom tugasnya hilang"))
print(senti.main("untuk kerja"))
print(senti.main("teams aplikasi yang sangat hebat"))
print(senti.main("setelah perbarui versi kenapa menu bagikan video menjadi tidak ada di ponsel samsung"))
print(senti.main("kuat aplikasi"))
print(senti.main("bagus"))
print(senti.main("kesalahan"))
print(senti.main("tolong batasi penggunaan fitur sorotan saat pertemuan"))
print(senti.main("untuk belajar penting ini jadi saya pasang ini aplikasi ini bagus untuk siswa jadi ayo mengunduh dulu aplikasi ini ya bagi siswa yang membutuhkan sesuatu"))
print(senti.main("aplikasi yang bagus"))
print(senti.main("tolol aplikasi tolol kenapa yang main tendang dalam rapat tidak ditulis"))
print(senti.main("oke tapi berat"))
print(senti.main("jelek banget anjir ini aplikasi"))
print(senti.main("tolong supaya bisa ada wallpaper di belakang kita di android"))
print(senti.main("bagus dan sangat mudah untuk memakainya"))
print(senti.main("oke"))
print(senti.main("seperti"))
print(senti.main("nomor ragu"))
print(senti.main("bagus"))
print(senti.main("tidak ramah selalu ada masalah ketika bergabung pertemuan"))
print(senti.main("bosan daring corona pergilah yang jauh"))
print(senti.main("tidak bisa masuk"))
print(senti.main("terbaik aplikasi"))
print(senti.main("memuat nya tidak mengotak gara² kamu saya jadi alfa gara² tidak bisa masuk masuk pertemuan aplikasi haram perbarui tidak guna sialan"))
print(senti.main("aplikasi ini sayat seperti biasa"))
print(senti.main("fasilitas pembelajaran daring yang baik"))
print(senti.main("aplikasinya sangat membantu anak saya dalam belajar"))
print(senti.main("baru verifikasi saja lama banget padahal nomornya sudah benar"))
print(senti.main("keren"))
print(senti.main("puas"))
print(senti.main("butuh chat balas seperti whatsapp"))
print(senti.main("bagus"))
print(senti.main("oke"))
print(senti.main("sering eror tapi bagus kok"))
print(senti.main("tersendat"))
print(senti.main("bagus"))
print(senti.main("ini kenapa setelah perbarui tugas nya hilang semua tulisannya kartu tidak didukung padahal masih ada tugas yang harus saya kumpulkan tolong dong bagaimana ini"))
print(senti.main("oi teams kok lama"))
print(senti.main("bagus"))
print(senti.main("banyak menghabiskan ruang penyimpanan aplikasi suka tersendat boros baterai"))
print(senti.main("oke"))
print(senti.main("bagus banget"))
print(senti.main("menggila"))
print(senti.main("buka tugas lelet di perbarui masuk ulang bintang"))
print(senti.main("saat memuat papan tulis lama sekali"))
print(senti.main("sangat membantu"))
print(senti.main("setiap kirim verifikasi nomor telepon kode tidak pernah masuk"))
print(senti.main("bagus"))
print(senti.main("oke"))
print(senti.main("bagus sangat membantu"))
print(senti.main("ini punya saya kok keluar sendiri terus ya saat tampilan mau masuk"))
print(senti.main("aplikasi bagus"))
print(senti.main("sangat membantu bisa chat perbaruan dirumah"))
print(senti.main("bagus"))
print(senti.main("siap"))
print(senti.main("bagus aplikasi"))
print(senti.main("paling cocok untuk kuliah"))
print(senti.main("tidak bisa di mengunduh"))
print(senti.main("tidak guna"))
print(senti.main("teams ini bagus bisa untuk islam juga"))
print(senti.main("ketika ada notifikasi saya beri tanda suka namun saat saya membuka aplikasinya tidak ada tanda suka dari saya lalu saya beri tanda suka lagi tapi ketika saya masuk ulang ke dalam aplikasinya tanda suka dari saya menghilang lagi bagaimana ini saya menuntut agar segera diperbaiki"))
print(senti.main("sangat mudah digunakan untuk siswa sekolah"))
print(senti.main("halo pengembang tolong dong bagaimana ya cara mengatasi supaya tidak di tendang terus"))
print(senti.main("mantap jiwa"))
print(senti.main("tiap hari di kasih tugas"))
print(senti.main("bagus"))
print(senti.main("bagus banget coy¡¡…¿"))
print(senti.main("tempat ganti nama dimana bang"))
print(senti.main("bagus"))
print(senti.main("dapat belajar dengan baik"))
print(senti.main("oke sangat bagus"))
print(senti.main("ini menu tugasnya kenapa hilang"))
print(senti.main("tolong ini kenapa teams saya eror saya sudah masukkan sandi dari beberapa bulan yang lalu dan semuanya sudah benar tetapi sekarang saya di suruh memasukkan sandi lagi padahal sudah dari dulu sekarang sandi nya aku lupa karena sandinya di untuk beberapa bulan lalu tolong ini kenapa ya tiba tiba di suruh masukkan sandi lagi padahal hari hari sebelumnya normal normal saja tidak ada yang seperti itu"))
print(senti.main("jangan lelet ya"))
print(senti.main("aneh"))
print(senti.main("aplikasi microsoft teams sangat penting"))
print(senti.main("ribet banget"))
print(senti.main("tidak bisa bisa memasang mengempos"))
print(senti.main("sama teams sangat membantu dalam pembelajaran"))
print(senti.main("apa saat masuk di siaran selalu memuat tidak terbuka buka kebiasaan atau seperti mana tuh"))
print(senti.main("tolong kenapa saya tidak bisa masuk"))
print(senti.main("oke saja"))
print(senti.main("the terbaik untuk saya yang masih pelajar terima kasih kamu teams"))
print(senti.main("sangat bagus"))
print(senti.main("sangat bagus untuk kuliah daring tapi sering eror masih teams nya"))
print(senti.main("kadang tidak ada notifikasi masuk padahal sudah dinyalakan semua notifikasinya kalaupun masuk pasti selalu terlambat ini juga sering tidak bisa memuat tugas tolong benarkan lagi hanya karena kesalahan pada aplikasi nilai saya menjadi terancam"))
print(senti.main("mantap"))
print(senti.main("mantap"))
print(senti.main("berguna untuk daring saya banget"))
print(senti.main("bagus"))
print(senti.main("jelek banget kok balik saat ikuti rapat"))
print(senti.main("hasil unduhan dari teams ekspor untuk excel berantakkan sulit untuk dibaca hasilnya"))
print(senti.main("baik"))
print(senti.main("oke banget"))
print(senti.main("ini bagus sekali bisa membantu saya belajar dengan benar dan saya bisa zoom di sini seru banget untuk sekolah daring bagus lah bintang dong sebagus ini terima kasih ya"))
print(senti.main("kok kata sandinya salah mulu sih padahal sudah benar memasukkan kata sandinya"))
print(senti.main("assalamualaikum warahmatullahi wabarokatuh saya kasih bintang karena aplikasi microsoft teams membantu saya untuk daringaplikasi microsoft teams sangat lah bagus dan terbaik jazaakumullah khairan aplikasi microsoft teams sudah membantu saya untuk daringsaya sudah memakai aplikasi microsoft teams sudah dari kelas sampai kelas sampai ini"))
print(senti.main("amin"))
print(senti.main("bagus aplikasi kenapa tidak bisa di jatuh"))
print(senti.main("ini aplikasi ada eror nya tolong di perbaiki masak saya lagi makan muncul tugas tolong di perbaiki eror nya"))
print(senti.main("teams fitur tidak lengkap kalau mau keluar dari kelas tidak ada fitur keluar pokoknya teams jelek banget"))
print(senti.main("tidak bisa masuk akun setelah diperbarui"))
print(senti.main("untuk mengunggah tugas ges"))
print(senti.main("karena saya bisa mengumpulkan tugas dan mengikuti sekolah dengan baik"))
print(senti.main("mantap"))
print(senti.main("bagus"))
print(senti.main("wow"))
print(senti.main("pekerjaan saya cukup membantu"))
print(senti.main("baik"))
print(senti.main("sangat membantu siswa"))
print(senti.main("aplikasi yang bagus dan bermanfaat sekali"))
print(senti.main("aplikasi memang bagus namun materi yang di bagi guru kadang tidak kelihatan kalau mau saran tambah latar belakang yang bergerak supaya lebih menarik"))
print(senti.main("mantap"))
print(senti.main("sangat baik dan berguna"))
print(senti.main("tidak bisa di mengunduh"))
print(senti.main("mikrofon nya eror bagaimana ini caranya di perbaiki"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("masalah tidak bisa masuk mohon bantuannya"))
print(senti.main("oke"))
print(senti.main("sering muncul notifikasi tugas"))
print(senti.main("jelas gambarnya"))
print(senti.main("bagus banget"))
print(senti.main("aplikasi tidak bisa di ajak kompromi sudah pakai data tidak bisa masa pakai wifi juga tidak bisa anjing mau sekolah susah saya"))
print(senti.main("bagus aplikasi"))
print(senti.main("serius mau untuk tugas tapi tugasnya berhenti di memuat padahal koneksi lancar² saja"))
print(senti.main("sangat bagus"))
print(senti.main("aplikasi ya bagus banget"))
print(senti.main("sangat membantu"))
print(senti.main("ikut memakai"))
print(senti.main("mantap"))
print(senti.main("lebih mudah dan sederhana untuk kerja"))
print(senti.main("ribet"))
print(senti.main("ini aplikasi nya sangat jelek karena ini selalu tersendat terus bahkan ponsel nya jadi tersendat terus semoga sekolah ku ganti bukan di teams lagi"))
print(senti.main("oke aplikasi tidak lelet"))
print(senti.main("wajib ada"))
print(senti.main("jelek setiap buka tautan absen selalu diminta akun log masuk ulang dan ketika memasukkan kata sandi selalu salah ini aplikasi agen rahasia atau aplikasi pelajar ketat benar"))
print(senti.main("tolong diperbaiki ya aplikasi nya soal nya kalau ubah mikrofon ke speaker jadi nya terlempar keluar sedikit mengecewakannya kasih bintang terima kasih"))
print(senti.main("banyak eror"))
print(senti.main("teams nya sekarang eror tidak seperti dulu"))
print(senti.main("susah banget untuk masuk padahal seharusnya kalau aplikasi untuk sekolah lebih dipermudah ini menyusahkan banget"))
print(senti.main("lancar dalam pertemuan tidak putus putus kalau sinyal bagus"))
print(senti.main("eror mulu aplikasi sialan tidak jelas"))
print(senti.main("perbaiki ini papan tulis nya selalu memuat selalu tidak ada gabung tapi kadang baterainya boros"))
print(senti.main("saya suka karena aplikasinya fleksibel bisa dibuka di aplikasi ponsel komputer maupun browser jadi tidak perlu khawatir jika memori sedang penuh bisa buka lewat browser mudah juga saat ingin melihat aktivitas terbaru"))
print(senti.main("akhir akhir ini kenapa banyak eror ya tolong di perbaiki lagi pengembang"))
print(senti.main("suka eror membuat suasana hati jelek"))
print(senti.main("susah masuk dan bilin tautan pertemuan"))
print(senti.main("baru saja di perbarui bagian menu tugas kemana hey microsoft di cari tidak ada mengasih tugasnya bagaimana coba"))
print(senti.main("aplikasi ini bagus juga untuk sekolah"))
print(senti.main("tidak dapat memasang aplikasi di android"))
print(senti.main("saya terganggu saat baru masuk kenapa setiap saya kirim kode verifikasi ke nomor saya tidak terkirim kirim padahal itu nomor telepon asli saya untung saya hanya bergabung teams tidak lebih dari itu tapi mohon diperbaiki siapa tahu nanti saya gunakan lagi supaya saya bisa masuk akun"))
print(senti.main("bagus"))
print(senti.main("bagus tapi bagus"))
print(senti.main("tolong di hapus aplikasinya saya malas pembelajaran jarak jauh bangsat"))
print(senti.main("ini seru tapi mohon microsoft nya di tingkat lagi ya"))
print(senti.main("gagal interview gara tidak bisa bergabung padahal sinyal oke banget"))
print(senti.main("setiap saya mau mengumpul kan tugas pasti memuat terus"))
print(senti.main("sederhana dan bermanfaat"))
print(senti.main("tiba tiba keluar dari rapat tanpa ada alasan"))
print(senti.main("kadang eror kadang tidak"))
print(senti.main("bagus"))
print(senti.main("papan tulisnya katanya terjadi kesalahan kesal"))
print(senti.main("banyak eror paksa tutup sendiri najis ikut temu tidak bisa sering banget keluar sendiri najis banget"))
print(senti.main("karena bisa bantu banget disaat daring"))
print(senti.main("lebih mudah untuk komunikasi untuk pertemuan teams"))
print(senti.main("banyak tugas"))
print(senti.main("mantap"))
print(senti.main("sangat tidak ramah bintang"))
print(senti.main("sangat bagus"))
print(senti.main("saking bagus nya mutar terus di huawei nova"))
print(senti.main("aplikasi nya keren dan sekolah saya juga pakai dan yang kesulitan masuk bisa meminta tolong ke ayah ibu pasti mereka tahu pelajar dari sekolah dian harapan sudah"))
print(senti.main("bagus"))
print(senti.main("bagus sih bagus tapi bintang karena mikrofon saya eror terus dari kemarin padahal tidak pakai telinga ponsel headset tolong teams dibetulkan agar bisa untuk sekolah bekerja dan lain"))
print(senti.main("selalu mengalami masalah ketika masuk menggunakan ponsel android sangat mengecewakan"))
print(senti.main("keren"))
print(senti.main("kenapa deh kok saat mau masuk rapat malah di suruh konfirmasi akun padahal deh masuk bisa dijawab"))
print(senti.main("tolong nih masa selesai saya perbarui disuruh memasukkan sandi saat sudah dimasuki sandi benarnya kok disalahkan sih bagaimana nih saya mau mengerjakan tugas tidak bisa karena sandi"))
print(senti.main("bagus"))
print(senti.main("bagus banget"))
print(senti.main("suka"))
print(senti.main("aplikasi bagus"))
print(senti.main("mengunduh lama banget"))
print(senti.main("bagus"))
print(senti.main("siap"))
print(senti.main("memuat langka selesainya kontol"))
print(senti.main("mantap"))
print(senti.main("oke"))
print(senti.main("suka keren abc"))
print(senti.main("sudah coba daftar belum bisa masuk"))
print(senti.main("membantu"))
print(senti.main("sangat bijak"))
print(senti.main("oke"))
print(senti.main("i harapan masuk the lanjut perbarui kami bisa menemukan keluar siapa the orang siapa ditendang kami keluar dari the temu"))
print(senti.main("enak untuk belajar terima kasih tapi kadang membuat kaget karena ada tugas"))
print(senti.main("terima kasih"))
print(senti.main("bagus aplikasi tapi masih tidak akrab"))
print(senti.main("kenapa saya selalu ada tulisan seseorang mengeluarkan anda begitu"))
print(senti.main("tidak bisa di mengunduh"))
print(senti.main("lumayan lah untuk daring"))
print(senti.main("sangat membantu untuk kegiatan daring"))
print(senti.main("bagus"))
print(senti.main("mantap"))
print(senti.main("tolong kenapa eror terus anjing"))
print(senti.main("oke deh"))
print(senti.main("teams lama semakin tidak nyaman kalau dipakai untuk media belajar tolong benarkan ini sinyal dirumah saya bagus kenapa lelet banget saat buka teams mana setiap buka tugas harus masuk dulu"))
print(senti.main("sangat bagus dan bermaafat"))
print(senti.main("membantu pekerjaan saya"))
print(senti.main("berat"))
print(senti.main("lumayan sih tapi saat di unduh mbak sesudah unduh mbak"))
print(senti.main("oke lah"))
print(senti.main("aplikasi jelek tidak guna norak"))
print(senti.main("sangat membantu"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("setelah saya perbarui aplikasi ini malah saya disuruh untuk memasukkan sandi lagi dan saat saya sudah masukkan sandi dengan benar justru malah di salahkan mohon di benarkan secepatnya karena saya butuh aplikasi ini untuk belajar"))
print(senti.main("muka kamu yang jelas membuat aplikasi kontol temanku tidak isi buka putih papan i loh ingin"))
print(senti.main("bagus aplikasi"))
print(senti.main("oke saya paham bagaimana cara mengatasi microsoft teams saat eror mudah tinggal hapus mengunduh lagi"))
print(senti.main("istimewa"))
print(senti.main("bagus"))
print(senti.main("bagus sih iya sudah sih"))
print(senti.main("woy kenapa saya tidak bisa masuk pertemuan anjing"))
print(senti.main("mantap"))
print(senti.main("notifikasi tugasnya mengganggu mohon dihilangkan saja untuk kepentingan bersama"))
print(senti.main("mantap"))
print(senti.main("bagus"))
print(senti.main("enak untuk sekolah tapi kadang eror aplikasi nya tautan nya tidak bisa di buka"))
print(senti.main("ini teams habis di perbarui malah semakin ribet tidak bisa buka tugas"))
print(senti.main("buruk"))
print(senti.main("padahal jaringan bagus tetap tidak bisa masuk"))
print(senti.main("saya sangat suka"))
print(senti.main("bagus"))
print(senti.main("aplikasi nya bagus"))
print(senti.main("besar"))
print(senti.main("kalau perbarui tidak dikondisikan tiba tiba perbarui sendiri jadi telat saya padahal harus sudah di temu dari pagi"))
print(senti.main("saya sangat suka teams karena saya sekolah menggunakan teams"))
print(senti.main("malas nih aplikasi tidak membuat enak saat lagi bebas terus notifikasi tugasnya lama muncul jadi harus cek manual informasi notifikasi kalau ada pemberitahuan saja lambat muncul"))
print(senti.main("kecewa sangat semakin perbarui semakin jelek kalau tidak karena tugas sekolah tidak mau saya mengunduh ini aplikasi"))
print(senti.main("saya sangat tertarik karena ini sesuatu yang baru bagi saya"))
print(senti.main("aplikasi tidak guna kalau tidak corona tidak akan saya mengunduh ini aplikasi yang ada banyak tugas diunduh juga lama banget anjing habis gb saya mengunduh ini aplikasi tidak bisa terpasang juga"))
print(senti.main("mantap atasan juragan"))
print(senti.main("boros kuota data terpaksa saya mengunduh karena pelajaran saja"))
print(senti.main("nomor komentar"))
print(senti.main("bagus"))
print(senti.main("tolong tolong untuk bisa di pasang"))
print(senti.main("sangat bagus untuk melakukan kegiatan belajar bersama dalam aplikasi ini"))
print(senti.main("kalau perbarui akunya sering hilang"))
print(senti.main("oke"))
print(senti.main("tidak ramah bintang"))
print(senti.main("sudah dikasih nomor telepon tidak dikasih kode bagaimana sih"))
print(senti.main("tersendat mulu kalau sedang digunakan"))
print(senti.main("teams tolong eror saat membuka tugas segera di perbaiki memuat nya lama padahal internet saya cepat kok tidak bisa munculkan tugas sih tolong saya sering mengalami ini saat tenggat waktu segera di perbaiki"))
print(senti.main("crot"))
print(senti.main("microsoft teams nya bagus tapi kadang suka tidak bisa kalau untuk mengirim tugas padahal sinyal saya bagus² saja tolong diperbaiki nanti saya beri bintang"))
print(senti.main("mohon maaf aplikasi nya tidak bisa di mengunduh saya sudah coba berkali kali padahal ram ponsel saya masih banyak"))
print(senti.main("sandinya"))
print(senti.main("kok tidak bisa di mengunduh ya"))
print(senti.main("aplikasi ini sangat penting untuk saya"))
print(senti.main("terbaik"))
print(senti.main("aplikasi ini sangat baik sekali"))
print(senti.main("tidak bisa masuk sama sekali"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("sangat baik"))
print(senti.main("ini aplikasi adalah sangat membantu apa lebih kapan ini situasi i zoom masuk the aplikasi sangat dengan lembut"))
print(senti.main("aplikasi tidak jelas"))
print(senti.main("mewakili aspirasi"))
print(senti.main("oke"))
print(senti.main("keren"))
print(senti.main("oke"))
print(senti.main("oke"))
print(senti.main("lengkap dan mudah digunakan sangat membantu"))
print(senti.main("sangat membantu"))
print(senti.main("bagus untuk sekolah"))
print(senti.main("kadang ada channel tertentu yang hanya memuat sedangkan channel yang lain aman aman saja"))
print(senti.main("bagus untuk dijadikan metode belajar ketika daring"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("sebenarnya aplikasi ini bagus dan banyak fitur nya tapi masih banyak kekurangan yaitu padahal sinyal di rumah saya bagus tapi kok selalu tidak ada tidak bisa masuk rapat ya padahal saya bisa pakai aplikasi lain dan kenapa saya tidak bisa mengirim pesan ya padahal tugas saya ada banyak yang harus dikumpulkan tolong diperbaiki"))
print(senti.main("tersendat mulu tapi itu bagus"))
print(senti.main("habis diperbarui jadi seperti kontol"))
print(senti.main("terima kasih"))
print(senti.main("keren"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("istimewa"))
print(senti.main("bermasalah mulu tiap masuk rapat"))
print(senti.main("bagus"))
print(senti.main("mantap"))
print(senti.main("tiap mau memuat sekarang lelet parah sudah diulang beberapa kali tetap saja mutar terus"))
print(senti.main("mantap"))
print(senti.main("cepat dan baik"))
print(senti.main("sangat membantu untuk pertemuan dan chat"))
print(senti.main("sangat membantu"))
print(senti.main("bagus"))
print(senti.main("lelet banget anjing padahal baru beli paketan bangsat aplikasi teams tidak bisa cekatan mana tugas banyak dan kadang himbauan kalau ada tugas tidak adik beri tahu padahal aku cek masuk tiap hari tiba sudah ada tugas yang sudah di kirim hari yang lalu bangsat tidak aku saja tapi teman pada curhat begitu juga tolong ya ini untuk belajar tidak mengerjakan tugas di bilang malas padahal aplikasinya anjing"))
print(senti.main("mantap"))
print(senti.main("sangat bagus"))
print(senti.main("bagus sekali"))
print(senti.main("bagus"))
print(senti.main("serius tanya ini kenapa memuat nya lama banget ya pada saat buka tautan pertemuan dan ketik nama lalu tekan ikuti rapat kenapa tidak masuk juga ya dan sudah coba masuk pakai nomor ponsel dan email pun sama saja pada saat mau gabung tautan pertemuan aplikasi teams nya memuat nya lama alhasil saya tidak bisa gabung pertemuan ini sudah hari berturut sudah coba pasang ulang tapi tetap sama saja huft"))
print(senti.main("coba dulu"))
print(senti.main("lama banget saat proses memasang"))
print(senti.main("tidak bisa untuk buka tugas"))
print(senti.main("mohon informasinya mengapa saat daftar dan memasukkan nomor ponsel sms kodenya tidak ada terus ya padahal nomor ponsel sudah benar"))
print(senti.main("parah ribet tidak memudahkan pekerjaan malah membuat masalah baru terkait masuk yang mutar mutar saja neraka berdarah"))
print(senti.main("oke bozo"))
print(senti.main("kenapa ih di aku mah belum bisa pakai latar belakang padahal ini sudah versi terbaru tidak bisa di diperbarui lagi"))
print(senti.main("sangat membantu"))
print(senti.main("memang bisa untuk teleponan tapi berat banget awalnya puluhan mbak saja tapi setelah di mengunduh hu gb sendiri jadi tambah berat aplikasi lainnya walaupun ponsel ku tidak kentang"))
print(senti.main("aplikasinya jadi lelet tidak bisa membuat teams baru kadang keluar sendiri"))
print(senti.main("sangat edukatif aplikasi keren untuk komunikasi virtual direkomendasikan aplikasi"))
print(senti.main("mengunduh saja tidak bisa bagaimana kalau pakai"))
print(senti.main("mantap jiwa"))
print(senti.main("baik"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("tolong segera di perbaiki aplikasi ini agar tidak kebanyakan eror dan sebagainya"))
print(senti.main("saya sangat puas"))
print(senti.main("belum di coba"))
print(senti.main("aplikasi ini sangat membantu saya dalam pelajaran sekolah"))
print(senti.main("sangat baik dan bagus"))
print(senti.main("bagus"))
print(senti.main("sangat bagus"))
print(senti.main("ini tidak bisa di mengunduh kenapa ya sudah coba berkali kali tetap tidak bisa padahal aplikasi lain bisa"))
print(senti.main("aku bisa belajar dengan baik"))
print(senti.main("aplikasi sering eror"))
print(senti.main("lumayan jernih untuk tampilan sharescreen dan panggilan untuk saran dikembangkan lagi aplikasi agar lebih berguna dan ramah pengguna"))
print(senti.main("pusing karena ini"))
print(senti.main("punya masalah hidup apa sih susah banget di mengunduh sudah mengunduh susah banget masuk kenapa deh"))
print(senti.main("saya cinta tugas"))
print(senti.main("lengkap dan praktis cukup kembangkan lagi keringanan operasionalnya dan di lengkapi fitur nya"))
print(senti.main("sangat membantu"))
print(senti.main("maksudnya bagaimana sih mengisi sandi begini salah begitu salah ini mah bukan mempermudah namanya malah mempersulit umat manusia tahu tidak bodoh memang"))
print(senti.main("mantap"))
print(senti.main("kadang bisa kadang tidak"))
print(senti.main("keren"))
print(senti.main("sangat bagus"))
print(senti.main("mantap"))
print(senti.main("uas saya tidak terkirim"))
print(senti.main("tidak bisa di mengunduh"))
print(senti.main("bagus"))
print(senti.main("mantap tapi sering memuat nya lama"))
print(senti.main("sangat bagus dan membantu"))
print(senti.main("aplikasi microsoft teams memiliki lebih banyak fitur dibanding aplikasi daring pertemuan lainnya ini akan sangat membantu pengguna mengatur rapat dengan baik tapi di sisi lain aplikasi ini memerlukan kualitas internet yang baik selain itu aplikasi ini mengunduh lebih banyak data ketika digunakan microsoft teams lebih cocok digunakan oleh pengguna di daerah perkotaan dengan kualitas internet yang memadai"))
print(senti.main("kecewa parah sandi nya ke ganti sendiri fitur tambah jelek karena perbarui dan saya jadi tidak bisa mengirim tugas padahal saat itu saya sedang ulangan jadi karena tidak bisa masuk masih teams saya di beri nilai di apa kah itu yang di sebut aplikasi bijak"))
print(senti.main("bagus satu"))
print(senti.main("mantap aplikasi nya"))
print(senti.main("bagus ya"))
print(senti.main("sangat bagus"))
print(senti.main("lama sekali"))
print(senti.main("kok saya selalu keluar dari teams nya"))
print(senti.main("oke banget"))
print(senti.main("aplikasi ini sangat membantu terima kasih"))
print(senti.main("oke"))
print(senti.main("guru saya tidak sih banyak pelajaran"))
print(senti.main("semenjak saya kenal dengan microsoft teams percintaan saya hancur"))
print(senti.main("tolong ini kenapa setelah di perbarui malah tidak ada notifikasi untuk tugas saya jadi nya suka telat untuk mengumpulkan tugas sekolah karena tidak ada notifikasi nya tolong di perbaiki kembali supaya tidak menyusahkan orang banyak terima kasih"))
print(senti.main("sangat membantu"))
print(senti.main("masih ter simpan tidak taunya"))
print(senti.main("bagus"))
print(senti.main("di saat pandemi ini aplikasi teams sangat membantu dalam mengirim tugas maupun materi"))
print(senti.main("bagus"))
print(senti.main("kenapa jadi memuat tidak bisa dipakai untuk pertemuan padahal awal bagus"))
print(senti.main("baik"))
print(senti.main("sangat bagus"))
print(senti.main("oke"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("mantap"))
print(senti.main("bagus"))
print(senti.main("aplikasi tolol"))
print(senti.main("itu sangat bermanfaat"))
print(senti.main("sangat bermanfaat"))
print(senti.main("saat mengunggah lampiran langsung keluar aplikasi itu apa penyebab nya"))
print(senti.main("kepada pihak admin untuk menambahkan fitur kuis dan ruang kelas yang bisa digunakan untuk menampilkan materi pelajaran"))
print(senti.main("tidak direkomendasikan"))
print(senti.main("sudah bagus"))
print(senti.main("tidak bisa verifikasi nomor telepon saya sudah menunggu kode verifikasi minggu tapi hasilnya nihil saya yakin nomor saya sudah benar"))
print(senti.main("karena aplikasi cocok untuk daring"))
print(senti.main("mantab"))
print(senti.main("oke"))
print(senti.main("sangat membantu"))
print(senti.main("oke"))
print(senti.main("tidak bisa menerima verifikasi nomor telepon"))
print(senti.main("semakin lebih di tingkatkan lagi untuk pengguna ramah nya"))
print(senti.main("itu buruk membingungkan dan lebih"))
print(senti.main("tidak jelas asyik"))
print(senti.main("bagus"))
print(senti.main("tugasnya tidak muncul woy"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("tidak bisa masuk padahal email sudah betul"))
print(senti.main("aplikasi bodoh tidak bisa buka tugas alhasil jam sampai sekarang jam belum bisa rusak saya belum bisa mengumpulkan tugas sampai sekarang tanggung jawab memuat terus terus ujung ujung nya tidak dapat memuat tugas coba lagi coba lagi mata bapak kau sudah pakai sumber data kuota sama wifi sama saja memperbaiki"))
print(senti.main("keren"))
print(senti.main("aplikasi nya jelek memuat sejam"))
print(senti.main("kalau mau lihat tugas kadang suka memuat lama padahal jaringan bagus saja terus kenapa sekarang harus daftar ulang lagi waktu memasukkan sandi salah mulu padahal deh benar"))
print(senti.main("baik"))
print(senti.main("tidak bisa log masuk mau konfirmasi nomor telepon tidak bisa selalu salah"))
print(senti.main("oke"))
print(senti.main("oke lah sangat membantu"))
print(senti.main("bagus"))
print(senti.main("daftarnya harus verifikasi nomor ponsel tapi tidak ada kode yang di kirim kakak ponsel maunya bagaimana sih"))
print(senti.main("unduh aplikasi terus tapi tidak bisa buka"))
print(senti.main("bagus"))
print(senti.main("keren"))
print(senti.main("aplikasinya sangat bagus"))
print(senti.main("tiap buka tugas keluar mulu saya kecewa dengan ini aplikasi"))
print(senti.main("aplikasi rumit sudah verifikasi masih saja tidak bisa log masuk"))
print(senti.main("aplikasi lelet asli nya penyimpanan menggunakan gb jangan tertipu mending kelas saja jangan unduh protes saja"))
print(senti.main("oke"))
print(senti.main("tidak pernah bisa mengirim kode verifikasi"))
print(senti.main("tugas tidak ramah bintang"))
print(senti.main("ribet susah masuknya tidak ada respon lambat"))
print(senti.main("oke"))
print(senti.main("baik"))
print(senti.main("selesai diperbarui kenapa tidak bisa dibuka tolong dong"))
print(senti.main("bagus"))
print(senti.main("tidak bagus"))
print(senti.main("bagus"))
print(senti.main("mantap"))
print(senti.main("mantap"))
print(senti.main("bagus"))
print(senti.main("bagus aplikasi"))
print(senti.main("kadang suka lelet"))
print(senti.main("aplikasinya bagus"))
print(senti.main("keren"))
print(senti.main("sangat mudah di gunakan dan sangat membantu"))
print(senti.main("sama teams"))
print(senti.main("sederhana dan keren sekali"))
print(senti.main("fitur aplikasi ini lebih lengkap dan sangat membantu jujur saya lebih merasa nyaman dalam pelajaran daring kemarin dengan aplikasi ini daripada aplikasi sebelah tetapi kembali dengan pendapat masing masing terima kasih"))
print(senti.main("bagus"))
print(senti.main("sangat membantu"))
print(senti.main("sangat membantu untuk daring"))
print(senti.main("bagus aplikasi"))
print(senti.main("oke"))
print(senti.main("habis perbarui eror"))
print(senti.main("eror terus anjinglah paksa tutup segala macam apalagi saat buka aplikasinya"))
print(senti.main("tidak suka banget kenapa aplikasinya tiap masuk selalu keluar sendiri mana memuat tugasnya lama padahal sudah memakai wifi kalau begini saya mencari nilai bagaimana ikut konferensi video saja suka tersendat sendat suaranya eh tong ini pakai paketan sudah diisi kemarin masih saja begitu teams kalau kemusuhan sama saya bilang dong sebal deh"))
print(senti.main("jelek banget memuat lama"))
print(senti.main("teams saya kenapa ya padahal saya sudah memasukkan sandi yang tepat tolong bantuannya dong"))
print(senti.main("baik"))
print(senti.main("belum bisa masuk"))
print(senti.main("semakin kesini semakin tidak maksimal repot pertemuan lagi berjalan tiba log keluar"))
print(senti.main("susah untuk di buka"))
print(senti.main("oke karena eror nya sudah tidak ada jadi saya kasih bintang"))
print(senti.main("bagus tapi sering eror"))
print(senti.main("bagus aplikasi"))
print(senti.main("oke"))
print(senti.main("tersendat"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("lumayan untuk mengunggah tugas ulangan ujian"))
print(senti.main("bagus untuk pertemuan dengan guru"))
print(senti.main("mantap"))
print(senti.main("oke"))
print(senti.main("tolong kenapa tidak bisa di mengunduh"))
print(senti.main("selalu mendapat tolak sementara mencoba untuk masuk"))
print(senti.main("mantap"))
print(senti.main("mantap"))
print(senti.main("ini tidak akan terpotong saat saya sedang daring kelas"))
print(senti.main("saya suka aplikasi ini untuk tugas daring tapi aplikasi ini memangnya tidak ada notifikasi"))
print(senti.main("bagus saya jadi bisa sekolah dan bertemu dengan teman²"))
print(senti.main("memuat padahal sinyal bagus lebih bagus teams yang dulu"))
print(senti.main("aplikasi ini sangat bagus dan menarik"))
print(senti.main("hebat"))
print(senti.main("tolong ini kenapa setiap mengunduh tidak bisa terus"))
print(senti.main("sangat membantu"))
print(senti.main("kenapa jadi bermasalah begini sih tolong segera di perbaiki"))
print(senti.main("mantap untuk belajar daring"))
print(senti.main("bagus untuk melakukan pertemuan virtual bisa di transfer juga dari laptop ke ponsel atau sebaliknya tanpa perlu masuk dari awal ini sangat memudahkan panggilan juga stabil"))
print(senti.main("kegiatan daring menjadi lebih mudah"))
print(senti.main("bagus sih aplikasinya terutama untuk pembelajaran daring sudah bagus banget tapi kadang masih suka eror waktu mau bergabung temu tidak bisa bisa itu yang tolong diperbaiki"))
print(senti.main("bagus"))
print(senti.main("kadang putus menyambung suara nya"))
print(senti.main("bagus"))
print(senti.main("keren"))
print(senti.main("bagus untuk belajar"))
print(senti.main("ini hebat"))
print(senti.main("perbarui aplikasinya sungguh menyusahkan ya bunda kadang kala mesti masuk ulang saya sekolah kadal untung lah saya masih ingat tuh kata sandi tidak usah ada begininya bisa tidak"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("banyak eror notifikasi tugas"))
print(senti.main("semenjak pasang aplikasi ini jadi sering ada notifikasi tugas"))
print(senti.main("aplikasi ini sangat bagus aplikasi ini saya pakai untuk sekolah istimewa"))
print(senti.main("bagus"))
print(senti.main("kenapa tidak bisa di gunakan ya"))
print(senti.main("aplikasi ini menyusah kan dan membuat stres anak sd mau pun kuliah semoga di hapus cepat"))
print(senti.main("sangat membantu"))
print(senti.main("aku sih menjadi saja mungkin yang baca saja sih banyak yang tidak mengerti apa maksud dari tujuan aplikasi teams ini"))
print(senti.main("oke"))
print(senti.main("fitur lebih menarik akses lebih mudah dimengerti"))
print(senti.main("terbaik aplikasi untuk pertemuan"))
print(senti.main("aplikasi terbaik buatan lawannya steve pekerjaan"))
print(senti.main("sering keluar terutama kalau ada perbarui menghambat banyak eror"))
print(senti.main("mantap"))
print(senti.main("diunduh saja susah ampun ampun"))
print(senti.main("bagaimana ini susah menerima kode verifikasi"))
print(senti.main("ih teams nya kenapa sih deh pasang berkali kali tidak bisa bisa"))
print(senti.main("bagus"))
print(senti.main("kalau mau perbarui banyak banget sampai tidak bisa mengerjakan ph"))
print(senti.main("bagus sayang"))
print(senti.main("tidak mendapat kode sms"))
print(senti.main("hebat"))
print(senti.main("aplikasi lelet mutar mutar seperti gasing padahal jaringan bagus kebanyakan masalah gagal tes daring cuma gara gara aplikasi tahi ini bangsat"))
print(senti.main("memudahkan kita untuk suatu pertemuan jarak jauh mengingat musimnya pandemi covid maka sangat cocok untuk yang melakukan kegiatan pertemuan terima kasih"))
print(senti.main("kok keluar mulu harus memasukkan sandi lagi aku lupa sandi ya"))
print(senti.main("mantap"))
print(senti.main("mantap"))
print(senti.main("membuat saya masuk bk"))
print(senti.main("oke"))
print(senti.main("buang kuota saja gagal pasang terus berulang tong diperbaiki lagi"))
print(senti.main("bagus"))
print(senti.main("payah tidak bisa masuk rapat"))
print(senti.main("oke banget"))
print(senti.main("baru mau mencoba semoga bagus dan tidak tunda sinyalnya"))
print(senti.main("maaf kok tidak bisa di mengunduh"))
print(senti.main("di mengunduh saja tidak bisa"))
print(senti.main("bagus pekerjaan"))
print(senti.main("aplikasi kontol"))
print(senti.main("bagus banget aplikasinya"))
print(senti.main("eeeee maimunah jamal kamong pun aplikasi ini membuat kantong dapat fitnah dari guru² mau kirim tugas saya nanti bisa mikrofon saya biasa gangguan jaringan putus² baru makan memori lain anjing memori masih lain makan baterai kalau lapar itu ya makan babi jangan memori deng baterai kurang ajar tapi bagus lain kamong anjing membuat para saya yang pakai ponsel bisa para ganti latar belakang danke banya² lain"))
print(senti.main("bagus"))
print(senti.main("kenapa sih selalu keluar terus untuk masuk"))
print(senti.main("bagus"))
print(senti.main("tidak jelas tiba tiba meminta kata sandi"))
print(senti.main("nyaman untuk mengerjakan tugas"))
print(senti.main("bagus"))
print(senti.main("sangat membantu dalam pekerjaan"))
print(senti.main("berguna aplikasi dan stabil"))
print(senti.main("oke"))
print(senti.main("sok jauh sok bagus"))
print(senti.main("hehe ini bagaimana aku terjebak memuat maksudnya bagaimana"))
print(senti.main("bagus"))
print(senti.main("terima kasih sangat bermanfaat"))
print(senti.main("teams sangat membantu kami untuk mengerjakan tugas anak sekolah"))
print(senti.main("saya kecewa berat karena saat saya mau kirim tugas malah keluar terbodoh aplikasi dari semua waktu idiot lagi"))
print(senti.main("masuk bisa giliran mau rapat teams tidak bisa mengecewakan"))
print(senti.main("kebanyakan gagal nya dari pada berhasilnya saat masuk masa nomor telepon salah terus aneh"))
print(senti.main("oke bagus"))
print(senti.main("plus nya bisa merekam sendiri tanpa izin tuan rumah"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("bagus membantu sekali"))
print(senti.main("aplikasinya bagus"))
print(senti.main("banyak eror notifikasi tidak keluar tab tidak bisa di buka tidak ada tombol gabung webinar sangat mempersulit"))
print(senti.main("bagus"))
print(senti.main("lumayan"))
print(senti.main("bagus membantu belajar mengajar tetapi terkadang susah untuk di buka semoga selalu membantu kami terima kasih"))
print(senti.main("luar biasa"))
print(senti.main("mantap aplikasinya"))
print(senti.main("suka"))
print(senti.main("the terbaik"))
print(senti.main("sangat nyaman untuk belajar"))
print(senti.main("bagus"))
print(senti.main("suka banget puas sudah"))
print(senti.main("sempurna"))
print(senti.main("suka banget eror notifikasi tolong di benarkan"))
print(senti.main("keren"))
print(senti.main("rapat"))
print(senti.main("sangat membantu khusus nya dalam komunikasi dengan tim kerja"))
print(senti.main("oke"))
print(senti.main("wah bagus sekali saya bisa sekolah dengan aplikasi ini btw jika kalian memuat lama coba refresh atau nyala matikan koneksi"))
print(senti.main("bagus"))
print(senti.main("mau mengunduh saja lama banget tertunda terus sedih"))
print(senti.main("astaghfirullah hal azim mau mengunduh aplikasi ini saja susahnya meminta ampun padahal memori masih muat banyak kuota banyak sinyal bagus tapi mutar terus sudah jam saya mengunduh belum bisa"))
print(senti.main("waduh teams ini kenapa tiap perbarui ada masalah harus verifikasi mulu tapi dipersulit ada juga saat perbarui aku masuk ada memuat semua siap yang gambar kolom chat buah dan memuat lama sekali dan bisa dibilang tidak jalan sama sekali sepertinya pengembang kamu ini kenapa jadi kena amuk guru tahu karena kelamaan mohon diperbaiki semoga kedepannya lebih baik lagi jujur saya lebih suka sama teams yang versi juli itu nah pengembang lebih enak itu sama saya harus lepas pasang agar masuk"))
print(senti.main("bagus tapi kadang tersendat tapi tidak papa bagus untuk tugas dikasih jadwal saat mengumpulkan tugas di tekan keluar gambar imut"))
print(senti.main("sangat berguna"))
print(senti.main("bagus"))
print(senti.main("oke banget"))
print(senti.main("keren"))
print(senti.main("luar biasa"))
print(senti.main("aplikasi nya bagus kalau di untuk sekolah tapi kadang tidak merespon makannya aku kasih bintang"))
print(senti.main("mantap"))
print(senti.main("pembelajaran yang sangat disiplin"))
print(senti.main("oke"))
print(senti.main("keren"))
print(senti.main("woy ini kenapa masih teams saya tidak bisa di buka katanya suruh kasih pas sudah ku kasih itu pas nya terus gagal padahal saya ada tugas sekolah dikumpulkan hari ini"))
print(senti.main("aplikasinya bagus sangat membantu perkuliahan"))
print(senti.main("setiap masuk panggilan suara dan gambarku tidak ada jadi kecewa banget dimarahi sama guru deh padahal jaringan lancar tolong diperbaiki ya"))
print(senti.main("bagus meski kompatibilitas perangkat harus ditingkatkan"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("oke"))
print(senti.main("terbaik"))
print(senti.main("suka"))
print(senti.main("aplikasi tidak berbobot robek lama"))
print(senti.main("oke"))
print(senti.main("aplikasi yang bagus untuk pembelajaran dan sangat membantu untuk pembelajaran jarak jauh selain itu dilengkapi dengan berbagai aplikasi lain dari microsoft terima kasih"))
print(senti.main("tidak bisa masuk akun sekolah katanya tidak bisa membantu masuk mana lagi temu penting"))
print(senti.main("mengecewakan setiap mau lihat tugas selalu berhenti di memuat sudah hapus cache jaringan lancar bahkan sampai pasang ulang aplikasi tetap saja selalu berhenti memuat"))
print(senti.main("tidak bisa di pasang pasti keluar tulisan kirim saran dan oke"))
print(senti.main("bagus kali ini aplikasi"))
print(senti.main("sinyal lancar setiap buka tugas dan bergabung temu selalu kendala padahal salah satu aplikasi yang diandalkan pemerintah tapi kinerja nya lamban"))
print(senti.main("bagus"))
print(senti.main("gagal interview gara gara ini aplikasi koneksi internet bagus kok tapi tidak bisa masuk terus kesalahan saat masuk terus"))
print(senti.main("kenapa sih harus masuk ulang terus aplikasi tidak sehat"))
print(senti.main("tolong perbaiki eror ini masak kalau mau mengunggah tugas tidak bisa harus ke laptop dulu dan saya tidak punya laptop jadi nilai saya kosong"))
print(senti.main("aplikasi paling buruk sudah masuk malah salah email mulu kirim tautan verifikasi terus tidak tidak bisa masuk"))
print(senti.main("tidak jelas aplikasinya mau ikut pertemuan malah tidak bisa nomor salah lah kode salah lah padahal nomor email benar semua mengecewakan"))
print(senti.main("microsoft teams sering paksa tutup tolong di benarkan"))
print(senti.main("tingkatkan lagi kualitas untuk penerimaan audio video"))
print(senti.main("memuat terus padahal kuota ada"))
print(senti.main("kalau belum di perbarui tidak bisa mengumpulkan menyebalkan anjing"))
print(senti.main("masuk"))
print(senti.main("oke banget"))
print(senti.main("bagus pekerjaan"))
print(senti.main("kamu keluhan karena apa saya untuk sekolah tiba tiba keluar sendiri"))
print(senti.main("bagus aplikasi ini bisa belajar bersama teman teman walaupun pembelajaran jarak jauh"))
print(senti.main("lumayan untuk aplikasi nya zoom nya juga bagus cuma agak eror sedikit di zoom nya jadi kalau semuanya sudah bagus"))
print(senti.main("sangat membantu"))
print(senti.main("kok jadi begini sih waktu temu tiba keluar sendiri"))
print(senti.main("berat dan lelet chat di teams ponsel tidak muncul semua seperti di versi desktop"))
print(senti.main("mantap"))
print(senti.main("susah tidak bisa ikut gabung di pertemuan"))
print(senti.main("benar benar buruk untuk perbarui data diri dan masuk tidak bisa terus padahal akun nya penting tapi tidak bisa masuk terus mohon perbaikan segera"))
print(senti.main("sangat mudah menggunakannya"))
print(senti.main("mantan"))
print(senti.main("bagus sekali"))
print(senti.main("bagus"))
print(senti.main("aplikasi yang hebat dan sangat membantu"))
print(senti.main("mantap"))
print(senti.main("ini aplikasi khusus sekolah"))
print(senti.main("sangat membantu sekali"))
print(senti.main("memudahkan untuk belajar daring"))
print(senti.main("aplikasi keren"))
print(senti.main("bagus sekali bisa terdokumentasi hasil pertemuan atau koordinasi"))
print(senti.main("mantap"))
print(senti.main("aplikasinya membantu sekali untuk sekolah"))
print(senti.main("aplikasi sangat membantu sekali"))
print(senti.main("mantap tapi kok kadang kadang tidak ada notifikasi ya cara mengatasinya bagaimana"))
print(senti.main("sangat membantu dalam proses pembelajaran"))
print(senti.main("mantap"))
print(senti.main("susah terbuka"))
print(senti.main("oke"))
print(senti.main("oke"))
print(senti.main("aplikasi bagus"))
print(senti.main("cuma lelet banget seperti internet nya di sedot paksa"))
print(senti.main("saya coba menggunakan microsoft teams untuk kegiatan rapat dengan kolega perusahaan kami"))
print(senti.main("ini gangguan"))
print(senti.main("sangat bagus"))
print(senti.main("keren"))
print(senti.main("tolol nge tersendat"))
print(senti.main("sangat baik"))
print(senti.main("susah untuk tersambung dan pasang"))
print(senti.main("memuat nya lama banget anjing perbarui terus tapi masalah jaringan tidak di betulkan"))
print(senti.main("halo saya sudah mengunduh tapi tidak bisa daftar notifikasi lewat sms tidak datang ada yang sama"))
print(senti.main("aplikasinya sudah bagus tapi kenapa banyak tugas"))
print(senti.main("oke"))
print(senti.main("ya ponsel kamu yang kentang"))
print(senti.main("bagus"))
print(senti.main("saya pakai ini direkomendasikan memang relasi kita juga wajib pakai ini"))
print(senti.main("sering pertemuan"))
print(senti.main("sangat bermanfaat bagi saya"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("ini aplikasinya tolong diperbaiki karena saya internet nya bagus kuota masih ada tapi tidak bisa dibuka dan muncul tulisan tidak ada internet padahal jelas internet ada dan bagus tolong segera diperbaiki karena ini sangat mengganggu"))
print(senti.main("capek saya"))
print(senti.main("sangat membantu dalam kuliah daring"))
print(senti.main("bagus"))
print(senti.main("rekaman nya bisa diputar ulang dan chat obrolannya tidak hilang saat kita langsung"))
print(senti.main("bagus"))
print(senti.main("jelek"))
print(senti.main("soalnya kalau ada aplikasi kasih ini jadi mengirim tugas tidak rumit"))
print(senti.main("mantap"))
print(senti.main("aplikasi sangat membingungkan"))
print(senti.main("mantap"))
print(senti.main("sangat membantu"))
print(senti.main("tugas banyak"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("membantu"))
print(senti.main("ketika guru saya memulai temu tidak ada tautan untuk masuk seperti biasanya sebelum perbarui sehingga saya dianggap tidak masuk saya kira cuma saya doang ternyata teman teman saya juga tolong di perbaiki eror nya dong"))
print(senti.main("lumayan untuk yang bisa integrasikan banyak aplikasi"))
print(senti.main("oke"))
print(senti.main("oke"))
print(senti.main("saja dulu padahal saya tidak tahu"))
print(senti.main("sangat baik"))
print(senti.main("sangat bagus sekali"))
print(senti.main("mantap"))
print(senti.main("i tidak sengaja tipe the salah kode untuk bergabung a teams dan sekarang i tidak bisa pergi lol"))
print(senti.main("mantap"))
print(senti.main("oke"))
print(senti.main("kebanyakan perbarui doang membuat berat memori"))
print(senti.main("sangat membantu dan mudah digunakan"))
print(senti.main("permainan nya tidak rapi"))
print(senti.main("bagus"))
print(senti.main("sangat bagus sekali"))
print(senti.main("bagus"))
print(senti.main("aplikasi paling busuk lagi pertemuan tiba keluar sendiri mau log masuk saja susah banget katanya salah nama lah email lah nomor telepon lah kirim kode verifikasi saja seabad"))
print(senti.main("halo saya ibu komando saya zoom selalu menggunakan ini bagi kalian yang sering mengunduh aplikasi ini dan tidak lancar saat menggunakannya mungkin tidak cocok dengan ponsel milik kalian dan terima kasih"))
print(senti.main("bagus lebih pengguna ramah"))
print(senti.main("oke"))
print(senti.main("sering eror"))
print(senti.main("sangat membantu untuk organisasi terima kasih banyak teams"))
print(senti.main("wow"))
print(senti.main("keren"))
print(senti.main("mantap sangat bermanfaat untuk komunikasi dan koordinasi dalam pekerjaan"))
print(senti.main("bagus sih tapi agak eror aplikasi nya kalau pakai lewat ponsel"))
print(senti.main("masuk ke beda perangkat harus ubah kata sandi ini sangat merepotkan orang yang suka memakai teams di beberapa perangkat tolong dong kalau membuat fitur itu dipikirkan dahulu"))
print(senti.main("bagus untuk anak sama"))
print(senti.main("lelet"))
print(senti.main("bagus"))
print(senti.main("sangat membantu"))
print(senti.main("suka eror karena tidak bisa menjadi tamu selalu meminta masuk akun tetapi selalu gagal"))
print(senti.main("bagus"))
print(senti.main("sangat bagus"))
print(senti.main("bagus"))
print(senti.main("kalau ada temu mbok jangan ada tunda begitu lho"))
print(senti.main("aplikasi haram suka eror"))
print(senti.main("bagus banget"))
print(senti.main("tambahkan opsi keluar tetapi masih di dalam kelas"))
print(senti.main("bagus sekali"))
print(senti.main("sangat bagus"))
print(senti.main("bagus"))
print(senti.main("tidak memadai"))
print(senti.main("aku tidak bisa foto tugasku bagaimana ini aduh aduh"))
print(senti.main("iya menyebut boo kiki terus"))
print(senti.main("aplikasi nya bagus dan cukup baik hanya saja kadang jika saya masuk pertemuan sedikit susah tolong diperbaiki lagi"))
print(senti.main("juga membingungkan untuk gunakan tidak ramah pengguna untuk a pemula permulaan seperti saya"))
print(senti.main("cuma untuk kuliah saja"))
print(senti.main("memudahkan kami dalam memberikan kuis dalam teams mudah dipahami sama siswa"))
print(senti.main("kenapa microsoft teams ini semakin lama susah untuk di akses"))
print(senti.main("disana adalah a eror dimana i log keluar untuk nomor alasan pada semua ku teman teman juga mendapat ini eror tolong perbaiki ini"))
print(senti.main("mutar terus memuat kapan mengumpulkan tugas"))
print(senti.main("pergi"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("bagaimana ya bagus sih tapi kebelakangan ini kalau mau atas foto video lelet padahal jaringan dan paket data aku masih banyak aktif kalau mau ikut temu video begitu keluar sendiri di kasih foto video sama gurunya tidak keluar dan saat mau buka file lelet juga"))
print(senti.main("sangat bagus sekali untuk pertemuan nya memang terbaik untuk pertemuan nya"))
print(senti.main("sangat bermanfaat dan mempermudah dalam bekerja"))
print(senti.main("jelek"))
print(senti.main("sudah ikuti rapat tapi tidak bisa masuk masa kalah sama aplikasi sebelah seperti zoom microsoft boleh kalau membuat aplikasi kantor tapi kalau membuat aplikasi rapat pertemuan daring masih perlu banyak belajar"))
print(senti.main("wow"))
print(senti.main("microsoft teams ini sangat bisa diandalkan untuk rapat sangat bagus bisa ada background dan sangat aman banyak fitur tambahan"))
print(senti.main("tidak bisa masuk"))
print(senti.main("sangat bagus untuk belajar"))
print(senti.main("aplikasi yang sangat terpakai untuk aktivitas pertemuan berbagi file"))
print(senti.main("aplikasi ini bagus¡¡¡¡¡¡¡¡¡¡apa lihat lihat"))
print(senti.main("perbarui"))
print(senti.main("the terbaik"))
print(senti.main("bagus"))
print(senti.main("bagus pekerjaan"))
print(senti.main("oke"))
print(senti.main("bagus sangat membantu"))
print(senti.main("oke"))
print(senti.main("bagus"))
print(senti.main("kadang lelet"))
print(senti.main("baik biru mencoba"))
print(senti.main("semoga bermanfaat"))
print(senti.main("sudah perbarui malah tidak bisa masuk sama sekali kan jadi ketinggalan tutorial daring nya"))
print(senti.main("untuk stres sekaligus melatih keluar"))
print(senti.main("kenapa ya kok saya tidak terima kode otp yang dikirim lewat ponsel tolong bantuannya"))
print(senti.main("bagus banget untuk mengerjakan tugas dan sampai saat ini saya belum ada masalah sama aplikasinya"))
print(senti.main("agak susah"))
print(senti.main("untuk saat ini masih berguna"))
print(senti.main("ini aplikasi membuat jengkel banget mau di perbarui malah tertunda terus setiap buka meminta di perbarui padahal saya pakai aplikasi ini untuk pelajaran sehari hari ketar ketir kalau seperti begini caranya tidak bisa belajar padahal juga sudah aku mengunduh ulang berkali kali tetap tidak bisa di mengunduh bagaimana sih ayo dong di perbaiki lagi sistemnya"))
print(senti.main("jelek"))
print(senti.main("kesal banget aplikasi tolol tidak bisa masuk akun sendiri"))
print(senti.main("bagus"))
print(senti.main("mantap berjalan lancar"))
print(senti.main("sangat membantu dalam proses perkuliahan daring seperti saat ini"))
print(senti.main("bagus aplikasi untuk situasi pandemi covid tolong tingkatkan ui lebih fitur"))
print(senti.main("sangat membantu dan bagus sekali aplikasinya"))
print(senti.main("dari google kelas ruangan pindah ke sini"))
print(senti.main("the aplikasi adalah bagus tapi itu sungguh membantu kapan i ingin untuk buka the channel itu tetap bermain bahkan meskipun ku jaringan adalah bagus tolong perbarui lagi untuk the koreksi terima kasih kamu"))
print(senti.main("itu sok keren"))
print(senti.main("bagus"))
print(senti.main("mantap"))
print(senti.main("sangat berguna sekali microsoft teams ini dapat zoom lagi"))
print(senti.main("sangat bagus"))
print(senti.main("tidak jelas"))
print(senti.main("mau pasang ulang kenapa tidak bisa ini"))
print(senti.main("bagus"))
print(senti.main("tidak bisa di pasang"))
print(senti.main("baik untuk mendukung kerja"))
print(senti.main("saya suka teams ini"))
print(senti.main("bagus sekali"))
print(senti.main("bagus tapi membuat stres terima kasih"))
print(senti.main("kenapa saya tidak bisa masuk ke akun saya padahal sinyal saya bagus"))
print(senti.main("tolong kalau guru dan siswa beda versi microsoft teams tugasnya jangan eror tidak bisa di buka selebihnya tidak buruk"))
print(senti.main("oke"))
print(senti.main("oke"))
print(senti.main("ya betul"))
print(senti.main("terima kasih sangat bermanfaat"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("bagus"))
print(senti.main("aplikasi teams ini membuat saya bisa semua tugas cepatan deh mengunduh aplikasi teams ini pokoknya bagus untuk kalian yang sekolah sampai lulus pokoknya mantap aplikasi teams ini aku suka banget"))
print(senti.main("mempermudah kita dalam alignment dengan teams"))
print(senti.main("oke"))
print(senti.main("kecewa sudah pakai perangkat ponsel oppo dan vivo tapi semuanya tidak bisa mengaktifkan kamera dan speaker"))
print(senti.main("tidak apa apa"))
print(senti.main("sangat jelek setiap mau masuk dari tautan selalu tidak bisa heran"))
print(senti.main("aplikasi tolol internet lancar tidak bisa buka tugas"))
print(senti.main("hmm lumayan"))
print(senti.main("bagus aplikasi"))
print(senti.main("tidak tau"))
print(senti.main("sering tidak ke refresh kalau ada rapat baru sering tidak muncul padahal sudah di perbarui"))
print(senti.main("kalau dibuat buka soal mutar terus kenapa ya"))
print(senti.main("mantap tapi mantap"))
print(senti.main("baik sekali"))
print(senti.main("bagus"))
print(senti.main("bagaimana ceritanya ini perbarui tidak jelas harus memasukkan kode akun google lagi padahal perbarui doang malah memasukkan akun google lagi saya yang membuat akun baru pakai sandi asal asalan ya susah masuk lagi sudah coba berbagai cara masih tidak bisa lagian juga untuk apa memasukkan sandi akun google kamu pikir mau ganti sandi google apa sudah pembaruan nya bertambah bodoh lagi performanya semakin jelek pula"))
print(senti.main("mantap untuk pertemuan"))
print(senti.main("teams kenapa sih setiap mau melakukan apapun saat mau nge temu tiba² layar nya langsung kosong dan selanjutnya kembali lagi dan itu akan terulang lagi kali mau melakukan sesuatu saat temu tolong lah ya"))
print(senti.main("lama memuatnya"))
print(senti.main("kenapa tidak bisa masuk admin"))
print(senti.main("bagus"))
print(senti.main("kenapa aplikasi ini tidak bisa pindah ke kartu sd"))
print(senti.main("ini sangat bagus"))
print(senti.main("benci ini aplikasi juga banyak pekerjaan rumah dan dan sebagainya"))
print(senti.main("kok aplikasi ini tidak bisa untuk mengirim foto sih"))
print(senti.main("bagus"))
print(senti.main("bagus kerja"))
print(senti.main("biasanya mutar terus tapi ada yang tidak mutar tugas nya"))
print(senti.main("bagus"))
print(senti.main("setiap mau mengirim tugas kok lelet ya padahal paketan masih banyak terus wifi juga lancar kalau dipakai aplikasi lain tapi kenapa kalau di teams lelet banget ya tolong lebih baik untuk kedepannya"))
print(senti.main("sangat menarik dan efisien waktu"))
print(senti.main("tidak bisa di unduh anjing"))
print(senti.main("bagus banget aplikasinya untuk sekolah dan kerja kasih bintang deh"))
print(senti.main("oke"))
print(senti.main("bagus sekali"))
print(senti.main("aplikasi ini memang membantu tetapi pada saat membuka file tidak bisa dikunci sehingga tambah pekerjaan lalu mengetik di microsoft kata terlebih dahulu lalu di menyalin terus di taruh dilembar jawab kali kerja tidak langsung di ketik di lembar jawab di sediakan tidak langsung di situ melainkan di microsoft kata terlebih dahulu"))
print(senti.main("bagus"))
print(senti.main("jaringan tidak stabil menggunakan data wifi sering memuat setiap memuat harus hapus data agar bisa cek tugas tugasnya"))
print(senti.main("dipakai dinas pendidikan kota surabaya untuk sekolah sekolah terutama sekolah negeri wajib memakai aplikasi ini"))
print(senti.main("bagus aplikasinya cara belajar anak saya jadi lancar"))
print(senti.main("mantap"))
print(senti.main("parah banget"))
print(senti.main("top banget"))
print(senti.main("keren"))
print(senti.main("itu sudah ku kasih becanda aplikasi nya bagus banget"))
print(senti.main("aplikasinya bagus banget terima kasih microsoft perusahaan telah membuat aplikasi yang cukup berguna bagi semua orang"))
print(senti.main("oke"))
print(senti.main("sangat membantu aplikasi nya"))
print(senti.main("bagus untuk pembelajaran di sekolah"))
print(senti.main("teams tahi"))
print(senti.main("latar belakang belum bisa diganti di ponsel"))
print(senti.main("puas a"))
print(senti.main("saat ada materi kok tidak bisa dibuka mutar terus saat tidak lihat jaringannya ternyata lancar lancar saja tidak ada kendala"))
print(senti.main("aplikasi ini sangat membantu"))
print(senti.main("tiba keluar masukkan sandi saya takut nanti keluar semua dari grup sekolah bagaimana solusinya"))
print(senti.main("kesal banget sudah mengirim tugas malah tiba tiba datang lagi dengan keadaan tugas itu kosong padahal jelas jelas sudah diisi mana itu tugas yang lama banget ada juga tugas yang baru datang malah tidak masuk tugas"))
print(senti.main("bagus fasilitas lengkap mudah digunakan"))
print(senti.main("ya bagus sih tapi aplikasi nya sering eror sendiri"))
print(senti.main("saya senang sekali dengan teams karena saya bisa memantau anak saya dalam mengerjakan tugasnya"))
print(senti.main("sudah tidak bisa di perbarui rusak microsoft teams jelek tidak berguna"))
print(senti.main("bagus"))
print(senti.main("aplikasi kurang bagus lebih baik zoom googlemeet aplikasi ini sangat berat dan bermasalah walau sinyal kuat kalah pekerjaan gara aplikasi ini jelek"))
print(senti.main("saya sang pada bergembira karena bisa bertemu teman teman"))
print(senti.main("bagus banget"))
print(senti.main("aplikasi ini jelek"))
print(senti.main("bagus"))
print(senti.main("oke"))
print(senti.main("kendala aplikasi ini selalu eror mohon bantuannya"))
print(senti.main("mantap"))
print(senti.main("bagus"))
print(senti.main("saya sudah mengumpulkan tugas nya eh tiba tiba tidak ke submit bagaimana sih terus kenapa bukannya lama banget ah tolong diperbaiki dong"))
print(senti.main("bagus aplikasi"))
print(senti.main("keren beda dari yang lain deh"))
print(senti.main("bagus jika bekerja dengan baik dan benar membuka soal memuat terus tidak bisa dibuka mutar terus memuat terus internet saya kencang lihat youtube lancar bagus permainan lancar menjelajah lancar kenapa membuka teams mau mengerjakan tugas tidak bisa kenapa meminta perbarui terus performa tidak tambah bagus"))
print(senti.main("siap sangat membantu untuk tugas sekolah"))
print(senti.main("bagus bisa untuk pertemuan bisa seperti duduk dibioskop terima kasih ya teams ayo segera mengunduh"))
print(senti.main("keren sangat berguna"))
print(senti.main("aplikasi bodoh"))
print(senti.main("bagus aplikasi"))
print(senti.main("aplikasi ini bagus cepat lancar dan sangat baik untuk semua kalangan"))
print(senti.main("memuat terus"))
print(senti.main("sangat bagus untuk saya yang kelas kali terima kasih"))
print(senti.main("sampah"))
print(senti.main("ya saya sangat suka sekali"))
print(senti.main("dia membuat aku seperti masih di dalam kelas bersama teman temanᕙ༼ ᴥ ༽ᕗ"))
print(senti.main("harap bersabar kami sedang berada di microsoft teams sampai menit cuma begitu doang tidak masuk padahal mau pertemuan sungguh keren"))
print(senti.main("sering mengalami memuat lama"))
print(senti.main("bagus"))
print(senti.main("aplikasi nya sangat bagus benaran bagus kok kenapa lihat sampai kebawah"))
print(senti.main("ini kenapa ya saat saya mau mengumpulkan tugas di foto di microsoft teams tidak bisa mohon di perbaiki saya sangat terganggu"))
print(senti.main("aplikasi ini bagus"))
print(senti.main("aplikasi ini bagus untuk sekolah yang di rumah"))
print(senti.main("bagus"))
print(senti.main("sangat membantu"))
print(senti.main("aplikasi yang sangat bagus dan berguna"))
print(senti.main("tuhan karena saya baik saya pasang ini aplikasi untuk daring lagi tapi mengirim tugasnya kok lelet yeah mohon maaf ini ya tugasnya sudah saya kerjakan dari malam lah kok sekarang lelet mungkin saja karena wifi saya belum dibayar eheq"))
print(senti.main("bagus"))
print(senti.main("kok sering memuat ya"))
print(senti.main("bagus jika saya tidak mengerjakan disini"))
print(senti.main("aku bisa panggilan video sama guru ku aplikasi ini sangat sangat hebat jadi aku kasih bintang yeay"))
print(senti.main("bagus"))
print(senti.main("aplikasinya bagus tapi tiap mau dibuka selalu memuat memuatnya baru selesai kalau aplikasi di lepas terus log masuk lagi dan itu berulang kali seperti itu tolong diperbaiki lagi sistemnya"))
print(senti.main("saya rasa ini tidak berkerja saat masuk selalu kesusahan harus begini begini dan lapor guru"))
print(senti.main("aplikasi jelek mau untuk akun saja sulitnya meminta ampun"))
print(senti.main("sangat bagus"))
print(senti.main("sudah dicoba berkali kode tidak kirimkan ke nomor telepon"))
print(senti.main("agak sebal jaringan sedikit jelek tidak bisa dipakai"))
print(senti.main("mantap"))
print(senti.main("aplikasi ini sangat bagus hanya saja terkadang sedikit lambat"))

{'classified_text': 'kenapa jaringan selalu tidak stabil [-4] padahal aplikasi lain stabil [4] terus', 'tweet_text': ' kenapa jaringan selalu tidak stabil padahal aplikasi lain stabil terus', 'sentence_score': ['kenapa jaringan selalu tidak stabil [-4] padahal aplikasi lain stabil [4] terus'], 'max_positive': 4, 'max_negative': -4, 'kelas': 'positive'}
{'classified_text': 'sudah masuk [2] tapi meminta masuk [2] lagi ketika tekan [1] tautan pertemuan', 'tweet_text': ' sudah masuk tapi meminta masuk lagi ketika tekan tautan pertemuan', 'sentence_score': ['sudah masuk [2] tapi meminta masuk [2] lagi ketika tekan [1] tautan pertemuan'], 'max_positive': 2, 'max_negative': -1, 'kelas': 'positive'}
{'classified_text': 'saya tidak bisa [-2] melihat chat di ponsel saya saya sudah seminggu tidak bisa [-2] menggunakan di ponsel ribet [1] kalau harus di laptop', 'tweet_text': ' saya tidak bisa melihat chat di ponsel saya saya sudah seminggu tidak bisa menggunakan di ponsel ribet kalau harus di lap

Hasil labeling sentimen disimpan secara manual kedalam format excel dan menyisakan kolom sentimennya saja.

Sentimen positif : 1

Sentimen negatif : 0